## Importing dependencies.

Will have to install the packages you don't already have, need to download 'chromedriver.exe' here: https://chromedriver.chromium.org/downloads

In [1]:
import requests as rq
from bs4 import BeautifulSoup
import csv
import traceback
from urllib.request import urlopen

import selenium as se
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.common.by import By
#This section allows the selenium driver package to wait until the information is scraped.
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
#Allows us to click buttons
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service

import numpy as np
import random
import locale
from openpyxl import Workbook, load_workbook
from openpyxl.utils import get_column_letter
from time import strptime
import pandas as pd

list=[]
import datetime
from selenium.common.exceptions import NoSuchElementException

#Change the path for your chromedriver download.

PATH = "C:\Program Files (x86)\chromedriver.exe"

from urllib.error import HTTPError
from urllib.error import URLError

#Supressing Deprecated warnings
import warnings

def fxn():
    warnings.warn("deprecated", DeprecationWarning)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    fxn()

In [ ]:
locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')
rows_scraped = []
def scraper_function(row):
    scraped_df = pd.DataFrame(columns=['Manufacturer', 'Model', 'Variant', 'Standard car description','Full Autotrader Vehicle description', 'Number of Miles', 'Difference to average Miles', 'Fuel type', 'Range', 'Body type', 'Engine', 'Gearbox', 'Doors', 'Seats', 'Service history', 'Owners', 'Manufacturer warranty', 'Free road tax', 'Insurance Group', 'Annual Tax', 'CO2 Emissions', 'Urban', 'Extra Urban', 'Combined', '0-60mph', '0-62mph', 'Top Speed', 'Cylinders', 'Valves', 'Engine power', 'Engine torque', 'Height', 'Length', 'Wheelbase', 'Width', 'Fuel tank capacity', 'Gross vehicle weight', 'Boot space (seats down)', 'Boot space (seats up)', 'Minimum kerb weight', 'Max loading weight', 'Driver convenience', 'Safety features', 'Exterior features', 'Interior features', 'Technical features', 'Price'])
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument("--disable-popup-blocking")
    chrome_options.add_argument("user-agent=Mozilla/3.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/59.0.3071.115 Safari/537.36")
    chrome_options.add_argument('--ignore-certificate-errors')
    chrome_options.add_argument('--incognito')
    chrome_options.add_argument('--headless')
    chrome_options.add_argument("--profile-directory=Default")
    chrome_options.add_argument("--window-size=1920,1080")
    #chrome_options.add_argument("--user-data-dir=C:/Users/benwallner5/AppData/Local/Google/Chrome/User Data")
    driver = webdriver.Chrome(PATH, options = chrome_options)
    driver.implicitly_wait(3)

    driver.get("https://www.autotrader.co.uk")
    time.sleep(0.5)
    driver.maximize_window()

    """
    scraper = scrape_car_info(Manufacturer = Manufacturer_Exc, Model = Model_Excel, Variant = Variant_Exc , Body_type = BodyType_Exc, Gearbox = Gearbox_Exc, Primary_Fuel = PrimaryFuelType_Exc, Secondary_Fuel = SecondaryFuelType_Exc, Electric_Vehicle_Type = ElectricVehicleType_Exc, Start_year = StartYear_Exc, End_year = EndYear_Exc, Number_Doors = NumberOfDoors_Exc)
    return scraper
    """
    #Setting excel values equal to Autotrader search values.
    
    #Switches the iframe (different windows) to 'Accept' cookies and close down the window.

    try:
        driver.switch_to.frame("sp_message_iframe_687971")
        WebDriverWait(driver,5).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="notice"]/div[3]/div[2]/button[2]'))).click()
    except:
        pass
    
    try:
        driver.switch_to.frame("sp_message_iframe_687972")
        WebDriverWait(driver,5).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="notice"]/div[3]/div[2]/button[2]'))).click()
    except:
        pass
    #This moves into the 'More options' window to allow the search of specific makes and models.
    
    try:
        WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.LINK_TEXT, 'More options'))).click()
    except:
        return
    
    
    try:
        WebDriverWait(driver,3).until(EC.element_to_be_clickable((By.ID, 'postcode'))).send_keys("RH16EG")
    except:
        return
    
    #This clicks the search button returning cars that match the input values.
    #time.sleep(1)
    
    try: 
        WebDriverWait(driver,3).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[type='submit']"))).click()
    except:
        return
    
    try:
        driver.switch_to.frame("sp_message_iframe_687971")
        WebDriverWait(driver,5).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="notice"]/div[3]/div[2]/button[2]'))).click()
    except:
        pass
    
    try:
        driver.switch_to.frame("sp_message_iframe_687972")
        WebDriverWait(driver,5).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="notice"]/div[3]/div[2]/button[2]'))).click()
    except:
        pass
    #Randomising the selected car
    try:
        num_results = WebDriverWait(driver,3).until(EC.element_to_be_clickable((By.CSS_SELECTOR, '.search-form__count.js-results-count'))).text.split(' ')[0]
        
        full_page_or_not = min(10, locale.atoi(num_results))
        result_numb = random.randint(1,full_page_or_not)
    except:
        return
    
    #time.sleep(1)
    #Clicking on a randomised result on the first page that isn't a leasing advert ('Vanarama'), must have data-label = "search appearance click", this doesn't seem to work?
    
    try:
        WebDriverWait(driver,3).until(EC.element_to_be_clickable((By.XPATH, "(// a[contains(@data-label,\'search appearance click')])[{}]".format(result_numb)))).click()
    except:
        return
    
    #Function for formatting scraped data into dictionaries.
    def Convert(lst):
        res_dct = map(lambda i: (lst[i], lst[i+1]), range(len(lst)-1)[::2])
        return dict(res_dct)
    #Getting basic information that is missing in vehicle file (number of seets, variant/trim specifics etc)
    time.sleep(0.5)
    
    try:
        full_car_description = WebDriverWait(driver,3).until(EC.element_to_be_clickable((By.XPATH,'//*[@id="layout-desktop"]/aside/section[1]/p[1]'))).text.strip() + " " + WebDriverWait(driver,3).until(EC.presence_of_element_located((By.XPATH,'//*[@id="layout-desktop"]/aside/section[1]/span'))).text.strip()+ " " + WebDriverWait(driver,3).until(EC.presence_of_element_located((By.XPATH,'//*[@id="layout-desktop"]/aside/section[1]/p[2]'))).text.strip().split('/n')[0]
    except:
        full_car_description = ''
        pass
    #print(full_car_description)
    if full_car_description == '':
        full_car_description = WebDriverWait(driver,3).until(EC.element_to_be_clickable((By.XPATH,'//*[@id="layout-desktop"]/aside/section[1]/p[1]'))).text.strip() + WebDriverWait(driver,3).until(EC.presence_of_element_located((By.XPATH,'//*[@id="layout-desktop"]/aside/section[1]/p[2]'))).text.strip().split('/n')[0]
    else:
        pass
                                                                                                                                                                                                                                            
    try:
        price = WebDriverWait(driver,3).until(EC.element_to_be_clickable((By.XPATH,'//*[@id="layout-desktop"]/aside/section[1]/div[1]/div[1]/h2'))).text.strip() 
        price_dict = {'Price' : price}
    except:
        price_dict = {'Price' : ''}
        pass

    try:
        NumMiles = WebDriverWait(driver,3).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="layout-desktop"]/article/section[2]/span/span[2]'))).text.strip().split(" ")
        NumMiles_dict = {'Number of Miles' : str(NumMiles[0])}
    except:
        NumMiles_dict = {'Number of Miles' : ''}
        pass
    
    try:
        DiffMiles = WebDriverWait(driver,3).until(EC.element_to_be_clickable((By.XPATH, "//span[@class='atc-type-picanto atc-type-picanto--medium']"))).text.strip().split(" ")
        High_Low = WebDriverWait(driver,3).until(EC.element_to_be_clickable((By.XPATH, "//span[@class='atc-type-picanto']"))).text.strip().split(" ")
        DiffMiles_dict = {'Difference to average Miles' : str(DiffMiles[0]) + " " + str(High_Low[0])}
    except:
        DiffMiles_dict = {'Difference to average Miles' : ''}
        pass
    
    
    time.sleep(0.5)
    
    
    try:
        driver.execute_script("window.scrollBy(0,400)")
    except:
        pass
    time.sleep(0.5)

    try:
        key_attributes = WebDriverWait(driver,3).until(EC.presence_of_all_elements_located((By.XPATH, '//*[@id="layout-desktop"]/article/section[2]/ul/li')))
        key_attribute_dict = {}
        for pair in key_attributes:
            ind_dict = Convert(pair.text.strip().split('\n'))
            key_attribute_dict = key_attribute_dict|ind_dict
        
    except:
        key_attribute_dict = {'Fuel type': '', 'Body type': '', 'Engine': '', 'Gearbox': '', 'Doors': '', 'Seats': '', 'Owners': '', 'Manufacturer warranty' : '', 'Free road tax' : ''}
        pass
    #print(key_attribute_dict)
    
    #Finding the location of Specs on page and scrolling to them. At the minute have just scrolled to the middle of the page but need to find a way to ensure it finds the element.
    #specification_location = driver.find_element(By.XPATH, '//*[@id="layout-desktop"]/article/div[1]/button/span[2]')
    try:
        driver.execute_script("window.scrollBy(0,600)")
    except:
        pass
    time.sleep(0.5)
    
    """
    try:
        num_divs = WebDriverWait(driver,3).until(EC.presence_of_all_elements_located((By.XPATH,'//*[@id="layout-desktop"]/article/div')))
    except:
        pass
    """
    try:
        standard_car_description = WebDriverWait(driver,3).until(EC.presence_of_element_located((By.XPATH,'//*[@id="layout-desktop"]/article/div[5]/h3'))).text.strip().split(' ')
        standard_car_description_val = " ".join(standard_car_description[:-1])
        
    except:
        standard_car_description_val = ''
        pass
    #Running costs/environmental rating.
    try:
        running_costs = WebDriverWait(driver,3).until(EC.element_to_be_clickable((By.XPATH, "//span[normalize-space()='Running costs']")))
        run_info = running_costs.click()
    except:
        pass
    insurance_info_dict={}
    try:
        insurance_info = WebDriverWait(driver,3).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="modal-root"]/div/section/div[2]/div/div[1]/div/div/dl/dd')))
        insurance_info_dict['Insurance Group'] = insurance_info.text.strip()
    except:
        insurance_info_dict = {'Insurance Group' : ''}
        pass
    tax_info_dict={}
    try:
        tax_info = WebDriverWait(driver,3).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="modal-root"]/div/section/div[2]/div/div[2]/div/div/dl/dd[1]')))
        tax_info_dict['Annual Tax'] = tax_info.text.strip()
        
    except:
        tax_info_dict = tax_info_dict['Annual Tax'] = ''
        pass
    try:
        CO2 = WebDriverWait(driver,3).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="modal-root"]/div/section/div[2]/div/div[2]/div/div/dl/dd[2]')))
        tax_info_dict['CO2 Emissions'] = CO2.text.strip()
        
    except:
        tax_info_dict = tax_info_dict['CO2 Emissions'] = ''
        pass
    
    try:
        fuel_info = WebDriverWait(driver,3).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="modal-root"]/div/section/div[2]/div/div[3]/div/div/dl')))
        fuel_info_dict = Convert(fuel_info.text.strip().split('\n'))
    except:
        fuel_info_dict = {'Urban' : '', 'Extra Urban' : '', 'Combined' : ''}
        pass
    #print(fuel_info_dict)
    time.sleep(0.5)
    try:
        WebDriverWait(driver,3).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="modal-root"]/div/section/div[1]/button'))).click()
    except:
        pass
    time.sleep(0.5)
    
    #Scraping the required information for the car specified. These XPATHs may have to be altered by the time the webscraper is run!
    try:
        specifications = WebDriverWait(driver,3).until(EC.element_to_be_clickable((By.XPATH, "//span[normalize-space()='Specification']")))
        spec = specifications.click()
    except:
        car_type = {'Manufacturer' : Manufacturer, 'Model' : Model, 'Full Autotrader Vehicle description' : full_car_description, 'Standard Car description' : standard_car_description_val}
        overall_dict= {**car_type, **NumMiles_dict , **DiffMiles_dict, **key_attribute_dict, **insurance_info_dict, **tax_info_dict, **fuel_info_dict, **price_dict}
        scraped_df = pd.concat([scraped_df, pd.DataFrame.from_dict([overall_dict])], axis = 0)
        scraped_df.to_csv(file_path, index=False, header=False, mode='a') 
        try:
            driver.close()
        except:
            pass

    
    index = 1
    #Extracting performance data from the car searched.
    try:
        WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[data-gui='performance-expander']"))).click()
        time.sleep(0.5)
        performance = WebDriverWait(driver,3).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="modal-root"]/div/section/div[2]/div/ul/li[1]/span/div/ul')))
        performance_info_dict = Convert(performance.text.strip().split('\n'))
        #print(performance_info_dict)
        index+=1
        WebDriverWait(driver,3).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[data-gui='performance-expander']"))).click()
    except:
        performance_info_dict = {'0-60mph' : '', '0-62mph' : '', 'Top Speed' : '', 'Cylinders' : '', 'Valves' : '', 'Engine power' : '', 'Engine torque' : ''}
        pass
    
    #Extracting driver convenience features from the car searched.
    try:
        WebDriverWait(driver,3).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[data-gui='comfort-expander']"))).click()
        time.sleep(0.5)
        driver_convenience = WebDriverWait(driver,3).until(EC.element_to_be_clickable((By.XPATH, "//ul[@data-testid='Driver Convenience-list']")))
        driver_convenience_dict = {'Driver convenience' : ", ". join(driver_convenience.text.strip().split('\n'))}
        #print(driver_convenience_dict)
        index+=1
        WebDriverWait(driver,3).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[data-gui='comfort-expander']"))).click()
    except:
        driver_convenience_dict = {'Driver convenience' : ''}
        pass
    #Extracting safety features from the car searched.
    try:
        WebDriverWait(driver,3).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[data-gui='s-and-s-expander']"))).click()
        time.sleep(0.5)
        safety = WebDriverWait(driver,3).until(EC.element_to_be_clickable((By.XPATH, "//ul[@data-testid='Safety-list']")))
        safety_info_dict = {'Safety features' : ", ". join(safety.text.strip().split('\n'))}
        #print(safety_info_dict)
        index+=1
        WebDriverWait(driver,3).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[data-gui='s-and-s-expander']"))).click()
    except:
        safety_info_dict = {'Safety features' : ''}
        pass
    
    #Extracting exterior features from the car searched.
    try:
        WebDriverWait(driver,3).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[data-gui='exterior-expander']"))).click()
        time.sleep(0.5)
        exterior_features = WebDriverWait(driver,3).until(EC.element_to_be_clickable((By.XPATH, "//ul[@data-testid='Exterior Features-list']")))
        exterior_features_info_dict = {'Exterior features' : ", ".join(exterior_features.text.strip().split('\n'))}
        #print(exterior_features_info_dict)
        index+=1
        WebDriverWait(driver,3).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[data-gui='exterior-expander']"))).click()
    except:
        exterior_features_info_dict = {'Exterior features' : ''}
        pass
    
    #Extracting interior features from the car searched.
    try:
        WebDriverWait(driver,3).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[data-gui='interoirExpander']"))).click()
        time.sleep(0.5)
        interior_features = WebDriverWait(driver,3).until(EC.element_to_be_clickable((By.XPATH, "//ul[@data-testid='Interior Features-list']")))
        interior_features_info_dict = {'Interior features' : ", ".join(interior_features.text.strip().split('\n'))}
        #print(interior_features_info_dict)
        index+=1
        WebDriverWait(driver,3).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[data-gui='interoirExpander']"))).click()
    except:
        interior_features_info_dict = {'Interior features' : ''}
        pass
    #print(interior_features_info_dict)
    #Extracting technical features from the car searched.
    time.sleep(0.5)
    try:
        WebDriverWait(driver,3).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[data-gui='technicalExpander']"))).click()
        time.sleep(0.5)
        technical_features = WebDriverWait(driver,3).until(EC.element_to_be_clickable((By.XPATH, "//ul[@data-testid='Technical-list']")))
        technical_features_info_dict = {'Technical features' : ", ".join(technical_features.text.strip().split('\n'))}
        #print(technical_features_info_dict)
        index+=1
        WebDriverWait(driver,3).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[data-gui='technicalExpander']"))).click()
    except:
        technical_features_info_dict = {'Technical features' : ''}
        pass
    
    #Extracting dimension data from the car searched.
    try:
        WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[data-gui='dimentions-expander']"))).click()
        time.sleep(1)
        dimensions = WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="modal-root"]/div/section/div[2]/div/ul/li[{}]/span/div/ul'.format(index))))
        dimension_info_dict = Convert(dimensions.text.strip().split('\n'))
    except:
        dimension_info_dict = {'Height' : '', 'Length' : '', 'Wheelbase	Width' : '', 'Fuel tank capacity' : '', 'Gross vehicle weight' : '', 'Boot space (seats down)' : '', 'Boot space (seats up)' : '', 'Minimum kerb weight' : '', 'Max loading weight' : ''}
        pass
    #print(dimension_info_dict)

    
    #Adding results to dataframe.
    car_type = {'Manufacturer' : Manufacturer, 'Model' : Model, 'Variant' : Variant, 'Standard car description' : standard_car_description_val , 'Full Autotrader Vehicle description' : full_car_description}
    overall_dict= {**car_type, **NumMiles_dict , **DiffMiles_dict, **insurance_info_dict, **tax_info_dict, **fuel_info_dict, **key_attribute_dict, **performance_info_dict, **dimension_info_dict, **driver_convenience_dict, **safety_info_dict, **exterior_features_info_dict, **interior_features_info_dict, **technical_features_info_dict, **price_dict}
    scraped_df = pd.concat([scraped_df, pd.DataFrame.from_dict([overall_dict])], axis = 0)
    
    scraped_df.to_csv(file_path, index=False, header=False, mode='a') 
    
    rows_scraped.append(row)
    try:
        driver.close()
    except:
        pass
    return row


## This is where you change the path of the output csv where you want to store the data scraped.

In [ ]:
scraped_df.to_csv(file_path, index=False, header=False, mode='a') 


In [18]:

print(row) 

50099


In [20]:
scraped_df

,Manufacturer,Model,Variant,Standard car description,Full Autotrader Vehicle description,Number of Miles,Difference to average Miles,Fuel type,Body type,Engine,...,Safety features,Exterior features,Interior features,Technical features,Price,Range,Manufacturer warranty,Free road tax,Wheelbase\tWidth,Max loading weight
0,Mercedes-Benz,C Class,NaN,Mercedes-Benz C Class Coupe (2011 - 2015),Mercedes-Benz C Class 2.1 C220 CDI BlueEfficie...,"110,723","19,149 above",Diesel,Coupe,2.1L,...,"ABS - Anti-Lock Braking System, ASR - Accelera...","18in Alloy Wheels - AMG 7 Spoke Design, Brake ...",Automatic Climate Control - Two-Zone-Independe...,Agility Control Suspension with Adaptive Dampi...,"£5,850",NaN,NaN,NaN,NaN,NaN
0,Mercedes-Benz,C Class,NaN,Mercedes-Benz C Class Coupe (2011 - 2015),Mercedes-Benz C Class 2.1 C220 CDI Executive S...,"108,000","26,640 above",Diesel,Coupe,2.1L,...,"ABS - Anti-Lock Braking System, ASR - Accelera...","16in Alloy Wheels - 7-Spoke Design, Spare Whee...",Automatic Climate Control - Two-Zone-Independe...,Agility Control Suspension with Adaptive Dampi...,"£5,499",NaN,NaN,NaN,NaN,NaN
0,Mercedes-Benz,C Class,NaN,Mercedes-Benz C Class Estate (2007 - 2015),Mercedes-Benz C Class 3.0 C350 CDI V6 BlueEffi...,"143,000","21,049 above",Diesel,Estate,3.0L,...,,"17in 6 Twin Spoke AMG Alloy Wheels, Tinted win...",Auto Climate Control,,"£4,700",NaN,NaN,NaN,NaN,NaN
0,Mercedes-Benz,C Class,NaN,Mercedes-Benz C Class Estate (2007 - 2015),Mercedes-Benz C Class 2.1 C220 CDI Sport Auto ...,"99,000","41,135 below",Diesel,Estate,2.1L,...,,"17in 6 Twin Spoke AMG Alloy Wheels, Tinted win...",Auto Climate Control,,"£4,900",NaN,NaN,NaN,NaN,NaN
0,Mercedes-Benz,C Class,NaN,Mercedes-Benz C Class Estate (2007 - 2015),Mercedes-Benz C Class 3.0 C350 CDI V6 BlueEffi...,"143,000","21,049 above",Diesel,Estate,3.0L,...,,"17in 6 Twin Spoke AMG Alloy Wheels, Tinted win...",Auto Climate Control,,"£4,700",NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,BMW,5 Series,NaN,BMW 5 Series Estate (1997 - 2004),BMW 5 Series 2.5 525d SE Touring 5dr Diesel Ma...,"120,100",,Diesel,Estate,2.5L,...,"Air Bag Driver, Air Bag Passenger, Air Bag Sid...","Alloy Wheels (16in), Body Coloured Bumpers, Mi...","Computer, Mirrors Internal",,"£3,900",NaN,NaN,NaN,NaN,NaN
0,BMW,5 Series,NaN,BMW 5 Series Estate (1997 - 2004),BMW 5 Series 3.0 530i Sport Touring 5dr 2002 (...,"141,000",,Petrol,Estate,3.0L,...,"Air Bag Driver, Air Bag Passenger, Air Bag Sid...","Alloy Wheels (17in), Body Coloured Bumpers, Mi...","Computer, Mirrors Internal",,"£3,950",NaN,NaN,NaN,NaN,NaN
0,BMW,5 Series,NaN,BMW 5 Series Saloon (1996 - 2003),BMW 5 Series 2.5 525i Sport 4dr 2003 (03 reg),"108,000",,Petrol,Saloon,2.5L,...,"Air Bag Driver, Air Bag Passenger, Air Bag Sid...","Alloy Wheels (17in), Body Coloured Bumpers, Mi...","Computer (Driver Information System), Mirrors ...",,"£5,250",NaN,NaN,NaN,NaN,NaN
0,Jaguar,XJ,NaN,Jaguar XJ Saloon (2003 - 2008),Jaguar XJ 3.0 SE 4dr 2003 (53 reg),"111,652",,Petrol,Saloon,3.0L,...,"Air Bag Driver, Air Bag Passenger, Air Bag Sid...","Alloy Wheels (18in), Mirrors External (Electri...",,,"£2,895",NaN,NaN,NaN,NaN,NaN


In [7]:
scraped_df['Full Autotrader Vehicle description']

0    Ford Fiesta 1.0 ZETEC 3d 99 BHP | 14-DAYS MONE...
Name: Full Autotrader Vehicle description, dtype: object

In [ ]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--disable-popup-blocking")
chrome_options.add_argument("user-agent=Mozilla/3.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/59.0.3071.115 Safari/537.36")
chrome_options.add_argument('--ignore-certificate-errors')
chrome_options.add_argument('--incognito')
chrome_options.add_argument('--headless')
chrome_options.add_argument("--profile-directory=Default")
chrome_options.add_argument("--window-size=1920,1080")
#chrome_options.add_argument("--user-data-dir=C:/Users/benwallner5/AppData/Local/Google/Chrome/User Data")
driver = webdriver.Chrome(PATH, options = chrome_options)
driver.implicitly_wait(3)

driver.get("https://www.autotrader.co.uk")
driver.maximize_window()

all_cars_make_model_variant = []

try:
    driver.switch_to.frame("sp_message_iframe_687971")
    WebDriverWait(driver,5).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="notice"]/div[3]/div[2]/button[2]'))).click()
except:
    pass

try:
    driver.switch_to.frame("sp_message_iframe_687972")
    WebDriverWait(driver,5).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="notice"]/div[3]/div[2]/button[2]'))).click()
except:
    pass


#This moves into the 'More options' window to allow the search of specific makes and models.

try:
    WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.LINK_TEXT, 'More options'))).click()
except:
    pass

"""
#Make/Manufacturer
try:
    make = Select(WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="make"]'))))
    make_object = WebDriverWait(driver,10).until(EC.presence_of_all_elements_located((By.XPATH, '//*[@id="make"]/optgroup[2]/option')))
    all_makes = []
    for make in make_object:
        make_val = make.text.strip().split()[:-1]
        all_makes.append(" ".join(make_val))
except:
    pass
print(all_makes)
"""
time.sleep(1)
    
#Model

make_button = Select(WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="make"]'))))
make_object = WebDriverWait(driver,10).until(EC.presence_of_all_elements_located((By.XPATH, '//*[@id="make"]/optgroup[2]/option')))

all_make_models = []
for make in make_object:
    all_variants = []
    variant_val = ""
    make_val = make.text.strip().split()[:-1]
    make_value = make.get_attribute("value")
    try:
        make_button.select_by_value(make_value)
    except:
        pass

    time.sleep(1)
    
    model_button = Select(WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="model"]'))))
    model_object = WebDriverWait(driver,10).until(EC.presence_of_all_elements_located((By.XPATH, '//*[@id="model"]/option')))
    
    #print(len(model_object))
    all_models = []
    
    if len(model_object)>2:
        for model in model_object[1:]:
            model_val = model.text.strip().split()[:-1]
            model_value = model.get_attribute("value")
            all_models.append(" ".join(model_val))
            time.sleep(2)
            try:
                model_button.select_by_value(model_value)
            except:
                pass
            time.sleep(2)
            variant_button = Select(WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="aggregatedTrim"]'))))
            variant_object = WebDriverWait(driver,10).until(EC.presence_of_all_elements_located((By.XPATH, '//*[@id="aggregatedTrim"]/option')))
            #print(model.text)
            #print(len(variant_object))
            time.sleep(1)

            if len(variant_object)>2:
                all_variants = []
                time.sleep(1)
                for variant in variant_object[1:]:
                    variant_val = variant.text.strip().split()[:-1]
                    all_variants.append(" ".join(variant_val))
                make_model_variant = {" ".join(make_val):{" ".join(model_val):all_variants}}
                all_cars_make_model_variant.append(make_model_variant)
                #print(make_model_variant)
                
            else:
                time.sleep(1)
                try:
                    variant = WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="aggregatedTrim"]/option[2]')))
                    #print(variant.text)
                    variant_val = variant.text.strip()
                except:
                    #print('cant find variant')
                    variant_val = ""
                    pass
                make_model_variant = {" ".join(make_val):{" ".join(model_val):variant_val}}
                all_cars_make_model_variant.append(make_model_variant)
                #print(make_model_variant)
    else:
        try:
            model = WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="model"]/option[2]')))
            model_value = model.get_attribute("value")
            #print(model.text)
        except:
            pass
    
        model_val = model.text.strip()
        #print(model_val)
        time.sleep(1)  
        variant_button = Select(WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="aggregatedTrim"]'))))
        variant_object = WebDriverWait(driver,10).until(EC.presence_of_all_elements_located((By.XPATH, '//*[@id="aggregatedTrim"]/option')))
        time.sleep(1)
        all_variants = []
        if len(variant_object)>2:
            for variant in variant_object[1:]:
                variant_val = variant.text.strip().split()[:-1]
                all_variants.append(" ".join(variant_val))
            make_model_variant = {" ".join(make_val):{model.text.strip():all_variants}}
            all_cars_make_model_variant.append(make_model_variant)
            #print(make_model_variant)
        else:
            time.sleep(1)
            try:
                variant = WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="aggregatedTrim"]/option[2]')))
                variant_val = variant.text.strip()
            except:
                variant_val = ""
                pass
            make_model_variant = {" ".join(make_val): {model_val: variant_val}}
            all_cars_make_model_variant.append(make_model_variant)
            #print(make_model_variant)
driver.close()
print(all_cars_make_model_variant)

In [ ]:
for i in all_cars_make_model_variant:
    print(i['AC'])

In [12]:
makes_on_autoTrader = ['AC', 'AK', 'Abarth', 'Alfa Romeo', 'Alpine', 'Ariel', 'Aston Martin', 'Audi', 'Austin', 'BMW', 'Beauford', 'Bentley', 'Bowler', 'Bristol', 'Bugatti', 'Cfmoto', 'CUPRA', 'Cadillac', 'Caterham', 'Chesil', 'Chevrolet', 'Chrysler', 'Citroen', 'Corvette', 'Crendon', 'Custom Vehicle', 'DFSK', 'DS Automobiles', 'Dacia', 'Daewoo', 'Daihatsu', 'Daimler', 'Datsun', 'DAX', 'De Tomaso', 'Dodge', 'Ferrari', 'Fiat', 'Ford', 'GMC', 'Gwm Ora', 'Gardener', 'Great Wall', 'Hillman', 'Holden', 'Honda', 'Hummer', 'Hyundai', 'Infiniti', 'Isuzu', 'Iveco', 'JBA', 'Jaguar', 'Jeep', 'Jensen', 'KIA', 'Koenigsegg', 'LEVC', 'Lada', 'Lamborghini', 'Lancia', 'Land Rover', 'Lexus', 'Leyland', 'Lincoln', 'London Taxis International', 'Lotus', 'Maxus', 'MEV', 'MG', 'MINI', 'MK', 'Moke', 'Maserati', 'Maybach', 'Mazda', 'McLaren', 'Mercedes-Benz', 'Mercury', 'Microcar', 'Mitsubishi', 'Morgan', 'Morris', 'Nosmoke', 'Nissan', 'Noble', 'Opel', 'Packard', 'Panther', 'Perodua', 'Peugeot', 'Pilgrim', 'Polaris', 'Polestar', 'Pontiac', 'Porsche', 'Proton', 'Radical', 'Rage', 'Reliant', 'Renault', 'Reva', 'Riley', 'Robin Hood', 'Rolls-Royce', 'Rover', 'Royale', 'SEAT', 'SKODA', 'Saab', 'Shelby', 'Smart', 'Spyker', 'Ssangyong', 'Subaru', 'Sunbeam', 'Suzuki', 'TVR', 'Tesla', 'Toyota', 'Triumph', 'Ultima', 'Vauxhall', 'Volkswagen', 'Volvo', 'Westfield', 'Zimmer']

In [1]:
make_model_variant_AutoTrader = [{'AC': {'Cobra': ['MK IV', 'MK IV CRS']}}, {'AK': {'Cobra': ''}}, {'Abarth': {'124 Spider': ['GT', 'MultiAir', 'Scorpione']}}, {'Abarth': {'500': 'T-Jet'}}, {'Abarth': {'500c': 'T-Jet'}}, {'Abarth': {'595': ['50th Anniversario', '70th', 'Competizione', 'Competizione 70th', 'EsseEsse', 'EsseEsse 70th', 'F595', 'F595 Pista', 'Monster Yamaha', 'Pista 70th', 'Scorpioneoro', 'T-Jet', 'Trofeo', 'Turismo', 'Turismo 70th', 'Yamaha Factory Racing']}}, {'Abarth': {'595c': ['70th', 'Competizione', 'Competizione 70th', 'EsseEsse', 'F595', 'F595 Pista', 'Pista 70th', 'T-Jet', 'Trofeo', 'Turismo', 'Turismo 70th']}}, {'Abarth': {'695': ['70th', 'Biposto', 'Biposto Record', 'Competizione', 'EsseEsse', 'Rivale', 'T-Jet', 'Tributo 131 Rally', 'Turismo', 'XSR']}}, {'Abarth': {'695c': ['Competizione', 'Edizione Maserati', 'Rivale', 'T-Jet', 'Turismo', 'XSR']}}, {'Abarth': {'Grande Punto': 'Turbo'}}, {'Abarth': {'Punto Evo': ['Supersport', 'Turbo']}}, {'Alfa Romeo': {'145': ['Cloverleaf', 'T.Spark']}}, {'Alfa Romeo': {'147': ['Cloverleaf', 'Collezione', 'GTA', 'JTD', 'JTDM', 'Lusso', 'T.Spark', 'Turismo']}}, {'Alfa Romeo': {'156': ['GTA', 'JTD', 'JTS', 'Lusso', 'Turismo', 'V6', 'Veloce']}}, {'Alfa Romeo': {'156 Sportwagon': ['JTD', 'Veloce']}}, {'Alfa Romeo': {'159': ['JTDM', 'JTS', 'JTS V6', 'Lusso', 'TBi', 'TI', 'Turismo Sport']}}, {'Alfa Romeo': {'159 Sportwagon': ['JTDM', 'Lusso', 'TI', 'Turismo']}}, {'Alfa Romeo': {'166': ['Lusso', 'T.Spark']}}, {'Alfa Romeo': {'2000': ''}}, {'Alfa Romeo': {'4c': ['50th Anniversary', 'Launch Edition', 'TBi']}}, {'Alfa Romeo': {'75': 'T.Spark'}}, {'Alfa Romeo': {'8c': 'V8'}}, {'Alfa Romeo': {'Brera': ['JTDM', 'JTS', 'JTS V6', 'S', 'SV']}}, {'Alfa Romeo': {'Gt': ['BlackLine', 'Cloverleaf', 'JTDM', 'JTS', 'Lusso', 'T.Spark', 'Turismo', 'V6']}}, {'Alfa Romeo': {'Gtv': ['Lusso', 'T.Spark', 'Turismo', 'V6']}}, {'Alfa Romeo': {'Giulia': ['Estrema', 'Giulia', 'Lusso Ti', 'Nero Edizione', 'Quadrifoglio', 'Quadrifoglio NRING', 'Speciale', 'Sprint', 'Super', 'T', 'TD', 'Tecnica', 'V6 Bi-Turbo', 'Veloce', 'Veloce Ti']}}, {'Alfa Romeo': {'Giulietta': ['Business Edition', 'Cloverleaf', 'Collezione', 'Distinctive', 'Exclusive', 'Giulietta', 'JTDM-2', 'Lusso', 'MultiAir', 'Nero Edizione', 'Progression', 'QV Line', 'Quadrifoglio Verde', 'Quadrifoglio Verde Launch Ed', 'Speciale', 'Sport', 'Sportiva', 'Sportiva Nav', 'Sprint', 'Sprint Speciale', 'Super', 'TB', 'TBi', 'Tecnica', 'Turismo', 'Veloce']}}, {'Alfa Romeo': {'Mito': ['Cloverleaf', 'Collezione', 'Distinctive', 'JTDM', 'JTDM-2', 'Junior', 'Live', 'Lusso', 'MultiAir', 'Progression', 'QV Line', 'Quadrifoglio Verde', 'Speciale', 'Sportiva', 'Sprint', 'Super', 'TB', 'Turismo', 'TwinAir', 'Veloce']}}, {'Alfa Romeo': {'Spider': ['1750', 'JTDM', 'JTS', 'JTS V6', 'Limited Edition', 'Lusso', 'T.Spark', 'TBi', 'Turismo', 'V6']}}, {'Alfa Romeo': {'Stelvio': ['Estrema', 'Lusso Ti', 'Milano Edizione', 'Nero Edizione', 'Quadrifoglio', 'Quadrifoglio NRING', 'Speciale', 'Sprint', 'Super', 'T', 'TD', 'Ti', 'V6 Bi-Turbo', 'Veloce', 'Veloce Ti']}}, {'Alfa Romeo': {'Tonale': ['Speciale', 'Ti', 'VGT MHEV', 'Veloce']}}, {'Alpine': {'A110': ['GT', 'Legende', 'Legende GT', 'Pure', 'S', 'Turbo']}}, {'Ariel': {'Atom': ''}}, {'Ariel': {'Atom 4': ''}}, {'Ariel': {'Nomad': ''}}, {'Aston Martin': {'Cygnet': 'VVT'}}, {'Aston Martin': {'Db11': ['AMR', 'Launch Edition', 'V12', 'V12 BiTurbo', 'V8']}}, {'Aston Martin': {'Db2/4': ''}}, {'Aston Martin': {'Db4': ''}}, {'Aston Martin': {'Db5': ''}}, {'Aston Martin': {'Db6': ''}}, {'Aston Martin': {'Db7': ['GTA', 'Vantage']}}, {'Aston Martin': {'Db9': ['Carbon Edition', 'GT', 'GT Bond Edition', 'Luxury Edition', 'Sports Edition', 'V12']}}, {'Aston Martin': {'Dbs': ['Carbon Black', 'Superleggera', 'Superleggera OHMSS', 'V12', 'V12 BiTurbo']}}, {'Aston Martin': {'Dbx': ['707', 'V8']}}, {'Aston Martin': {'Lagonda': ''}}, {'Aston Martin': {'Le Mans': ''}}, {'Aston Martin': {'One-77': ''}}, {'Aston Martin': {'Rapide': ['Luxury Edition', 'V12']}}, {'Aston Martin': {'Rapide S': ['AMR', 'S', 'Shadow Edition', 'V12']}}, {'Aston Martin': {'V12 Speedster': ''}}, {'Aston Martin': {'V12 Zagato': ''}}, {'Aston Martin': {'V8': 'Vantage'}}, {'Aston Martin': {'Vanquish': ['Carbon Edition', 'Carbon Edition Black', 'S', 'S Ultimate', 'S Zagato', 'V12', 'Zagato']}}, {'Aston Martin': {'Vantage': ['007 Edition', 'AMR', 'AMR 59 Edition', 'AMR Hero', 'Centenary', 'F1 Edition', 'GT12', 'GT8', 'N420', 'N430', 'S', 'S Centenary', 'SP10', 'V12', 'V12 BiTurbo', 'V8']}}, {'Aston Martin': {'Virage': 'V12'}}, {'Audi': {'80': ['E', 'S2']}}, {'Audi': {'A1': ['Amplified Edition', 'Black Edition', 'CoD', 'Competition Line', 'Contrast Edition', 'Contrast Edition Plus', 'S line', 'S line Competition', 'S line Contrast Edition', 'S line Style Edition', 'SE', 'Sport', 'TDI', 'TFSI', 'Technik', 'Vorsprung']}}, {'Audi': {'A2': ['FSI', 'SE', 'Sport', 'TDI']}}, {'Audi': {'A3': ['Black Edition', 'CoD', 'Edition 1', 'FSI', 'S line', 'S line Competition', 'SE', 'SE Technik', 'Sport', 'T', 'TD', 'TDI', 'TDIe', 'TFSI', 'TFSIe', 'Technik', 'Technik SE', 'Vorsprung', 'e-tron', 'ultra']}}, {'Audi': {'A3 Cabriolet': ['Black Edition', 'CoD', 'S line', 'S line Final Edition', 'SE', 'Sport', 'Sport Final Edition', 'TDI', 'TFSI', 'Technik']}}, {'Audi': {'A4': ['Black Edition', 'Black Edition Plus', 'DTM Edition', 'Dynamik', 'Executive S line', 'Executive SE', 'FSI', 'FSI V6', 'S line', 'S line Special Edition', 'SE', 'SE Technik', 'Sport', 'Sport Edition', 'T', 'TDI', 'TDI V6', 'TDIe', 'TFSI', 'Technik', 'Vorsprung', 'ultra']}}, {'Audi': {'A4 Allroad': ['Sport', 'TDI', 'TDI V6', 'TFSI', 'Vorsprung']}}, {'Audi': {'A4 Avant': ['Black Edition', 'Black Edition Plus', 'Dynamik', 'Executive S line', 'Executive SE', 'FSI', 'FSI V6', 'S line', 'S line Special Edition', 'SE', 'SE Technik', 'Sport', 'Sport Edition', 'T', 'TDI', 'TDI V6', 'TDIe', 'TFSI', 'Technik', 'Vorsprung', 'ultra']}}, {'Audi': {'A4 Cabriolet': ['FSI', 'S line', 'Sport', 'T', 'TDI', 'TFSI']}}, {'Audi': {'A5': ['Black Edition', 'Black Edition Plus', 'Edition 1', 'FSI V6', 'S line', 'S line Special Edition', 'SE', 'SE Technik', 'Sport', 'TDI', 'TDI V6', 'TDIe', 'TFSI', 'TFSI V6', 'Vorsprung', 'ultra']}}, {'Audi': {'A5 Cabriolet': ['Edition 1', 'S line', 'S line Edition', 'S line Special Edition', 'S line Special Edition Plus', 'SE', 'Sport', 'TDI', 'TDI V6', 'TFSI', 'Vorsprung']}}, {'Audi': {'A6 Allroad': ['BiTDI V6', 'FSI', 'Special Edition', 'Sport', 'TDI', 'TDI V6', 'TFSI V6']}}, {'Audi': {'A6 Avant': ['BiTDI V6', 'Black Edition', 'Le Mans', 'S line', 'SE', 'SE Executive', 'Sport', 'T', 'TDI', 'TDI V6', 'TDIe', 'TFSI', 'TFSI V6', 'TFSIe', 'Vorsprung', 'ultra']}}, {'Audi': {'A6 Saloon': ['BiTDI V6', 'Black Edition', 'FSI', 'FSI V6', 'Le Mans', 'S line', 'S line Special Edition', 'SE', 'SE Executive', 'Sport', 'TDI', 'TDI V6', 'TDIe', 'TFSI', 'TFSI V6', 'TFSIe', 'Vorsprung', 'ultra']}}, {'Audi': {'A6 Unspecified': ''}}, {'Audi': {'A7': ['BiTDI V6', 'Black Edition', 'FSI V6', 'S line', 'S line Exclusive', 'SE', 'SE Executive', 'Sport', 'Sport Edition', 'TDI', 'TDI V6', 'TFSI', 'TFSI V6', 'TFSIe', 'Vorsprung', 'ultra']}}, {'Audi': {'A8': ['Black Edition', 'Edition 21', 'Executive SE', 'FSI', 'FSI V8', 'Launch Edition', 'S line', 'SE', 'SE Executive', 'Sport', 'Sport Executive', 'TDI', 'TDI V6', 'TDI V8', 'TFSI', 'TFSI V6', 'TFSI V8', 'TFSIe V6', 'Vorsprung', 'W12']}}, {'Audi': {'Allroad': ['T', 'TDI']}}, {'Audi': {'Cabriolet': ['E', 'Final Edition']}}, {'Audi': {'Coupe': 'E'}}, {'Audi': {'Q2': ['Black Edition', 'CoD', 'Edition 1', 'S line', 'SE', 'Sport', 'TDI', 'TFSI', 'Technik', 'Vorsprung']}}, {'Audi': {'Q3': ['Black Edition', 'CoD', 'Edition 1', 'S line', 'S line Edition', 'S line Plus', 'SE', 'Sport', 'TDI', 'TFSI', 'TFSIe', 'Technik', 'Vorsprung']}}, {'Audi': {'Q4 E-tron': ['Edition 1', 'Launch Edition', 'S line', 'Sport', 'Vorsprung']}}, {'Audi': {'Q5': ['Black Edition', 'Competition', 'Competition Vorsprung', 'Edition 1', 'FSI V6', 'S line', 'S line Competition', 'S line Plus', 'S line Plus Special Edition', 'S line Special Edition', 'SE', 'Sport', 'TDI', 'TDI V6', 'TFSI', 'TFSIe', 'Vorsprung', 'Vorsprung Competition']}}, {'Audi': {'Q7': ['Black Edition', 'Competition', 'Competition Vorsprung', 'FSI', 'FSI V6', 'S line', 'S line Plus', 'S line Sport Edition', 'S line Style Edition', 'SE', 'Sport', 'TDI', 'TDI V6', 'TDI V8', 'TDIcd V6', 'TFSI V6', 'TFSIe V6', 'Vorsprung', 'e-tron']}}, {'Audi': {'Q8': ['Black Edition', 'Edition 1', 'S line', 'TDI V6', 'TFSI V6', 'TFSIe V6', 'Vorsprung']}}, {'Audi': {'Q8 E-tron': ['Black Edition', 'Vorsprung']}}, {'Audi': {'R8': ['FSI V10', 'FSI V8', 'GT', 'Limited Edition', 'Performance', 'Performance Carbon Black', 'Performance Edition', 'Plus']}}, {'Audi': {'Rs Q3': ['Audi Sport Edition', 'Performance', 'TFSI', 'Vorsprung']}}, {'Audi': {'Rs E-tron Gt': ['Carbon Black', 'Carbon Vorsprung']}}, {'Audi': {'Rs2': 'RS2'}}, {'Audi': {'Rs3': ['Audi Sport Edition', 'Carbon Black', 'Launch Edition', 'TFSI', 'Vorsprung']}}, {'Audi': {'Rs4': ''}}, {'Audi': {'Rs4 Avant': ['Bronze Edition', 'Carbon Black', 'Carbon Edition', 'FSI V8', 'Limited Edition', 'Nogaro', 'Nogaro Edition', 'Sport Edition', 'TFSI V6', 'Vorsprung']}}, {'Audi': {'Rs4 Cabriolet': ''}}, {'Audi': {'RS5': ['Carbon Black', 'Carbon Edition', 'FSI V8', 'Limited Edition', 'Nogaro Edition', 'Sport Edition', 'TFSI V6', 'Vorsprung']}}, {'Audi': {'Rs6': 'TFSI V10'}}, {'Audi': {'Rs6 Avant': ['Carbon Black', 'Launch Edition', 'Nogaro Edition', 'Performance', 'TFSI V10', 'TFSI V8', 'Vorsprung']}}, {'Audi': {'Rs7': ['Carbon Black', 'Performance', 'TFSI V8', 'Vorsprung']}}, {'Audi': {'Rsq8': ['Carbon Black', 'TFSI V8', 'Vorsprung']}}, {'Audi': {'S1': ['Competition', 'TFSI']}}, {'Audi': {'S3': ['Black Edition', 'TFSI', 'Vorsprung']}}, {'Audi': {'S4': ['Black Edition', 'TDI V6', 'TFSI V6', 'Vorsprung']}}, {'Audi': {'S4 Avant': ['Black Edition', 'TDI V6', 'TFSI V6', 'Vorsprung']}}, {'Audi': {'S4 Cabriolet': ''}}, {'Audi': {'S5': ['Black Edition', 'Edition 1', 'TDI V6', 'TFSI V6', 'V8', 'Vorsprung']}}, {'Audi': {'S6 Avant': ['Black Edition', 'TDI V6', 'TFSI V8', 'Vorsprung']}}, {'Audi': {'S6 Saloon': ['Black Edition', 'TDI V6', 'TFSI V8']}}, {'Audi': {'S7': ['Black Edition', 'TDI V6', 'TFSI V8', 'Vorsprung']}}, {'Audi': {'S8': ['Black Edition', 'FSI', 'Plus', 'TFSI V8', 'Vorsprung']}}, {'Audi': {'SQ2': ['Black Edition', 'TFSI', 'Vorsprung']}}, {'Audi': {'Sq5': ['BiTDI V6', 'Plus', 'Plus Special Edition', 'TDI V6', 'TFSI V6', 'Vorsprung']}}, {'Audi': {'Sq7': ['Black Edition', 'TDI V8', 'TFSI V8', 'Vorsprung']}}, {'Audi': {'Sq8': ['Black Edition', 'TDI V8', 'TFSI V8', 'Vorsprung']}}, {'Audi': {'TT': ['20 Years', 'Black Edition', 'S line', 'S line Special Edition', 'Sport', 'Sport Edition', 'T', 'TDI', 'TFSI', 'TFSI V6', 'V6', 'Vorsprung', 'ultra']}}, {'Audi': {'TT RS': ['Plus', 'RS', 'Sport Edition', 'TFSI', 'Vorsprung']}}, {'Audi': {'Tts': ['Black Edition', 'Limited Edition', 'TFSI', 'Vorsprung']}}, {'Audi': {'E-Tron': ['Black Edition', 'Edition 1', 'Launch Edition', 'S line', 'Sport', 'Technik', 'Vorsprung']}}, {'Audi': {'E-tron Gt': 'Vorsprung'}}, {'Audi': {'E-tron S': 'Vorsprung'}}, {'Audi': {'Quattro': ''}}, {'Austin': {'A30': ''}}, {'Austin': {'A40': ''}}, {'Austin': {'A55': ''}}, {'Austin': {'A60': ''}}, {'Austin': {'Ambassador': 'HL'}}, {'Austin': {'Healey': ''}}, {'Austin': {'Metro': 'L'}}, {'Austin': {'Mini': ['1275 GT', 'City', 'City E', 'Clubman', 'HL', 'Mayfair', 'Piccadilly']}}, {'Austin': {'Mini Moke': ''}}, {'Austin': {'Seven': ''}}, {'BMW': {'1 Series': ['114d', '114i', '116d', '116d ED', '116i', '118d', '118i', '120d', '120i', '123d', '125d', '125i', '128ti', '130i', '135i', 'BluePerformance', 'ED Plus', 'ES', 'Edition ES', 'EfficientDynamics', 'EfficientDynamics Business', 'Exclusive Edition', 'M', 'M Sport', 'M Sport LE', 'M Sport Shadow Edition', 'M135i', 'M140i', 'Performance Edition', 'SE', 'SE Business', 'Shadow Edition', 'Sport', 'Sport Plus Edition', 'Urban']}}, {'BMW': {'1502': ''}}, {'BMW': {'2 Series': ['218d', '218i', '220d', '220i', '225d', '228i', '230i', 'Luxury', 'M Sport', 'M235i', 'M240i', 'MHT', 'Modern', 'SE', 'Sport']}}, {'BMW': {'2 Series Active Tourer': ['216d', '218d', '218i', '220d', '220i', '223i', '225i', '225xe', '230xe', 'Luxury', 'M Sport', 'MHT', 'SE', 'Sport']}}, {'BMW': {'2 Series Gran Coupe': ['218d', '218i', '220d', '220i', 'M Sport', 'M235i', 'Sport']}}, {'BMW': {'2 Series Gran Tourer': ['216d', '218d', '218i', '220d', '220i', 'Luxury', 'M Sport', 'SE', 'Sport']}}, {'BMW': {'2002': ''}}, {'BMW': {'3 Series': ['316', '316d', '316i', '316ti', '318', '318 ES', '318 M Sport', '318 SE', '318 Sport', '318Ci', '318d', '318i', '318ti', '320', '320 M Sport', '320 SE', '320 Sport', '320Cd', '320Ci', '320d', '320d ED', '320i', '320i ED', '320td', '323', '323Ci', '323i', '325', '325 M Sport', '325 SE', '325 Sport', '325Ci', '325d', '325i', '325ti', '328', '328Ci', '328i', '330', '330 Clubsport', '330 M Sport', '330 SE', '330 Sport', '330Cd', '330Ci', '330d', '330e', '330i', '335d', '335i', '340i', 'Baur Conversion', 'BluePerformance', 'ED Plus', 'ED Sport', 'ES', 'Edition ES', 'Edition M Sport', 'Edition SE', 'EfficientDynamics', 'EfficientDynamics Business', 'Exclusive Edition', 'Lux', 'Luxury', 'M Sport', 'M Sport Business Edition', 'M Sport Highline', 'M Sport Plus Edition', 'M Sport Pro Edition', 'M Sport Shadow Edition', 'M340d', 'M340i', 'MHT', 'Modern', 'Motorsport Limited Edition', 'Performance Edition', 'S', 'SE', 'SE Business Edition', 'SE Highline', 'SE Pro', 'Sport', 'Sport Plus Edition', 'Sport Pro']}}, {'BMW': {'3 Series Gran Turismo': ['318d', '320d', '320i', '325d', '328i', '330d', '330i', '335d', '335i', '340i', 'Luxury', 'M Sport', 'Modern', 'SE', 'Sport']}}, {'BMW': {'4 Series': ['418d', '420d', '420i', '425d', '428i', '430d', '430i', '435d', '435i', '440i', 'Luxury', 'M Sport', 'M Sport Pro Edition', 'M440d', 'M440i', 'MHT', 'Modern', 'SE', 'Sport']}}, {'BMW': {'4 Series Gran Coupe': ['418d', '420d', '420i', '428i', '430d', '430i', '435d', '435i', '440i', 'Luxury', 'M Sport', 'M Sport Pro Edition', 'M440i', 'MHT', 'SE', 'Sport']}}, {'BMW': {'5 Series': ['518d', '518i', '520d', '520d ED', '520i', '523i', '525', '525d', '525i', '525td', '528i', '530d', '530e', '530i', '535d', '535i', '540i', '540i V8', '545e', '545i V8', '550i V8', 'BluePerformance', 'E', 'ES SE', 'EfficientDynamics', 'Lux', 'Luxury', 'M Sport', 'M Sport Business Edition', 'M Sport Edition', 'M550i V8', 'MHT', 'Modern', 'SE', 'SE Business Edition', 'Sport']}}, {'BMW': {'5 Series Gran Turismo': ['520d', '530d', '535d', '550i V8', 'Executive', 'Luxury', 'M Sport', 'SE']}}, {'BMW': {'6 Series': ['630i', '635CSi', '635d', '640d', '640i', '645Ci V8', '650i V8', 'M Sport', 'M Sport Limited Edition', 'SE', 'Sport']}}, {'BMW': {'6 Series Gran Coupe': ['640d', '640i', '650i V8', 'M Sport', 'SE']}}, {'BMW': {'6 Series Gran Turismo': ['620d', '630d', '630i', '640i', 'M Sport', 'SE']}}, {'BMW': {'7 Series': ['728i', '730Ld', '730d', '730i', '735i', '740Ld', '740Le', '740Li', '740d', '740e', '740i', '745Le', '745e', '750Li V8', '750i', '750i V8', '760Li V12', 'BluePerformance', 'Exclusive', 'L', 'Luxury Edition', 'M Sport', 'M Sport Exclusive', 'M Sport Luxury Edition', 'M760L V12', 'MHT', 'SE', 'SE Exclusive', 'SE Luxury Edition', 'Sport']}}, {'BMW': {'8 Series': ['840Ci V8', '840d', '840i', '850i', 'Ci', 'Ci Sport', 'First Edition', 'M Sport', 'M850i V8', 'MHT']}}, {'BMW': {'8 Series Alpina B8': 'i V8'}}, {'BMW': {'8 Series Gran Coupe': ['840d', '840i', 'M Sport', 'M850i V8', 'MHT']}}, {'BMW': {'Alpina B10': ''}}, {'BMW': {'Alpina B3': ['BiTurbo', 'S']}}, {'BMW': {'Alpina B5': ['BiTurbo V8', 'i V8']}}, {'BMW': {'Alpina B6': ''}}, {'BMW': {'Alpina B8': ''}}, {'BMW': {'Alpina D3': ['Lux', 'MHT', 'S', 'd', 'd BiTurbo']}}, {'BMW': {'Alpina D4': 'd BiTurbo'}}, {'BMW': {'Alpina D5': ['S', 'd', 'd BiTurbo']}}, {'BMW': {'Alpina Roadster': ''}}, {'BMW': {'Alpina Unspecified Models': ''}}, {'BMW': {'Alpina Xb7': 'i V8'}}, {'BMW': {'Alpina Xd3': ['MHT', 'd', 'd BiTurbo']}}, {'BMW': {'M2': ['BiTurbo', 'CS', 'Competition', 'i']}}, {'BMW': {'M3': ['30 Jahre Edition', 'Alpine', 'BiTurbo', 'CS', 'CSL', 'Competition', 'Competition M', 'Evolution', 'Evolution II', 'Limited Edition 500', 'Monte Carlo', 'V8', 'i', 'iV8']}}, {'BMW': {'M4': ['30 Jahre Edition', 'BiTurbo', 'CS', 'CSL', 'Competition', 'Competition M', 'DTM', 'GTS', 'Heritage Edition']}}, {'BMW': {'M5': ['CS', 'Competition', 'Competition 35 Jahre Edition', 'Competition Edition', 'M Performance Edition', 'V10', 'V8', 'i V8']}}, {'BMW': {'M6': ['V10', 'V8']}}, {'BMW': {'M6 Gran Coupe': 'V8'}}, {'BMW': {'M8': ['Competition', 'M8i V8', 'i V8']}}, {'BMW': {'M8 Gran Coupe': ['Competition', 'M8i V8', 'i V8']}}, {'BMW': {'X1': ['16d', '18d', '18i', '20d', '20d ED', '20i', '23d', '23i', '25d', '25e', '30e', 'EfficientDynamics', 'EfficientDynamics Business', 'M Sport', 'MHT', 'SE', 'Sport', 'xLine']}}, {'BMW': {'X2': ['18d', '18i', '20d', '20i', '25e', 'M Mesh Edition', 'M Sport', 'M Sport X', 'M35i', 'SE', 'Sport']}}, {'BMW': {'X3': ['18d', '20d', '20i', '30d', '30e', '35d', 'BluePerformance', 'Limited Sport Edition', 'M Sport', 'M40d', 'M40i', 'MHT', 'Pro Edition', 'SE', 'SE Edition Premium', 'Sport', 'd', 'i', 'sd', 'xLine']}}, {'BMW': {'X3 M': ['Competition', 'i']}}, {'BMW': {'X4': ['20d', '30d', '35d', 'M Sport', 'M Sport X', 'M40d', 'M40i', 'MHT', 'Pro Edition', 'SE', 'Sport', 'xLine']}}, {'BMW': {'X4 M': ['Competition', 'i']}}, {'BMW': {'X5': ['10-Year Edition', '25d', '30d', '35d', '40d', '40e', '40i', '45e', '48i V8', '50i V8', 'Black Vermilion Edition', 'Exclusive Edition', 'Le Mans Blue Sport Edition', 'M Sport', 'M50d', 'M50i V8', 'MHT', 'SE', 'Sport', 'Sport Exclusive Edition', 'd', 'i', 'i V8', 'iS V8', 'sd', 'si', 'xLine']}}, {'BMW': {'X5 M': ['BiTurbo V8', 'Competition', 'First Edition', 'i V8']}}, {'BMW': {'X6': ['30d', '35d', '35i', '40d', '40i', '50i V8', 'Black Vermilion Edition', 'BluePerformance', 'M Sport', 'M50d', 'M50i V8', 'MHT', 'SE', 'Sport', 'i V8']}}, {'BMW': {'X6 M': ['BiTurbo V8', 'Competition', 'i V8']}}, {'BMW': {'X7': ['30d', '40d', '40i', 'Dark Shadow Edition', 'Excellence', 'M Sport', 'M50d', 'M50i V8', 'M60i V8', 'MHT']}}, {'BMW': {'Z1': ''}}, {'BMW': {'Z3': ['Sport', 'Titanium Edition', 'i']}}, {'BMW': {'Z3 M': ''}}, {'BMW': {'Z4': ['18i', '20i', '23i', '28i', '30i', '35i', '35is', 'Edition Exclusive', 'Edition Sport', 'First Edition', 'Highline Edition', 'M Sport', 'M Sport Highline Edition', 'M40i', 'SE', 'Sport', 'i', 'si']}}, {'BMW': {'Z4 M': 'i'}}, {'BMW': {'Z8': 'V8'}}, {'BMW': {'I3': ['Fluid Grey Edition', 'Mr Porter', 'S']}}, {'BMW': {'I4': ['40', 'M Sport', 'M50', 'Sport']}}, {'BMW': {'I8': ['Protonic Frozen Black Edition', 'Protonic Frozen Yellow Edition', 'Protonic Red Edition']}}, {'BMW': {'Ix': ['40', '50', 'M Sport', 'M Sport Edition', 'M60', 'Sport']}}, {'BMW': {'Ix3': ['M Sport', 'M Sport Pro', 'Premier Edition', 'Premier Edition Pro']}}, {'Beauford': {'Series 3': ''}}, {'Bentley': {'4.25 Litre': ''}}, {'Bentley': {'Arnage': ['Final Series', 'Green Label', 'R', 'RL', 'Red Label', 'T', 'V8']}}, {'Bentley': {'Azure': ['Final Series', 'Mulliner', 'T']}}, {'Bentley': {'Bentayga': ['Azure', 'First Edition', 'Mulliner', 'S', 'Speed', 'TFSI V6', 'V8', 'W12', 'd V8']}}, {'Bentley': {'Brooklands': 'R'}}, {'Bentley': {'Continental': ['FlexFuel', 'Flying Spur', 'Flying Spur Speed', 'GT', 'GT Azure', 'GT First Edition', 'GT Mulliner', 'GT S', 'GT Speed', 'GT Supersports', 'GT V8', 'GT V8 S', 'GT3-R', 'GTC', 'GTC Azure', 'GTC First Edition', 'GTC Mulliner', 'GTC S', 'GTC Speed', 'GTC Supersports', 'GTC Supersports ISR', 'GTC V8', 'GTC V8 S', 'Mulliner GT', 'R', 'S', 'SC', 'Supersports', 'V8', 'W12']}}, {'Bentley': {'Corniche': ''}}, {'Bentley': {'Eight': ''}}, {'Bentley': {'Flying Spur': ['First Edition', 'Mulliner', 'S', 'TFSI V6', 'V8', 'W12']}}, {'Bentley': {'Mark Vi': ''}}, {'Bentley': {'Mulsanne': ['Speed', 'V8']}}, {'Bentley': {'R-type': ''}}, {'Bentley': {'T Series': ''}}, {'Bentley': {'Turbo R': ''}}, {'Bentley': {'Turbo Rt': ''}}, {'Bowler': {'Bulldog': ''}}, {'Bristol': {'411': ''}}, {'Bugatti': {'Veyron': ''}}, {'Cfmoto': {'Zforce 1000 Sport': ''}}, {'CUPRA': {'Ateca': ['TSI', 'VZ1', 'VZ2', 'VZ3']}}, {'CUPRA': {'Born': ['V1', 'V2', 'V3', 'e-Boost']}}, {'CUPRA': {'Formentor': ['TSI', 'V1', 'V2', 'VZ Edition', 'VZ1', 'VZ2', 'VZ3']}}, {'CUPRA': {'Leon': ['First Edition', 'TSI', 'VZ1', 'VZ2', 'VZ3']}}, {'Cadillac': {'Allante': ''}}, {'Cadillac': {'Bls': ['D', 'Elegance', 'SE', 'Sport Luxury']}}, {'Cadillac': {'Cts': 'Sport Luxury'}}, {'Cadillac': {'De Ville': ''}}, {'Cadillac': {'Escalade': ''}}, {'Cadillac': {'Srx': ['Elegance', 'Sport Luxury']}}, {'Cadillac': {'Sts': 'Sport Luxury'}}, {'Cadillac': {'Seville': 'STS'}}, {'Caterham': {'Seven': ['160', '270 S', '310 R', '420', '420 R', '420 S', '620 R', 'K-Series Super Sport', 'Roadsport', 'Superlight R300', 'Ti-VCT']}}, {'Caterham': {'Super Seven Convertible': ''}}, {'Caterham': {'Super Seven Sports 2dr': ''}}, {'Chesil': {'Speedster': ''}}, {'Chevrolet': {'Aveo': ['LS', 'LT', 'LTZ', 'S', 'VCDi', 'VCDi ECO']}}, {'Chevrolet': {'C10': ''}}, {'Chevrolet': {'Camaro': 'V8'}}, {'Chevrolet': {'Captiva': ['LS', 'LT', 'LTX', 'LTZ', 'VCDi', 'i']}}, {'Chevrolet': {'Chevelle': ''}}, {'Chevrolet': {'Chevy': ''}}, {'Chevrolet': {'Corvette': ''}}, {'Chevrolet': {'Corvette Stingray': ['2LT', '3LT', 'V8']}}, {'Chevrolet': {'Cruze': ['LS', 'LT', 'LTZ', 'VCDi']}}, {'Chevrolet': {'Epica': ['LT', 'VCDi']}}, {'Chevrolet': {'Gmc': ''}}, {'Chevrolet': {'Kalos': ['SE', 'SX']}}, {'Chevrolet': {'Lacetti': ['SE', 'SX', 'Sport']}}, {'Chevrolet': {'Matiz': ['S', 'SE', 'SE Flair', 'SE+', 'SX']}}, {'Chevrolet': {'Nova': ''}}, {'Chevrolet': {'Orlando': ['LS', 'LT', 'LTZ', 'VCDi']}}, {'Chevrolet': {'Silverado': ''}}, {'Chevrolet': {'Spark': ['+', 'LS', 'LT', 'LTZ', 'i']}}, {'Chevrolet': {'Suburban': ''}}, {'Chevrolet': {'Tacuma': 'SX'}}, {'Chevrolet': {'Trax': ['LS', 'LT', 'T', 'VCDi']}}, {'Chrysler': {'300c': ['CRD', 'CRD V6', 'Executive', 'Hemi', 'LUX', 'Limited', 'SRT Design', 'SRT-8']}}, {'Chrysler': {'Crossfire': ['Roadster', 'SRT-6']}}, {'Chrysler': {'Delta': ['M-Air', 'M-Jet', 'SE', 'SR']}}, {'Chrysler': {'Grand Voyager': ['25th Anniversary', 'CRD', 'Executive', 'Executive XS', 'LX', 'Limited', 'Limited XS', 'Signature', 'Touring', 'V6']}}, {'Chrysler': {'Pt Cruiser': ['CRD', 'Classic', 'Touring']}}, {'Chrysler': {'Sebring': 'CRD'}}, {'Chrysler': {'Viper': ''}}, {'Chrysler': {'Voyager': ['CRD', 'Executive', 'LX']}}, {'Chrysler': {'Windsor': ''}}, {'Chrysler': {'Ypsilon': ['Black&Red', 'Gold', 'Limited', 'MultiJet', 'S', 'S-Series', 'SE', 'TwinAir']}}, {'Citroen': {'2 Cv': ['Dolly', 'Special']}}, {'Citroen': {'Ami': ''}}, {'Citroen': {'Berlingo': ['Airdream', 'BlueHDi', 'BlueHDi 725', 'BlueHDi 850', 'D', 'Desire Multispace', 'Enterprise XL', 'Enterprise XL Edition', 'Enterprise XL Pro', 'Feel', 'Feel Edition Multispace', 'Feel Multispace', 'Flair', 'Flair Multispace', 'Flair XTR', 'Forte Multispace', 'HDi', 'HDi 725', 'L2 725 X', 'Multispace', 'Plus', 'PureTech', 'TD', 'VT', 'VTR', 'VTi', 'X', 'XTR', 'XTR Multispace', 'e-HDi', 'i']}}, {'Citroen': {'C-crosser': ['Code', 'Exclusive', 'HDi', 'VTR+']}}, {'Citroen': {'C-zero': ''}}, {'Citroen': {'C1': ['Airplay', 'Connexion', 'Cool', 'ELLE', 'Edition', 'Feel', 'Feel Edition', 'Flair', 'Flair Edition', 'Furio', 'HDi', 'JCC+', 'Origins', 'Platinum', 'PureTech', 'Rhythm', 'Sense', 'Shine', 'Splash', 'Touch', 'Urban Ride', 'VT', 'VTR', 'VTR+', 'VTi', 'Vibe', 'i']}}, {'Citroen': {'C2': ['Airplay+', 'Cachet', 'Code', 'Cool', 'Design', 'HDi', 'L', 'LX', 'Rhythm', 'SX', 'VT', 'VTR', 'VTS', 'Vibe', 'i']}}, {'Citroen': {'C3': ['+', 'Airdream', 'Airdream +', 'Airplay+', 'Black', 'BlueHDi', 'C-Series', 'C-Series Edition', 'Cachet', 'Connexion', 'Cool', 'Desire', 'ELLE', 'Edition', 'Elle', 'Exclusive', 'Feel', 'Feel Nav Edition', 'Flair', 'Flair Nav Edition', 'Flair Plus', 'HDi', 'L', 'LX', 'Origins', 'Platinum', 'PureTech', 'Rhythm', 'SX', 'Saint James', 'Selection', 'Sense', 'Shine', 'Shine Plus', 'Touch', 'VT', 'VTR', 'VTR+', 'VTi', 'White', 'XTR', 'YOU', 'e-HDi', 'e-VTi', 'i']}}, {'Citroen': {'C3 Aircross': ['BlueHDi', 'C-Series', 'C-Series Edition', 'Feel', 'Flair', 'Origins', 'PureTech', 'Rip Curl', 'Shine', 'Shine Plus']}}, {'Citroen': {'C3 Picasso': ['Airdream +', 'Blackcherry', 'BlueHDi', 'Code', 'Connexion', 'Edition', 'Exclusive', 'HDi', 'Platinum', 'PureTech', 'Selection', 'VT', 'VTR+', 'VTi']}}, {'Citroen': {'C3 Pluriel': ['Cote d Azur', 'Exclusive', 'i']}}, {'Citroen': {'C4': ['Airdream', 'BlueHDi', 'C-Series Edition', 'Cachet', 'Cool', 'Edition', 'Exclusive', 'Feel', 'Flair', 'HDi', 'Platinum', 'PureTech', 'SX', 'Selection', 'Sense', 'Sense Plus', 'Shine', 'Shine Plus', 'THP', 'Touch', 'VTR', 'VTR+', 'VTi', 'by LOEB', 'by Loeb', 'e-HDi', 'i']}}, {'Citroen': {'C4 Cactus': ['BlueHDi', 'Feel', 'Feel Edition', 'Flair', 'Flair Edition', 'Origins', 'PureTech', 'Rip Curl', 'Touch', 'W', 'e-HDi']}}, {'Citroen': {'C4 Picasso': ['Airdream', 'BlueHDi', 'Edition', 'Exclusive', 'Exclusive+', 'Feel', 'Flair', 'HDi', 'Lounge', 'Platinum', 'PureTech', 'SX', 'Selection', 'THP', 'Touch Edition', 'VTR', 'VTR+', 'VTi', 'e-HDi', 'i']}}, {'Citroen': {'C4 Spacetourer': ['BlueHDi', 'Feel', 'Flair', 'PureTech', 'Touch Edition']}}, {'Citroen': {'C5': ['Airdream', 'Design', 'Exclusive', 'HDi', 'HDi V6', 'V6', 'VTR', 'VTR+', 'e-HDi', 'i']}}, {'Citroen': {'C5 Aircross': ['Black Edition', 'BlueHDi', 'C-Series', 'C-Series Edition', 'Feel', 'Flair', 'Flair Plus', 'PureTech', 'Sense', 'Sense Plus', 'Shine', 'Shine Plus']}}, {'Citroen': {'C5 X': ['PureTech', 'Sense Plus', 'Shine', 'Shine Plus']}}, {'Citroen': {'C6': ['Exclusive', 'HDi', 'HDi V6', 'Lignage']}}, {'Citroen': {'C8': ['HDi', 'LX', 'SX', 'i']}}, {'Citroen': {'Ds3': ['Airdream', 'Black', 'BlueHDi', 'DSign', 'DSign Ice', 'DSign Plus', 'DSign by Benefit', 'DSire', 'DSire Plus', 'DSport', 'DSport Pink', 'DSport Plus', 'DSport Red', 'DStyle', 'DStyle Ice', 'DStyle Pink', 'DStyle Plus', 'DStyle Red', 'DStyle Techno', 'DStyle by Benefit', 'HDi', 'Orla Kiely', 'PureTech', 'Racing', 'Racing Black', 'THP', 'Ultra Prestige', 'VTi', 'White', 'e-HDi']}}, {'Citroen': {'Ds3 Cabrio': ['Airdream', 'BlueHDi', 'DSign', 'DSire', 'DSport', 'DSport Plus', 'DStyle', 'DStyle Plus', 'DStyle by Benefit', 'PureTech', 'THP', 'VTi', 'e-HDi']}}, {'Citroen': {'Ds4': ['Airdream', 'BlueHDi', 'DSign', 'DSport', 'DStyle', 'DStyle Nav', 'HDi', 'THP', 'VTi', 'e-HDi']}}, {'Citroen': {'Ds5': ['Airdream', 'BlueHDi', 'DSign', 'DSport', 'DStyle', 'HDi', 'e-HDi', 'h e-HDi']}}, {'Citroen': {'Dispatch': ['BlueHDi 1200', 'BlueHDi 1300', 'BlueHDi 1400', 'Driver M', 'Driver Pro M', 'Enterprise M', 'Enterprise Plus XL', 'Enterprise Pro M', 'Enterprise Pro XL', 'Enterprise XL', 'HDi', 'L1H1', 'M', 'SX']}}, {'Citroen': {'Grand C4 Picasso': ['Airdream', 'BlueHDi', 'Edition', 'Exclusive', 'Exclusive+', 'Feel', 'Flair', 'HDi', 'Platinum', 'PureTech', 'SX', 'Selection', 'THP', 'Touch Edition', 'VTR', 'VTR+', 'VTi', 'e-HDi', 'i']}}, {'Citroen': {'Grand C4 Spacetourer': ['BlueHDi', 'Feel', 'Feel Plus', 'Flair', 'Flair Plus', 'Live', 'PureTech', 'Sense', 'Shine', 'Touch Edition', 'Touch Plus']}}, {'Citroen': {'Nemo Multispace': ['HDi', 'i']}}, {'Citroen': {'Relay': ['30 L1', 'BlueHDi 35', 'Enterprise', 'HDi 30']}}, {'Citroen': {'Saxo': ['Desire', 'VTR', 'i']}}, {'Citroen': {'Spacetourer': ['BlueHDi', 'Business Lounge M', 'Business M', 'Business XS', 'Feel M', 'Feel XL', 'Flair M', 'Flair XS', 'Rip Curl M']}}, {'Citroen': {'Xantia': ['SX', 'i']}}, {'Citroen': {'Xsara': ['LX', 'i']}}, {'Citroen': {'Xsara Picasso': ['Desire', 'Exclusive', 'HDi', 'LX', 'SX', 'VTX', 'i']}}, {'Citroen': {'E-berlingo': ['700', 'Enterprise XL Edition', 'Enterprise XL Pro', 'Feel', 'Flair XTR']}}, {'Citroen': {'e-C4': ['C-Series Edition', 'Sense', 'Sense Plus', 'Shine', 'Shine Edition', 'Shine Plus']}}, {'Citroen': {'E-spacetourer': ['Business Edition M', 'Feel M', 'Flair M', 'Flair XL']}}, {'Corvette': {'C7': ''}}, {'Corvette': {'Stingray': ''}}, {'Crendon': {'Cobra': ''}}, {'Custom Vehicle': {'Replica': ''}}, {'DFSK': {'Glory': ''}}, {'DS Automobiles': {'DS 3': ['1955', 'Bastille', 'Black Lezard', 'BlueHDi', 'Cafe Racer', 'Chic', 'Connected Chic', 'DSign', 'DSport', 'DStyle', 'DStyle Nav', 'Dark Light', 'E-TENSE', 'Elegance', 'Forever', 'Givenchy Le Makeup', 'Ines de la Fressange', 'Parthenon Cream', 'Performance', 'Performance Line', 'Performance Line +', 'Prestige', 'PureTech', 'Rivoli', 'THP', 'Ultra Prestige']}}, {'DS Automobiles': {'DS 3 CABRIO': ['BlueHDi', 'Chic', 'DStyle', 'DStyle Nav', 'Elegance', 'Performance Line', 'Prestige', 'PureTech', 'THP']}}, {'DS Automobiles': {'DS 3 CROSSBACK': ['Bastille', 'BlueHDi', 'E-TENSE', 'Elegance', 'La Premiere', 'Louvre', 'Performance Line', 'Performance Line +', 'Prestige', 'PureTech', 'Rivoli', 'Ultra Prestige']}}, {'DS Automobiles': {'DS 4': ['Bastille +', 'BlueHDi', 'Cross Opera', 'Cross Rivoli', 'Cross Trocadero', 'DSign', 'DStyle Nav', 'E-TENSE', 'Elegance', 'La Premiere', 'Opera', 'Performance Line', 'Performance Line +', 'Prestige', 'PureTech', 'Rivoli', 'Trocadero']}}, {'DS Automobiles': {'DS 4 CROSSBACK': ['BlueHDi', 'Moondust', 'PureTech', 'Terre Rouge']}}, {'DS Automobiles': {'DS 5': ['1955', 'BlueHDi', 'Elegance', 'Prestige', 'THP']}}, {'DS Automobiles': {'Ds 7': ['E-TENSE', 'Opera', 'Performance Line']}}, {'DS Automobiles': {'DS 7 CROSSBACK': ['BlueHDi', 'E-TENSE', 'Elegance', 'La Premiere', 'Louvre', 'Opera', 'Performance Line', 'Performance Line +', 'Prestige', 'PureTech', 'Rivoli', 'Ultra Prestige']}}, {'DS Automobiles': {'Ds 9': ['E-TENSE', 'Performance Line +', 'PureTech', 'Rivoli +']}}, {'Dacia': {'Duster': ['Access', 'Air', 'Ambiance', 'Ambiance Prime', 'Blue dCi', 'Comfort', 'Essential', 'Expression', 'Extreme SE', 'Journey', 'Laureate', 'Laureate Prime', 'Nav+', 'Prestige', 'SCe', 'SE Summit', 'SE Twenty', 'TCe', 'TCe Bi-Fuel', 'Techroad', 'dCi']}}, {'Dacia': {'Jogger': ['Comfort', 'Essential', 'Expression', 'Extreme SE', 'TCe']}}, {'Dacia': {'Logan Mcv': ['Access', 'Ambiance', 'Ambiance Prime', 'Blue dCi', 'Comfort', 'Essential', 'Laureate', 'Laureate Prime', 'SCe', 'TCe', 'dCi']}}, {'Dacia': {'Logan Mcv Stepway': ['Blue dCi', 'Comfort', 'Laureate', 'SE Summit', 'SE Twenty', 'TCe', 'Techroad', 'dCi']}}, {'Dacia': {'Sandero': ['Access', 'Ambiance', 'Blue dCi', 'Comfort', 'Essential', 'Expression', 'Laureate', 'Laureate Prime', 'Midnight', 'SCe', 'TCe', 'dCi']}}, {'Dacia': {'Sandero Stepway': ['Ambiance', 'Blue dCi', 'Comfort', 'Essential', 'Expression', 'Journey', 'Laureate', 'Prestige', 'SCe', 'SE Summit', 'SE Twenty', 'TCe', 'Techroad', 'dCi']}}, {'Daewoo': {'Lacetti': 'SX'}}, {'Daewoo': {'Matiz': ['SE', 'Xtra']}}, {'Daihatsu': {'Charade': 'SL'}}, {'Daihatsu': {'Copen': ['Roadster', 'Vivid']}}, {'Daihatsu': {'Cuore': 'Avanzato'}}, {'Daihatsu': {'Fourtrak': 'TDX'}}, {'Daihatsu': {'Materia': ''}}, {'Daihatsu': {'Move': ''}}, {'Daihatsu': {'Sirion': ['EL', 'F-Speed', 'S', 'SE']}}, {'Daihatsu': {'Terios': ['Centenary Limited Edition', 'S', 'SE', 'SX', 'Sport', 'Sport Ecosse']}}, {'Daihatsu': {'Yrv': 'Turbo'}}, {'Daimler': {'Ds420': ''}}, {'Daimler': {'Double Six': ''}}, {'Daimler': {'Limousine': ''}}, {'Daimler': {'Saloon': ''}}, {'Daimler': {'Sovereign': ''}}, {'Daimler': {'Super Eight': 'V8'}}, {'Daimler': {'V8-250': ''}}, {'Daimler': {'Xj Series': ['Six', 'V8']}}, {'Datsun': {'280z': ''}}, {'DAX': {'Gt40': ''}}, {'DAX': {'Rush': ''}}, {'DAX': {'Tojeiro': ''}}, {'De Tomaso': {'Pantera': ''}}, {'Dodge': {'Avenger': ''}}, {'Dodge': {'Caliber': ['SXT', 'SXT Sport']}}, {'Dodge': {'Caravan': ''}}, {'Dodge': {'Challenger': ''}}, {'Dodge': {'Charger': ''}}, {'Dodge': {'Custom Royale': ''}}, {'Dodge': {'Durango': ''}}, {'Dodge': {'Journey': ['CRD', 'RT', 'SE', 'SXT']}}, {'Dodge': {'Nitro': ['CRD', 'SE', 'SXT', 'V6']}}, {'Dodge': {'Ram': ['Laramie', 'SRT10', 'V10', 'V8 Hemi']}}, {'Dodge': {'Viper': ['SRT 10', 'V10']}}, {'Ferrari': {'250': ''}}, {'Ferrari': {'296 Gtb': 'T V6'}}, {'Ferrari': {'308': 'GTB'}}, {'Ferrari': {'328': 'GTS'}}, {'Ferrari': {'330': ''}}, {'Ferrari': {'348': 'ts'}}, {'Ferrari': {'355 F1': ''}}, {'Ferrari': {'360': ['Challenge Stradale F1', 'Modena', 'Modena F1', 'Spider']}}, {'Ferrari': {'365': ''}}, {'Ferrari': {'400': ['Auto', 'GT']}}, {'Ferrari': {'430': 'Scuderia'}}, {'Ferrari': {'456': 'GTA'}}, {'Ferrari': {'456m': 'GTA'}}, {'Ferrari': {'458': ['HELE V8', 'Italia', 'Speciale', 'Speciale A Spider', 'Spider']}}, {'Ferrari': {'488': ['GTB', 'T V8']}}, {'Ferrari': {'488 Pista': 'T V8'}}, {'Ferrari': {'512': ''}}, {'Ferrari': {'550': 'Maranello'}}, {'Ferrari': {'575m': 'Maranello F1'}}, {'Ferrari': {'599': ['GTB Fiorano', 'GTO', 'V12']}}, {'Ferrari': {'612': ['Scaglietti', 'Scaglietti FI']}}, {'Ferrari': {'812 Gts': 'V12'}}, {'Ferrari': {'812 Superfast': 'V12'}}, {'Ferrari': {'California': ['30', 'HELE V8', 'T', 'V8']}}, {'Ferrari': {'Daytona': ''}}, {'Ferrari': {'Dino': ''}}, {'Ferrari': {'Dino 246 Gts': ''}}, {'Ferrari': {'F12 Berlinetta': 'V12'}}, {'Ferrari': {'F12 Tdf': 'V12'}}, {'Ferrari': {'F355': ['Berlinetta', 'GTS']}}, {'Ferrari': {'F40': ''}}, {'Ferrari': {'F430': ''}}, {'Ferrari': {'F512': ''}}, {'Ferrari': {'F8 Spider': 'T V8'}}, {'Ferrari': {'F8 Tributo': 'T V8'}}, {'Ferrari': {'Ff': 'V12'}}, {'Ferrari': {'Gtc4lusso': ['T V8', 'V12']}}, {'Ferrari': {'Mondial': 'T'}}, {'Ferrari': {'Portofino': 'T V8'}}, {'Ferrari': {'Portofino M': 'T V8'}}, {'Ferrari': {'Roma': 'T V8'}}, {'Ferrari': {'Sf90 Spider': 'T V8'}}, {'Ferrari': {'Sf90 Stradale': 'T V8'}}, {'Ferrari': {'Testarossa': ''}}, {'Fiat': {'124 Spider': ['Classica', 'Lusso', 'Lusso Plus', 'MultiAir']}}, {'Fiat': {'500': ['120th', 'Anniversario', 'Bicolour', 'BlackJack', 'ByDiesel', 'Collezione', 'Collezione Fall', 'Colour Therapy', 'Connect', 'Cult', 'Dolcevita', 'Dolcevita Plus', 'GQ', 'Hey Google', 'Launch Edition', 'Lounge', 'MHEV', 'Matt Black', 'Mirror', 'MultiJet', 'Plus', 'Pop', 'Pop Star', 'RED', 'Riva', 'Rock Star', 'Ron Arad Edition', 'S', 'Sport', 'Star', 'Start and Stop', 'Street', 'Top', 'TwinAir', 'Vintage 57', 'byGucci']}}, {'Fiat': {'500 Topolino': ''}}, {'Fiat': {'500c': ['120th', '60th', 'Anniversario', 'Collezione', 'Colour Therapy', 'Cult', 'Dolcevita', 'Dolcevita Plus', 'GQ', 'Hey Google', 'Launch Edition', 'Lounge', 'MHEV', 'Mirror', 'MultiJet', 'Pop', 'Pop Star', 'RED', 'Riva', 'Rock Star', 'S', 'Spiaggina 58', 'Sport', 'Star', 'TwinAir', 'byGucci']}}, {'Fiat': {'500l': ['Beats Edition', 'City Cross', 'Cross', 'Easy', 'Lounge', 'MultiJet', 'Pop Star', 'S-Design', 'T-Jet', 'Trekking', 'TwinAir', 'Urban']}}, {'Fiat': {'500x': ['120th', 'City Cross', 'Club', 'Connect', 'Cross', 'Cross Plus', 'E-Torq', 'FireFly Turbo', 'Hey Google', 'Lounge', 'MHEV', 'Mirror', 'MultiAir', 'MultiJet', 'MultiJetII', 'Pop', 'Pop Star', 'Pop Star Plus', 'RED', 'S-Design', 'Sport', 'Top', 'Urban']}}, {'Fiat': {'500x Dolcevita': ['FireFly Turbo', 'MHEV', 'RED', 'Sport', 'Top']}}, {'Fiat': {'500e': ['Action', 'Icon', 'La Prima', 'La Prima by Bocelli', 'Passion', 'RED']}}, {'Fiat': {'500e C': ['Icon', 'La Prima', 'La Prima by Bocelli', 'Passion', 'RED']}}, {'Fiat': {'Barchetta': 'LE'}}, {'Fiat': {'Brava': 'SX'}}, {'Fiat': {'Bravo': ['Active', 'Active Sport', 'Bravo', 'Dynamic', 'MultiAir', 'MultiJet', 'Sport', 'T-Jet']}}, {'Fiat': {'Coupe': 'Turbo'}}, {'Fiat': {'Doblo': ['(Eco)', 'Active', 'Dynamic', 'ELX', 'Easy', 'Eleganza', 'JTD', 'Lounge', 'Maxi', 'Maxi Active', 'Maxi SX', 'Maxi Tecnico', 'MultiJet', 'MultiJetII', 'MyLife', 'N1 Active', 'N1 SX', 'Pop', 'SX', 'Tecnico']}}, {'Fiat': {'Ducato': ['15', 'JTD', 'Maxi', 'MultiJetII 33', 'MultiJetII 35']}}, {'Fiat': {'Fiorino': ['Active', 'Adventure', 'Cargo', 'Cargo Adventure', 'Cargo SX', 'JTD', 'MultiJetII', 'Multijet', 'Multijet II', 'MultijetII', 'SX', 'Tecnico']}}, {'Fiat': {'Freemont': ''}}, {'Fiat': {'Fullback': ['Cross', 'D', 'LX', 'SX']}}, {'Fiat': {'Grande Punto': ['Active', 'Active Sport', 'Dynamic', 'Dynamic Sport', 'Eleganza', 'GP', 'MultiJet', 'Sound', 'Sporting', 'T-Jet']}}, {'Fiat': {'Idea': ['Dynamic', 'JTD', 'MultiJet']}}, {'Fiat': {'Multipla': ['ELX', 'Eleganza', 'JTD', 'MultiJet']}}, {'Fiat': {'Panda': ['4x4', '4x4 Antarctica', '4x4 Cross', 'Active', 'City Cross', 'City Life', 'Cross', 'Dynamic', 'Easy', 'Easy +', 'Eleganza', 'Garmin', 'Lounge', 'MHEV', 'MultiJet', 'MyLife', 'Panda', 'Pop', 'RED', 'Sport', 'Trekking', 'Trussardi', 'TwinAir', 'Waze', 'Wild']}}, {'Fiat': {'Punto': ['Active', 'Active Sport', 'Dynamic', 'ELX', 'Easy', 'Easy +', 'GBT', 'Jet Black', 'Jet Black II', 'Lounge', 'MultiAir', 'MultiJet', 'Pop', 'Pop +', 'Sporting', 'TwinAir']}}, {'Fiat': {'Punto Evo': ['Active', 'Dynamic', 'Eleganza', 'GP', 'MultiAir', 'MultiJet', 'My Life', 'Sporting']}}, {'Fiat': {'Qubo': ['Active', 'Dynamic', 'Lounge', 'MultiJet', 'MultiJetII', 'My Life', 'Trekking']}}, {'Fiat': {'Scudo': ['EL', 'Family', 'JTD', 'MultiJet', 'Panorama Family']}}, {'Fiat': {'Sedici': ['Dynamic', 'Eleganza', 'MultiJet']}}, {'Fiat': {'Seicento': 'SX'}}, {'Fiat': {'Stilo': ['Abarth', 'Active', 'Dynamic', 'JTD']}}, {'Fiat': {'Talento': ['Active', 'Ecojet 10', 'MultiJet 12', 'Onyx Edition', 'SX', 'Tecnico']}}, {'Fiat': {'Tipo': ['City Sport', 'Cross', 'Easy', 'Easy Plus', 'Elite', 'FireFly Turbo', 'Life', 'Lounge', 'MHEV', 'MPI', 'Mirror', 'MultiJetII', 'RED', 'S-Design', 'Sport', 'Street', 'T-Jet']}}, {'Fiat': {'Ulysse': ['Dynamic', 'JTD']}}, {'Fiat': {'X1/9': 'Gran Finale'}}, {'Ford': {'B-max': ['EcoBoost', 'Studio', 'T', 'TDCi', 'Titanium', 'Titanium Navigator', 'Titanium X', 'Titanium X Navigator', 'Zetec', 'Zetec Navigator']}}, {'Ford': {'Bronco': ''}}, {'Ford': {'C-max': ['ECOnetic', 'EcoBoost', 'Style', 'T', 'TDCi', 'Ti-VCT', 'Titanium', 'Titanium X', 'Zetec']}}, {'Ford': {'Capri': ['Injection Special', 'L', 'Limited Edition']}}, {'Ford': {'Consul': ''}}, {'Ford': {'Corsair': ''}}, {'Ford': {'Cortina': ['Crusader', 'L']}}, {'Ford': {'Custom Cab': ''}}, {'Ford': {'Dorchester': ''}}, {'Ford': {'E-transit': ['390', 'Trend']}}, {'Ford': {'Ecosport': ['Active', 'EcoBlue', 'EcoBoost', 'ST-Line', 'ST-Line Black', 'ST-Line Design', 'T', 'TDCi', 'Titanium', 'Titanium S', 'Zetec']}}, {'Ford': {'Econoline': ''}}, {'Ford': {'Edge': ['EcoBlue', 'ST-Line', 'Sport', 'TDCi', 'Titanium', 'Vignale', 'Zetec']}}, {'Ford': {'Escort': ['55', 'Calypso', 'D', 'Dash', 'Eclipse', 'Finesse', 'Freedom', 'GTi', 'Ghia', 'L', 'LX', 'RS', 'Silhouette', 'Turbo', 'XR3i', 'i']}}, {'Ford': {'Explorer': 'The North Face'}}, {'Ford': {'F100': ''}}, {'Ford': {'F150': ''}}, {'Ford': {'F350': ''}}, {'Ford': {'Fiesta': ['Active', 'Active 1', 'Active B&O Play', 'Active Edition', 'Active Vignale', 'Active X', 'Active X Edition', 'Centura', 'Chequered Flag', 'ECOnetic', 'EcoBoost', 'EcoBoost MHEV', 'Edge', 'Equipe', 'Finesse', 'Firefly', 'Flame', 'Flight', 'Freedom', 'Freestyle', 'Ghia', 'L', 'LX', 'Metal', 'RS', 'ST', 'ST Edition', 'ST Performance Edition', 'ST-1', 'ST-2', 'ST-200', 'ST-3', 'ST-500', 'ST-Line', 'ST-Line Black Edition', 'ST-Line Edition', 'ST-Line Red Edition', 'ST-Line Vignale', 'ST-Line X', 'ST-Line X Edition', 'Silver', 'Studio', 'Style', 'Style +', 'Style Climate', 'T', 'TD', 'TDCi', 'Ti-VCT', 'Titanium', 'Titanium B&O Play Series', 'Titanium Vignale', 'Titanium X', 'Trend', 'Vignale', 'Vignale Edition', 'XR2i', 'Zetec', 'Zetec B&O Play Series', 'Zetec Black Edition', 'Zetec Blue Edition', 'Zetec Climate', 'Zetec LX', 'Zetec S', 'Zetec S Black Edition', 'Zetec S Red Edition', 'Zetec White Edition', 'i']}}, {'Ford': {'Fiesta Van': ['Base', 'ECOnetic', 'ECOnetic II', 'EcoBoost', 'Sport', 'T', 'TD', 'TDCI', 'TDCi', 'Ti-VCT', 'Trend']}}, {'Ford': {'Focus': ['Active', 'Active Design', 'Active Edition', 'Active Style', 'Active Vignale', 'Active X', 'Active X Edition', 'Active X Vignale', 'Active X Vignale Edition', 'CL', 'ECOnetic', 'EcoBlue', 'EcoBoost', 'EcoBoost MHEV', 'Edge', 'Flight', 'Ghia', 'LX', 'RS', 'RS Edition', 'ST', 'ST Edition', 'ST-1', 'ST-170', 'ST-2', 'ST-3', 'ST-500', 'ST-Line', 'ST-Line Design', 'ST-Line Edition', 'ST-Line Style', 'ST-Line Vignale', 'ST-Line X', 'ST-Line X Edition', 'Sport', 'Sport S', 'Studio', 'Style', 'T', 'TDCi', 'TDdi', 'Titanium', 'Titanium Design', 'Titanium Edition', 'Titanium Navigator', 'Titanium Style', 'Titanium Vignale', 'Titanium X', 'Titanium X Edition', 'Titanium X Navigator', 'Trend', 'Vignale', 'Vignale Edition', 'Zetec', 'Zetec Climate', 'Zetec Edition', 'Zetec Navigator', 'Zetec S', 'i']}}, {'Ford': {'Focus C-max': ['Ghia', 'LX', 'Style', 'TD', 'TDCi', 'Zetec']}}, {'Ford': {'Focus Cc': ['CC-2', 'CC-3', 'TDCi']}}, {'Ford': {'Freda': ''}}, {'Ford': {'Fusion': ['1', '2', '3', 'Plus', 'Pursuit', 'Pursuit Climate', 'Style', 'Style +', 'Style Climate', 'TD', 'TDCi', 'Titanium', 'Zetec', 'Zetec Climate']}}, {'Ford': {'Gt': ''}}, {'Ford': {'Galaxy': ['Duratec', 'EcoBlue', 'EcoBoost', 'Edge', 'Ghia', 'LX', 'T', 'TDCi', 'TDi', 'Titanium', 'Titanium X', 'Zetec', 'h', 'i']}}, {'Ford': {'Granada': 'GL'}}, {'Ford': {'Grand C-max': ['EcoBoost', 'T', 'TDCi', 'Titanium', 'Titanium X', 'Zetec']}}, {'Ford': {'Grand Tourneo Connect': ['Active', 'EcoBlue', 'EcoBoost', 'Style', 'TDCi', 'Titanium', 'Zetec']}}, {'Ford': {'Ka': ['3', 'Collection', 'Design II', 'Edge', 'Grand Prix', 'Grand Prix II', 'Luxury', 'Metal', 'Sport', 'Sport SE', 'Studio', 'Studio Connect', 'Style', 'Style +', 'Style Climate', 'Sublime', 'TDCi', 'Titanium', 'Zetec', 'Zetec Black Edition', 'Zetec Climate', 'Zetec White Edition']}}, {'Ford': {'Ka+': ['Active', 'Studio', 'TDCi', 'Ti-VCT', 'Zetec']}}, {'Ford': {'Kuga': ['Duratec', 'EcoBlue', 'EcoBlue MHEV', 'EcoBoost', 'EcoBoost Duratec', 'ST-Line', 'ST-Line Edition', 'ST-Line First Edition', 'ST-Line X', 'ST-Line X Edition', 'ST-Line X First Edition', 'T', 'TDCi', 'Titanium', 'Titanium Edition', 'Titanium First Edition', 'Titanium Sport', 'Titanium X', 'Titanium X Edition', 'Titanium X Sport', 'Vignale', 'Zetec', 'h']}}, {'Ford': {'Maverick': 'XLT'}}, {'Ford': {'Mondeo': ['ECO', 'ECOnetic', 'EcoBlue', 'EcoBoost', 'Edge', 'Edge +', 'Ghia', 'Ghia X', 'Graphite', 'LX', 'ST', 'ST-200', 'ST-220', 'ST-Line', 'ST-Line Edition', 'ST-Line X', 'Silver', 'Sport', 'Style', 'T', 'TD', 'TDCi', 'TiVCT', 'Titanium', 'Titanium Edition', 'Titanium X', 'Titanium X Business Edition', 'Titanium X Sport', 'V6', 'Vignale', 'Zetec', 'Zetec Business Edition', 'Zetec Edition', 'i', 'i V6']}}, {'Ford': {'Mustang': ['55 Edition', 'Bullitt', 'EcoBoost', 'GT', 'GT California Special', 'GT Shadow Edition', 'Mach 1', 'T', 'V8']}}, {'Ford': {'Mustang Mach-e': ['Extended Range', 'First Edition', 'GT', 'Premium', 'Select', 'Standard Range']}}, {'Ford': {'Popular': ''}}, {'Ford': {'Probe': ''}}, {'Ford': {'Puma': ['Black', 'EcoBoost', 'EcoBoost MHEV', 'Racing', 'ST', 'ST Gold Edition', 'ST Performance Pack', 'ST-Line', 'ST-Line Design', 'ST-Line Design 2', 'ST-Line Vignale', 'ST-Line X', 'ST-Line X First Edition', 'ST-Line X First Edition Plus', 'ST-Line X Vignale', 'T', 'Thunder', 'Titanium', 'Titanium Design', 'Titanium First Edition']}}, {'Ford': {'Ranger': ['Black Edition', 'D', 'EcoBlue', 'EcoBoost', 'Limited', 'Limited 1', 'Limited 2', 'MS-RT', 'Raptor', 'Raptor Special Edition', 'Stormtrak', 'T V6', 'TD', 'TD V6', 'TDCi', 'TDdi', 'Thunder', 'Wildtrak', 'Wildtrak X', 'Wolftrak', 'XL', 'XLT', 'XLT Thunder']}}, {'Ford': {'S-max': ['Duratec', 'EcoBlue', 'EcoBoost', 'Edge', 'LX', 'ST-Line', 'T', 'TDCi', 'Titanium', 'Titanium Sport', 'Titanium X Sport', 'Vignale', 'Zetec', 'h']}}, {'Ford': {'Scorpio': 'Ultima'}}, {'Ford': {'Sierra': ['RS', 'XR4i', 'XRi', 'XRi Ghia']}}, {'Ford': {'Streetka': ['ICE', 'Luxury', 'Winter']}}, {'Ford': {'Thunderbird': ''}}, {'Ford': {'Tourneo Connect': ['Active', 'EcoBlue', 'EcoBoost', 'L', 'Style', 'TDCi', 'Titanium', 'Trend', 'Zetec']}}, {'Ford': {'Tourneo Courier': ['EcoBoost', 'TDCi', 'Titanium', 'Zetec']}}, {'Ford': {'Tourneo Custom': ['300 TDCi', '310', '320', 'Active', 'EcoBlue', 'EcoBlue MHEV', 'EcoBoost', 'Leader', 'Limited', 'Sport', 'Titanium', 'Titanium X', 'Trend', 'Zetec']}}, {'Ford': {'Transit': ['350', 'Duratorq', 'EcoBlue', 'Leader', 'Limited', 'TDCi 260', 'TDCi 280', 'TDCi 300', 'TDCi 350', 'Trend']}}, {'Ford': {'Transit Connect': ['220', '230', '240', 'EcoBlue', 'EcoBoost', 'Leader', 'TDCi 220', 'TDCi 230', 'TDCi T210', 'TDCi T220', 'TDCi T230', 'Trend']}}, {'Ford': {'Transit Courier': ['Base', 'Leader', 'TDCi']}}, {'Ford': {'Transit Custom': ['280', '300', '310', '310 L2H1 Limited', '310 Limited', '320', '340', 'Active', 'EcoBlue', 'EcoBlue MHEV', 'Leader', 'Limited', 'MS-RT', 'Sport', 'TDCi 270', 'TDCi 290', 'TDCi 310', 'TDCi 330', 'TDCi 340', 'Trail', 'Trend']}}, {'GMC': {'Sierra': ''}}, {'GMC': {'Yukon': ''}}, {'Gwm Ora': {'Ora Funky Cat': 'First Edition'}}, {'Gardener': {'Douglas': ''}}, {'Great Wall': {'Steed': ['S', 'SE', 'TD', 'Tracker']}}, {'Hillman': {'Minx': ''}}, {'Holden': {'Commodore': ''}}, {'Honda': {'Accord': ['ES', 'ES GT', 'EX', 'EX GT', 'Executive', 'S', 'SE', 'SE Executive', 'Sport', 'Sport GT', 'Type R', 'Type S', 'i', 'i V6', 'i-CDTi', 'i-DTEC', 'i-VTEC']}}, {'Honda': {'Cr-v': ['Black Edition', 'ES', 'ES Executive', 'ES-T', 'EX', 'Executive', 'LS', 'Premiere', 'S', 'S Navi', 'S Plus', 'S-T', 'SE', 'SE Executive', 'SE Navi', 'SE Plus', 'SE Plus Navi', 'SE Plus T', 'SE Sport', 'SE+', 'SE-T', 'SR', 'Sport', 'Sport Line', 'VTEC Turbo', 'h i-MMD', 'i', 'i-CDTi', 'i-DTEC', 'i-VTEC']}}, {'Honda': {'Cr-x': ['ESi', 'VTi']}}, {'Honda': {'Cr-z': ['GT', 'GT T', 'S', 'Sport', 'h IMA']}}, {'Honda': {'Civic': ['Advance', 'Black Edition', 'CRX', 'CTDi', 'ES', 'ES GT', 'ES-T', 'ESi', 'EX', 'EX GT', 'EX Plus', 'EX Sport Line', 'Elegance', 'Executive', 'IMA', 'Imagine', 'Inspire S', 'Jordan', 'LSi', 'Prestige', 'S', 'SE', 'SE Executive', 'SE Plus', 'SE Plus-T', 'SE Sport', 'SE-T', 'SR', 'Si', 'Si-T', 'Sport', 'Sport Line', 'Sport Plus', 'Ti', 'Tropica', 'Type R', 'Type R GT', 'Type R Limited Edition', 'Type R Sport Line', 'Type S', 'Type S GT', 'Type S GT-T', 'VTEC', 'VTEC Turbo', 'VTI', 'VTI-S', 'Vision', 'h i-MMD', 'i', 'i-CDTi', 'i-CTDi', 'i-DSI', 'i-DTEC', 'i-VTEC']}}, {'Honda': {'Elysion': ''}}, {'Honda': {'Fr-v': ['ES', 'EX', 'SE', 'Sport', 'i-CDTi', 'i-VTEC']}}, {'Honda': {'Fit': ''}}, {'Honda': {'Freed': ''}}, {'Honda': {'Hr-v': ['Advance', 'Advance Style', 'Black Edition', 'EX', 'Elegance', 'S', 'SE', 'SE Navi', 'Sport', 'VTEC Turbo', 'h i-MMD', 'i', 'i-DTEC', 'i-VTEC']}}, {'Honda': {'Honda E': 'Advance'}}, {'Honda': {'Insight': ['ES', 'ES-T', 'EX', 'HE-T', 'HS', 'HS-T', 'HX', 'SE', 'h IMA']}}, {'Honda': {'Integra': 'Type R'}}, {'Honda': {'Jazz': ['Crosstar EX', 'ES', 'ES Plus', 'ES Plus T', 'ES-T', 'EX', 'EX Navi', 'EX Style', 'EX-T', 'EXL', 'EXL-T', 'HE', 'HE-T', 'HS', 'HS-T', 'HX', 'S', 'SE', 'SE Navi', 'SE Sport', 'SE-T', 'SR', 'Si', 'Si-T', 'Sport', 'Sport Navi', 'h IMA', 'h i-MMD', 'i-DSI', 'i-VTEC']}}, {'Honda': {'Legend': ['EX', 'i', 'i-VTEC V6']}}, {'Honda': {'Mobilio': ''}}, {'Honda': {'Nsx': ['F-Matic', 'T', 'h V6']}}, {'Honda': {'Odyssey': ''}}, {'Honda': {'Prelude': ['16-4WS-SE', '4WS', 'Executive', 'Motegi', 'Sport', 'VTI', 'i']}}, {'Honda': {'S2000': ['GT', 'Roadster', 'i-VTEC']}}, {'Honda': {'S800': ''}}, {'Honda': {'Stepwagon': ''}}, {'Honda': {'Stream': ['SE Sport', 'i-VTEC']}}, {'Hummer': {'H1': ''}}, {'Hummer': {'H2': ['Luxury', 'V8']}}, {'Hummer': {'H3': ['Luxury', 'SE', 'V6']}}, {'Hyundai': {'Accent': ['Atlantic', 'CDX', 'GSi']}}, {'Hyundai': {'Amica': ['CDX', 'GSi', 'Si']}}, {'Hyundai': {'Bayon': ['MHEV', 'Premium', 'SE Connect', 'T-GDi', 'Ultimate']}}, {'Hyundai': {'Coupe': ['S', 'SE', 'SIII', 'SIII S', 'V6']}}, {'Hyundai': {'Elantra': ''}}, {'Hyundai': {'Genesis': 'GDi V6'}}, {'Hyundai': {'Getz': ['CDX', 'GSi', 'SE']}}, {'Hyundai': {'Ioniq': ['1st Edition', 'Premium', 'Premium SE', 'SE', 'SE Connect', 'h-GDi']}}, {'Hyundai': {'Ioniq 5': ['Namsan Edition', 'Premium', 'Project 45', 'SE Connect', 'Ultimate']}}, {'Hyundai': {'Ioniq 6': ['Premium', 'Ultimate']}}, {'Hyundai': {'Kona': ['Blue Drive', 'CRDi', 'Iron Man Edition', 'MHEV', 'N', 'N Line', 'Play', 'Premium', 'Premium GT', 'Premium SE', 'S', 'SE', 'SE Connect', 'T-GDi', 'Ultimate', 'h-GDi']}}, {'Hyundai': {'Matrix': ['Atlantic', 'CDX', 'CRTD', 'Comfort', 'GSi']}}, {'Hyundai': {'Santa Fe': ['Blue Drive', 'CDX', 'CDX+', 'CRDi', 'CRTD', 'Endurance Edition', 'GSI', 'Limited', 'Premium', 'Premium SE', 'Style', 'T-GDi', 'Ultimate', 'V6', 'Wiggins Edition', 'h T-GDi']}}, {'Hyundai': {'Sonata': ['CDX', 'CRTD']}}, {'Hyundai': {'Tucson': ['Blue Drive', 'CDX', 'CRDi', 'CRTD', 'GDi', 'GO! SE', 'GSi', 'Limited', 'MHEV', 'N Line', 'N Line S', 'Premium', 'Premium SE', 'S', 'S Connect', 'SE', 'SE Connect', 'SE Nav', 'Sport Edition', 'Style', 'T-GDi', 'Ultimate', 'V6', 'Xenith', 'h T-GDi']}}, {'Hyundai': {'Terracan': ['CRTD', 'Limited Edition']}}, {'Hyundai': {'Trajet': ['CRTD', 'SE']}}, {'Hyundai': {'Veloster': ['GDi', 'SE', 'Sport', 'T-GDi', 'Turbo', 'Turbo SE']}}, {'Hyundai': {'I10': ['Active', 'Blue Drive', 'Classic', 'Comfort', 'ES', 'Edition', 'GO!', 'GO! SE', 'N Line', 'Play', 'Premium', 'Premium SE', 'S', 'S Air', 'SE', 'SE Blue', 'SE Connect', 'Style', 'T-GDi']}}, {'Hyundai': {'I20': ['Active', 'Blue Drive', 'CRDi', 'Classic', 'Comfort', 'Edition', 'Element', 'GO!', 'GO! SE', 'MHEV', 'N', 'N Line', 'Play', 'Premium', 'Premium Nav', 'Premium SE', 'Premium SE Nav', 'S', 'S Air', 'S Connect', 'SE', 'SE Connect', 'SE Launch Edition', 'Sport', 'Sport Nav', 'Style', 'T-GDi', 'Turbo Edition', 'Ultimate']}}, {'Hyundai': {'I30': ['Active', 'Blue Drive', 'CRDi', 'Classic', 'Comfort', 'Drive-N Limited Edition', 'ES', 'Edition', 'GO! SE', 'MHEV', 'N', 'N Line', 'N Line +', 'N Performance', 'Premium', 'Premium SE', 'S', 'SE', 'SE Connect', 'SE Nav', 'Sport', 'Sport Nav', 'Style', 'Style Nav', 'T-GDi', 'Turbo']}}, {'Hyundai': {'I40': ['Active', 'Blue Drive', 'CRDi', 'GDi', 'Premium', 'Premium SE', 'S', 'SE Nav', 'SE Nav Business', 'Style']}}, {'Hyundai': {'I800': ['CRDi', 'SE', 'SE Nav', 'Style']}}, {'Hyundai': {'Iload': ['CRDI', 'Comfort']}}, {'Hyundai': {'Ix20': ['Active', 'Blue Drive', 'CRDi', 'Classic', 'Premium', 'Premium Nav', 'SE', 'SE Nav', 'Style']}}, {'Hyundai': {'Ix35': ['Blue Drive', 'CRDi', 'GDi', 'GO!', 'Premium', 'Premium Panorama', 'S', 'SE', 'Style']}}, {'Infiniti': {'Ex': ['30d V6', '37 V6', 'Black Premium']}}, {'Infiniti': {'Fx': ['30d V6', '37 V6', '50 V8', 'Executive', 'GT', 'S', 'S Limited Edition', 'S Premium']}}, {'Infiniti': {'G': ['37 V6', 'GT', 'GT Premium', 'S Premium']}}, {'Infiniti': {'M': ['30d V6', '35h V6', 'Business Edition', 'GT Premium']}}, {'Infiniti': {'Q30': ['Business Executive', 'Luxe', 'Luxe Tech', 'Premium', 'Premium InTouch', 'Premium Tech', 'Premium Tech InTouch', 'SE', 'Sport', 'Sport InTouch', 'T', 'd']}}, {'Infiniti': {'Q50': ['Executive', 'Premium', 'Premium Executive', 'S', 'S Sport', 'SE', 'Sport', 'Sport Executive', 'Sport Tech', 'T V6', 'd', 'h V6']}}, {'Infiniti': {'Q60': ['GT Premium', 'V6']}}, {'Infiniti': {'Q70': ['Premium', 'Premium Executive', 'Premium Tech', 'Sport', 'd']}}, {'Infiniti': {'Qx30': ['Luxe Tech', 'Premium', 'Premium Tech', 'd']}}, {'Infiniti': {'Qx50': ['GT', 'd V6']}}, {'Infiniti': {'Qx70': ['GT', 'GT Premium', 'S Design', 'S Premium', 'Ultimate', 'V8', 'd V6']}}, {'Infiniti': {'Vettel': ''}}, {'Isuzu': {'D-max': ['AT35', 'AT35 Safir', 'Arctic Trucks', 'Arctic Trucks AT35', 'Blade', 'Blade+', 'Centurion', 'DL20', 'DL40', 'Eiger', 'Fury', 'TD', 'TD AT35', 'Utah', 'Utah Huntsman', 'Utah Luxe', 'Utah V-Cross', 'Utah Vision', 'Utility', 'Utility Diff Lock', 'V-Cross', 'XTR', 'XTR Nav+', 'Yukon', 'Yukon Huntsman', 'Yukon Nav+', 'Yukon Vision']}}, {'Isuzu': {'Gemini': ''}}, {'Isuzu': {'Rodeo': ['CRD', 'Denver', 'Denver Max', 'Denver Max LE', 'Denver Max Plus', 'Sport', 'TD']}}, {'Isuzu': {'Trooper': ['Citation', 'Duty', 'TD']}}, {'Iveco': {'Daily': ['35C 3520', '35S 3520', 'D HPI 12V']}}, {'JBA': {'Falcon': ''}}, {'Jaguar': {'C-type': ''}}, {'Jaguar': {'E-pace': ['Chequered Flag', 'D150', 'D165', 'D180', 'D200', 'D240', 'First Edition', 'HSE', 'MHEV', 'P200', 'P250', 'P300', 'P300e', 'PHEV', 'R-Dynamic', 'R-Dynamic Black', 'R-Dynamic HSE', 'R-Dynamic S', 'R-Dynamic SE', 'S', 'SE']}}, {'Jaguar': {'E-type': ''}}, {'Jaguar': {'F-pace': ['300 Sport', 'Chequered Flag', 'D165', 'D200', 'D300 V6', 'First Edition', 'HSE', 'MHEV', 'P250 i', 'P400 V6', 'P400e', 'P550 V8', 'Portfolio', 'Prestige', 'R-Dynamic Black', 'R-Dynamic HSE', 'R-Dynamic S', 'R-Dynamic SE', 'R-Sport', 'S', 'SE', 'SVR', 'Sport', 'V6', 'V8', 'd', 'd V6', 'i']}}, {'Jaguar': {'F-type': ['400 Sport', 'British Design Edition', 'Chequered Flag', 'First Edition', 'Heritage 60 Edition', 'R', 'R-Dynamic', 'R-Dynamic Black', 'Reims Edition', 'S', 'SVR', 'V6', 'V8', 'i']}}, {'Jaguar': {'I-pace': ['Black', 'First Edition', 'HSE', 'HSE Black', 'S', 'SE']}}, {'Jaguar': {'Mark Ii': ''}}, {'Jaguar': {'Mark X': ''}}, {'Jaguar': {'S-type': ['D V6', 'Plus', 'R', 'S', 'SE', 'SE Plus', 'Spirit', 'Sport', 'Sport Plus', 'V6', 'V8', 'XS']}}, {'Jaguar': {'X-type': ['Classic', 'D', 'Plus', 'S', 'SE', 'SE Plus', 'Sovereign', 'Sport', 'Sport Plus', 'Sport Premium', 'V6', 'XS LE']}}, {'Jaguar': {'Xe': ['300 Sport', 'GTDi', 'HSE', 'Landmark', 'MHEV', 'Portfolio', 'Prestige', 'Project 8', 'R-Dynamic Black', 'R-Dynamic HSE', 'R-Dynamic S', 'R-Dynamic SE', 'R-Sport', 'Reims Edition', 'S', 'SE', 'V6', 'V8', 'd', 'i']}}, {'Jaguar': {'Xf': ['300 Sport', 'Chequered Flag', 'First Edition', 'Luxury', 'MHEV', 'Portfolio', 'Premium Luxury', 'Prestige', 'R-Dynamic HSE', 'R-Dynamic S', 'R-Dynamic SE', 'R-Sport', 'R-Sport Black', 'R-Sport Black Edition', 'S', 'SE', 'SE Business', 'SV8', 'Sport', 'V6', 'V8', 'XFR', 'd', 'd S V6', 'd V6', 'i']}}, {'Jaguar': {'Xfr-s': 'V8'}}, {'Jaguar': {'Xj': ['Autobiography', 'Executive', 'Luxury', 'Portfolio', 'Premium Luxury', 'R-Sport', 'SE', 'Series III Sovereign', 'Sovereign', 'Sport', 'Sport Premium', 'Super', 'Supersport', 'TD', 'TDVi', 'Ultimate', 'V6', 'V8', 'XJ12', 'XJ40 Sovereign', 'XJ40 XJ6', 'XJ40 XJR Sports', 'XJ50', 'XJ6', 'XJ6 Sovereign', 'XJ6 Sport', 'XJ8', 'XJ8 Executive', 'XJ8 SE', 'XJ8 Sovereign', 'XJ8 Sport', 'XJ8 Sport Premium', 'd V6']}}, {'Jaguar': {'Xj220': ''}}, {'Jaguar': {'Xjr': ['100 Limited Edition', 'Portfolio', 'Super', 'V8']}}, {'Jaguar': {'Xjs': ['HE', 'V12']}}, {'Jaguar': {'Xk': ['60', 'Artisan Special Edition', 'Dynamic R', 'Portfolio', 'Signature', 'Special Edition', 'V8']}}, {'Jaguar': {'Xk150': ''}}, {'Jaguar': {'Xk8': ['Premium', 'S']}}, {'Jaguar': {'Xkr': ['100', '400', '75', 'Carbon Fibre', 'S', 'Supercharged', 'V8']}}, {'Jaguar': {'Xkr-s': ['S', 'V8']}}, {'Jeep': {'Cherokee': ['75th Anniversary', 'CRD', 'Classic', 'Extreme Sport', 'Limited', 'Longitude', 'Longitude Plus', 'MultiJetII', 'Night Eagle', 'Orvis', 'Overland', 'Sport', 'TD', 'Trailhawk', 'V6']}}, {'Jeep': {'Commander': ['CRD V6', 'Limited', 'V8']}}, {'Jeep': {'Compass': ['70th Anniversary', 'CRD', 'GSE', 'GSE T4', 'Limited', 'Longitude', 'MultiAirII', 'MultiJetII', 'Night Eagle', 'S', 'Sport', 'Sport Plus', 'T', 'Trailhawk', 'Upland', 'eTorque']}}, {'Jeep': {'Grand Cherokee': ['75th Anniversary', 'CRD', 'Laredo', 'Limited', 'Limited Plus', 'Limited XS', 'MultiJetII', 'Night Eagle', 'Overland', 'S Limited', 'SRT', 'SRT Night', 'SRT Red Vapor', 'SRT8', 'Summit', 'Trackhawk', 'Trailhawk', 'V6', 'V6 CRD']}}, {'Jeep': {'Patriot': ['CRD', 'Limited', 'Overland', 'S Limited', 'Sport', 'Sport Plus']}}, {'Jeep': {'Renegade': ['80th Anniversary', 'Dawn of Justice', 'E-TorQ', 'GSE T3', 'GSE T4', 'Limited', 'Longitude', 'MHEV', 'MultiAirII', 'MultiJetII', 'Night Eagle', 'Night Eagle II', 'Opening Edition', 'RED', 'S', 'Sport', 'T', 'Tough Mudder', 'Trailhawk', 'Upland', 'eTorque']}}, {'Jeep': {'Willys': ''}}, {'Jeep': {'Wrangler': ['60th Anniversary', '70th Anniversary', '75th Anniversary', '80th Anniversary', 'Black Edition II', 'CRD', 'GME', 'Grizzly', 'JK Edition', 'Jamboree', 'MultiJet II', 'Night Eagle', 'Overland', 'Rubicon', 'Rubicon 1941', 'Rubicon Recon', 'Sahara', 'Sport', 'Ultimate', 'V6', 'X Edition']}}, {'Jensen': {'Interceptor': ''}}, {'KIA': {'Carens': ['1', '2', '3', '4', 'CRDi', 'EcoDynamics', 'GDi', 'GS', 'GS5', 'LE', 'LS', 'S', 'S5', 'SR7']}}, {'KIA': {'Ceed': ['1', '2', '2 NAV', '3', '4', '4 Tech', 'Blue Edition', 'CRDi', 'ECO', 'EcoDynamics', 'First Edition', 'GDi', 'GS', 'GT', 'GT Tech', 'GT-Line', 'GT-Line Lunar Edition', 'GT-Line S', 'LS', 'MHEV', 'S', 'SR', 'SR7', 'SW-7', 'Sport', 'Strike', 'T-GDi', 'VR-7', 'VR7']}}, {'KIA': {'Cerato': 'GS'}}, {'KIA': {'Ev6': ['Air', 'GT', 'GT-Line', 'GT-Line S']}}, {'KIA': {'Magentis': ['CRDi', 'GS', 'LE', 'TR']}}, {'KIA': {'Niro': ['2', '3', '4', '4+', 'Connect', 'First Edition', 'GDi', 'h GDi']}}, {'KIA': {'Optima': ['1', '2', '2 Luxe', '2 Tech', '3', '4', 'CRDi', 'EcoDynamics', 'GT-Line S', 'h GDi']}}, {'KIA': {'Picanto': ['1', '1 Air', '2', '3', '4', 'Chill', 'Chilli', 'City', 'DPi', 'Domino', 'EcoDynamics', 'Equinox', 'GS', 'GT-Line', 'GT-Line S', 'Glamour', 'Graphite', 'Halo', 'Ice', 'LS', 'LX', 'Quantum', 'Red', 'S', 'SE', 'SE+', 'SR7', 'Shadow', 'Spice', 'Sport', 'Strike', 'T-GDi', 'Titanium Edition', 'VR7', 'Wave', 'White', 'X-Line', 'X-Line S', 'Zapp!', 'Zest', 'Zipp+']}}, {'KIA': {'Proceed': ['1', '2', '3', '4', 'CRDi', 'EcoDynamics', 'GDi', 'GT', 'GT 1st Edition', 'GT Line', 'GT Tech', 'GT-Line', 'GT-Line Lunar Edition', 'GT-Line S', 'S', 'SE', 'SE Tech', 'SR7', 'Sport', 'Strike', 'T-GDi', 'VR-7', 'VR7']}}, {'KIA': {'Rio': ['1', '1 Air', '2', '3', '4', 'CRDi', 'Chill', 'Domino', 'EcoDynamics', 'First Edition', 'GS', 'GT-Line', 'GT-Line S', 'Graphite', 'Ice', 'LS', 'LX', 'MHEV', 'Pulse', 'SE', 'SR7', 'Sport', 'Strike', 'T-GDi', 'VR7', 'Zapp!']}}, {'KIA': {'Sedona': ['1', '2', '3', 'CRDi', 'GS', 'L', 'LE', 'LS', 'TS']}}, {'KIA': {'Sorento': ['1', '2', '3', '4', 'Atlas', 'CRDi', 'Edition', 'GT-Line', 'GT-Line S', 'KX-1', 'KX-2', 'KX-3', 'KX-4', 'T-GDi', 'Titan', 'V6', 'XE', 'XS', 'XT', 'h T-GDi']}}, {'KIA': {'Soul': ['1', '2', '3', 'Burner', 'CRDi', 'Connect', 'Connect Plus', 'EV', 'Echo', 'Explore', 'First Edition', 'GDi', 'Hunter', 'Maxx', 'Mixx', 'Quantum', 'Samba', 'Shaker', 'Sport', 'Start', 'T-GDi', 'Tempest', 'Urban']}}, {'KIA': {'Sportage': ['1', '2', '3', '4', 'Alpine Edition', 'Axis Edition', 'Black Edition', 'CRDi', 'EcoDynamics', 'EcoDynamics+', 'Edition 25', 'First Edition', 'GDi', 'GT-Line', 'GT-Line Edition', 'GT-Line S', 'JBL Black Edition', 'KX-1', 'KX-2', 'KX-3', 'KX-4', 'KX-5', 'Launch Edition', 'MHEV', 'Platinum Edition', 'T-GDi', 'Titan', 'V6', 'VGT', 'White Edition', 'XE', 'XR', 'XS', 'Xi', 'h T-GDi']}}, {'KIA': {'Stinger': ['Blue Edition', 'CRDi', 'GT S', 'GT-Line', 'GT-Line S', 'T-GDi', 'T-GDi V6']}}, {'KIA': {'Stonic': ['2', '3', '4', 'CRDi', 'Connect', 'First Edition', 'GT-Line', 'GT-Line S', 'MHEV', 'Maxx', 'Mixx', 'T-GDi']}}, {'KIA': {'Venga': ['1', '1 Air', '2', '3', '4', 'CRDi', 'EcoDynamics', 'SR7']}}, {'KIA': {'XCeed': ['2', '3', '4', 'CRDi', 'Connect', 'Edition', 'First Edition', 'GDi', 'GT Line', 'GT Line S', 'MHEV', 'T-GDi']}}, {'Koenigsegg': {'Regera': ''}}, {'LEVC': {'Tx': ['Vista', 'Vista Comfort Plus']}}, {'Lada': {'Niva': ''}}, {'Lamborghini': {'Aventador': ['LP 700-4', 'LP 700-4 Pirelli Edition', 'LP 740-4 S', 'LP 750-4 Superveloce', 'LP 770-4 SVJ', 'LP 780-4 Ultimae', 'V12']}}, {'Lamborghini': {'Countach': 'Anniversary'}}, {'Lamborghini': {'Diablo': 'SV'}}, {'Lamborghini': {'Gallardo': ['LP 560-4', 'SE', 'Superleggera', 'V10']}}, {'Lamborghini': {'Huracan': ['LP 580-2', 'LP 610-2 EVO', 'LP 610-4', 'LP 610-4 Avio', 'LP 640-2 STO', 'LP 640-4 EVO', 'LP 640-4 Performante', 'V10']}}, {'Lamborghini': {'Jalpa': ''}}, {'Lamborghini': {'Murcielago': ['LP 670-4 Superveloce', 'V12', 'VT']}}, {'Lamborghini': {'Urus': ['Graphite Capsule', 'Pearl Capsule', 'S', 'V8 BiTurbo']}}, {'Lancia': {'Delta': '2000 HF'}}, {'Lancia': {'Fulvia': ''}}, {'Lancia': {'Y10': 'Fire'}}, {'Land Rover': {'110': ['County', 'TD']}}, {'Land Rover': {'90': ['County', 'D', 'TD']}}, {'Land Rover': {'Defender 110': ['75th Limited Edition', 'Adventure', 'Black', 'Bond Edition', 'Carpathian Edition', 'County', 'D', 'D200', 'D250', 'D300', 'First Edition', 'G4', 'HSE', 'Heritage', 'Landmark', 'MHEV', 'P300', 'P400', 'P400e', 'P525 V8', 'S', 'SD4', 'SE', 'Si4', 'Silver', 'TD', 'TD5', 'TDCi', 'TDi', 'X', 'X-Dynamic HSE', 'X-Dynamic S', 'X-Dynamic SE', 'XS', 'XS Edition', 'XTech Limited Edition', 'i6']}}, {'Land Rover': {'Defender 130': ['County', 'D', 'D300', 'DPF', 'First Edition', 'HSE', 'MHEV', 'P300', 'P400', 'TD5', 'TDi', 'X', 'X-Dynamic HSE', 'X-Dynamic SE']}}, {'Land Rover': {'Defender 90': ['75th Limited Edition', '90 Heritage', 'Adventure', 'Autobiography', 'Black', 'Carpathian Edition', 'County', 'D', 'D200', 'D250', 'D300', 'First Edition', 'G4', 'HSE', 'Heritage', 'LVX 65th Anniversary', 'Landmark', 'MHEV', 'P300', 'P400', 'P525 V8', 'S', 'SE', 'SV', 'SVX', 'Silver', 'TD', 'TD5', 'TDCi', 'TDi', 'X', 'X-Dynamic HSE', 'X-Dynamic S', 'X-Dynamic SE', 'XS', 'XS Edition', 'XTech Limited Edition']}}, {'Land Rover': {'Defender Unspecified': ''}}, {'Land Rover': {'Discovery': ['Adventurer', 'Anniversary Edition', 'D250', 'D300', 'ES', 'ES Premium', 'First Edition', 'GS', 'HSE', 'HSE Luxury', 'Landmark', 'Landmark Edition', 'MHEV', 'Metropolitan Edition', 'P360', 'Pursuit', 'R-Dynamic HSE', 'R-Dynamic S', 'R-Dynamic SE', 'S', 'SD V6', 'SD4', 'SE', 'Serengeti', 'Si4', 'Si6 V6', 'TD V6', 'TD5', 'TDi', 'XS', 'i V8']}}, {'Land Rover': {'Discovery 3': ['GS', 'HSE', 'S', 'SE', 'TD', 'TD V6', 'V8', 'XS']}}, {'Land Rover': {'Discovery 4': ['GS', 'Graphite', 'HSE', 'HSE Luxury', 'Landmark', 'Landmark LE', 'SD', 'SD V6', 'SE', 'SE Tech', 'TD', 'TD V6', 'XS', 'XXV']}}, {'Land Rover': {'Discovery Sport': ['D150', 'D165', 'D180', 'D200', 'D240', 'HSE', 'HSE Black', 'HSE Dynamic Lux', 'HSE Luxury', 'Landmark', 'MHEV', 'P200', 'P250', 'P290', 'P300e', 'Pure', 'Pure Edition', 'R-Dynamic Black', 'R-Dynamic HSE', 'R-Dynamic S', 'R-Dynamic S Plus', 'R-Dynamic SE', 'S', 'SD4', 'SE', 'SE Tech', 'Si4', 'TD4', 'Urban Edition', 'eD4']}}, {'Land Rover': {'Freelander': ['Adventurer', 'E', 'ES', 'ES Premium', 'Freestyle', 'GS', 'HSE', 'Kalahari', 'Maasai Mara', 'S', 'SE', 'Sport', 'Sport Premium', 'TD4', 'XEdi', 'XEi']}}, {'Land Rover': {'Freelander 2': ['Black and White', 'Dynamic', 'GS', 'HSE', 'HSE Lux', 'HST', 'Metropolis', 'S', 'SD4', 'SE', 'SE Tech', 'Sport LE', 'TD4', 'TD4e', 'XS', 'eD4', 'i6']}}, {'Land Rover': {'Lightweight': ''}}, {'Land Rover': {'Modified': ''}}, {'Land Rover': {'Range Rover': ['35th Anniversary', 'Autobiography', 'Autobiography Black', 'Bordeaux', 'D300', 'D350', 'EFI V8', 'Fifty', 'First Edition', 'HSE', 'In Vogue', 'MHEV', 'P V8', 'P400', 'P400e', 'P440e', 'P510e', 'P525 V8', 'P530 V8', 'P565 V8', 'SD V6', 'SD V8', 'SE', 'SV', 'SV Autobiography', 'SV Autobiography Dynamic', 'SV Autobiography Dynamic Black', 'TD', 'TD V6', 'TD V8', 'TDi', 'Td6', 'V6', 'V8', 'V8 Supercharged', 'Vogue', 'Vogue SE', 'Westminster', 'Westminster Black', 'h SDV6']}}, {'Land Rover': {'Range Rover Evoque': ['Autobiography', 'D150', 'D165', 'D180', 'D200', 'D204', 'D240', 'Dynamic', 'Dynamic Lux', 'Edition', 'Ember Special Edition', 'First Edition', 'HSE', 'HSE Dynamic', 'HSE Dynamic Lux', 'HST', 'Landmark', 'MHEV', 'P200', 'P250', 'P300', 'P300e', 'Prestige', 'Prestige Lux', 'Pure', 'Pure Tech', 'R-Dynamic', 'R-Dynamic HSE', 'R-Dynamic S', 'R-Dynamic SE', 'S', 'SD4', 'SE', 'SE Dynamic', 'SE Tech', 'Si4', 'Special Edition', 'TD4', 'eD4']}}, {'Land Rover': {'Range Rover Sport': ['Autobiography', 'Autobiography Dynamic', 'Autobiography Sport', 'D250', 'D300', 'D350', 'Dynamic SE', 'First Edition', 'HSE', 'HSE Black', 'HSE Dynamic', 'HSE Dynamic Black', 'HSE Luxury', 'HSE Red', 'HSE Silver', 'HST', 'MHEV', 'P300', 'P400', 'P400e', 'P440e', 'P510e', 'P525 V8', 'P575 V8', 'S', 'SD V6', 'SD V8', 'SD4', 'SE', 'SVR', 'SVR Carbon Edition', 'Si4', 'Stormer', 'Stormer SE', 'Supercharged', 'TD V6', 'TD V8', 'V6', 'V8', 'V8 Supercharged', 'h SDV6', 'i6']}}, {'Land Rover': {'Range Rover Velar': ['D180', 'D200', 'D240', 'D275', 'D300', 'Edition', 'First Edition', 'HSE', 'HST', 'MHEV', 'P250', 'P300', 'P400', 'P400e', 'P550', 'R-Dynamic', 'R-Dynamic HSE', 'R-Dynamic S', 'R-Dynamic SE', 'S', 'SD6 V6', 'SE', 'SVAutobiography Dynamic Edition', 'Si6 V6']}}, {'Land Rover': {'Series I': ''}}, {'Land Rover': {'Series Ii': ''}}, {'Land Rover': {'Series Iii': ['88', 'County', 'D']}}, {'Lexus': {'Ct 200h': ['Advance', 'Advance Plus', 'CT', 'Executive Edition', 'F Sport', 'Luxury', 'Premier', 'S', 'SE', 'SE-I', 'SE-L', 'SE-L Premier', 'Sport', 'Takumi']}}, {'Lexus': {'Es 300h': ['ES', 'ES Premium Edition', 'F Sport', 'Takumi']}}, {'Lexus': {'Gs 250': ['F Sport', 'Luxury', 'V6']}}, {'Lexus': {'Gs 300': ['Executive Edition', 'F Sport', 'LE', 'Luxury', 'Premier', 'S', 'SE', 'SE-L']}}, {'Lexus': {'Gs 430': 'SE'}}, {'Lexus': {'Gs 450h': ['F Sport', 'Luxury', 'Premier', 'SE', 'SE-L', 'Sport', 'V6', 'h V6']}}, {'Lexus': {'Gs F': 'V8'}}, {'Lexus': {'Is 200': ['Advance', 'F Sport', 'Premier', 'S', 'SE', 'SE-I', 'Sport', 'SportCross', 'T', 'd']}}, {'Lexus': {'Is 220d': ['SE', 'SE-I', 'SE-L', 'TD']}}, {'Lexus': {'Is 250': ['Advance', 'F Sport', 'Limited Edition', 'Luxury', 'Premier', 'SE', 'SE-I', 'SE-L', 'SR', 'Sport', 'V6']}}, {'Lexus': {'Is 300': ['Advance', 'Executive Edition', 'F Sport', 'F Sport Limited Edition', 'IS', 'Luxury', 'Premier', 'SE', 'Sport', 'SportCross', 'Takumi']}}, {'Lexus': {'Is F': 'V8'}}, {'Lexus': {'Is Unspecified': ''}}, {'Lexus': {'Lc 500': ['Black Inspiration', 'Launch Edition', 'Limited Edition', 'Sport', 'Sport Plus', 'V6', 'V8']}}, {'Lexus': {'Lfa': ''}}, {'Lexus': {'Ls 400': ''}}, {'Lexus': {'Ls 430': ''}}, {'Lexus': {'Ls 460': ['F Sport', 'Luxury', 'SE', 'SE-L', 'V8']}}, {'Lexus': {'Ls 500h': ['Luxury', 'Premier', 'Takumi', 'V6']}}, {'Lexus': {'Ls 600h': ['L', 'L RSR', 'Premier', 'V8']}}, {'Lexus': {'Ls Unspecified': ''}}, {'Lexus': {'Lx': ''}}, {'Lexus': {'Nx 300h': ['F Sport', 'Luxury', 'NX', 'Premier', 'S', 'SE', 'Sport', 'Takumi']}}, {'Lexus': {'Nx 350h': ['F Sport', 'NX', 'Takumi']}}, {'Lexus': {'Nx 450h+': ['F Sport', 'NX', 'Takumi']}}, {'Lexus': {'Rc 200t': ['F Sport', 'T']}}, {'Lexus': {'Rc 300h': ['F Sport', 'F Sport Black Edition', 'Luxury', 'Premier', 'RC']}}, {'Lexus': {'Rc F': ['Carbon', 'Track Edition', 'V8']}}, {'Lexus': {'Rx 200t': ['Luxury', 'T']}}, {'Lexus': {'Rx 300': ['Limited Edition', 'SE']}}, {'Lexus': {'Rx 350': ['LE', 'SE']}}, {'Lexus': {'Rx 400h': ['Executive', 'Limited Edition', 'SE', 'SE-L', 'SR']}}, {'Lexus': {'Rx 450h': ['Advance', 'F Sport', 'Luxury', 'Premier', 'Premier Pan Roof', 'Premium Sport Edition', 'RX', 'SE', 'SE-I', 'SE-L', 'SE-L Premier', 'Sport', 'Takumi', 'V6']}}, {'Lexus': {'Rx L 450h': ['Luxury', 'Premier', 'Premium Sport Edition', 'RX L', 'Takumi', 'V6']}}, {'Lexus': {'Rx Unspecified': ''}}, {'Lexus': {'Sc 430': ''}}, {'Lexus': {'Ux 250h': ['F Sport', 'F Sport Design', 'Premium Sport Edition', 'Takumi', 'UX']}}, {'Lexus': {'Ux 300e': 'UX'}}, {'Leyland': {'Mini': ''}}, {'Lincoln': {'Mkx': ''}}, {'Lincoln': {'Navigator': ''}}, {'Lincoln': {'Town Car': ''}}, {'London Taxis International': {'Tx4': ['Bronze', 'Elegance', 'Gold', 'Silver', 'TD']}}, {'London Taxis International': {'Txi': ''}}, {'London Taxis International': {'Txii': ''}}, {'Lotus': {'340r': ''}}, {'Lotus': {'Eclat': ''}}, {'Lotus': {'Elan': 'SE'}}, {'Lotus': {'Elise': ['111R', '111S', '220 Cup', 'CR', 'Cup 250', 'Cup 250 Final Edition', 'S', 'S CR', 'S Touring', 'SC', 'Sport', 'Sport 220', 'Sport 240 Final Edition', 'Touring']}}, {'Lotus': {'Emira': ['First Edition', 'V6']}}, {'Lotus': {'Esprit': ['GT', 'HC', 'SE', 'Turbo', 'Turbo Commemorative Limited Edition', 'V8']}}, {'Lotus': {'Europa': 'S'}}, {'Lotus': {'Evora': ['400', 'GT410', 'GT410 Sport', 'S', 'Sports Racer', 'V6']}}, {'Lotus': {'Excel': 'SE'}}, {'Lotus': {'Exige': ['S', 'S Sports', 'S Sports Touring Performance', 'S Touring', 'Sport 350', 'Sport 380', 'Sport 390 Final Edition', 'Sport 410', 'Sport 420 Final Edition', 'Touring', 'V6']}}, {'Maxus': {'Deliver 9': ''}}, {'Maxus': {'T90ev': ''}}, {'MEV': {'City': ''}}, {'MG': {'Gs': ['Excite', 'Exclusive', 'Explore', 'TGI']}}, {'MG': {'Mg Hs': ['Excite', 'Exclusive', 'T-GDI']}}, {'MG': {'Mg Zs': ['110', '120', 'Excite', 'Exclusive', 'Explore', 'Limited Edition', 'SE', 'T-GDI', 'Trophy', 'Trophy Connect', 'VTi-TECH']}}, {'MG': {'Mg3': ['3Form', '3Form Sport', '3Style', '3Style+', 'Excite', 'Exclusive', 'Exclusive Nav', 'Explore', 'VTi-TECH']}}, {'MG': {'Mg4': ['SE', 'Trophy']}}, {'MG': {'Mg5': ['Excite', 'Exclusive', 'SE', 'Trophy']}}, {'MG': {'Mg6': ['DTi Tech', 'GT S', 'GT SE', 'GT TSE', 'S', 'T']}}, {'MG': {'Mga': ''}}, {'MG': {'Mgb': 'Roadster'}}, {'MG': {'Mgb Gt': 'V8'}}, {'MG': {'Mgc': ''}}, {'MG': {'Mgf': ['160', 'SE', 'Trophy SE', 'VVC', 'i']}}, {'MG': {'Midget': ''}}, {'MG': {'Rv8': ''}}, {'MG': {'Ta': ''}}, {'MG': {'Td': ''}}, {'MG': {'Tf': ['80th Anniversary LE', 'Cool Blue', 'LE 500', 'Spark SE', 'Sunstorm SE']}}, {'MG': {'Zr': ['+', '105', 'TD']}}, {'MG': {'Zt': ['+ Sports', '180', '260', 'Sports', 'V8']}}, {'MG': {'Zt-t': ['160', 'T']}}, {'MINI': {'Clubman': ['Cooper', 'Cooper Bond Street', 'Cooper Classic', 'Cooper D', 'Cooper D Bond Street', 'Cooper D Classic', 'Cooper D Exclusive', 'Cooper D Hampton', 'Cooper D Soho', 'Cooper D Sport', 'Cooper Exclusive', 'Cooper Hampton', 'Cooper S', 'Cooper S Classic', 'Cooper S Exclusive', 'Cooper S Hampton', 'Cooper S Shadow Edition', 'Cooper S Sport', 'Cooper SD', 'Cooper SD Hampton', 'Cooper Shadow Edition', 'Cooper Sport', 'Cooper Untold Edition', 'John Cooper Works', 'One', 'One City', 'One City D', 'One D']}}, {'MINI': {'Clubvan': ['Cooper', 'Cooper D', 'One']}}, {'MINI': {'Convertible': ['Cooper', 'Cooper Classic', 'Cooper D', 'Cooper D Highgate', 'Cooper Exclusive', 'Cooper Highgate', 'Cooper Resolute Edition', 'Cooper S', 'Cooper S 25th Anniversary', 'Cooper S Classic', 'Cooper S Exclusive', 'Cooper S Highgate', 'Cooper S Open 150 Edition', 'Cooper S Resolute Edition', 'Cooper S Sidewalk', 'Cooper S Sidewalk Edition', 'Cooper S Sport', 'Cooper S Works', 'Cooper SD', 'Cooper Sidewalk', 'Cooper Soho', 'Cooper Sport', 'John Cooper Works', 'One', 'One Sidewalk']}}, {'MINI': {'Countryman': ['Cooper', 'Cooper Boardwalk Edition', 'Cooper Classic', 'Cooper D', 'Cooper D Business Edition', 'Cooper D Classic', 'Cooper D Exclusive', 'Cooper D Park Lane', 'Cooper D Sport', 'Cooper Exclusive', 'Cooper Park Lane', 'Cooper S', 'Cooper S Boardwalk Edition', 'Cooper S Classic', 'Cooper S Exclusive', 'Cooper S Park Lane', 'Cooper S Shadow Edition', 'Cooper S Special Edition', 'Cooper S Sport', 'Cooper SD', 'Cooper SE', 'Cooper SE Classic', 'Cooper SE Exclusive', 'Cooper SE Shadow Edition', 'Cooper SE Sport', 'Cooper SE Untamed Edition', 'Cooper Shadow Edition', 'Cooper Sport', 'Cooper Untamed Edition', 'John Cooper Works', 'One', 'One D']}}, {'MINI': {'Coupe': ['Cooper', 'Cooper S', 'Cooper SD', 'John Cooper Works']}}, {'MINI': {'Electric': ['Collection Edition', 'Cooper S Shadow Edition', 'Level 1', 'Level 2', 'Level 3', 'Resolute Edition']}}, {'MINI': {'Hatch': ['Challenge 210', 'Cooper', 'Cooper Baker Street', 'Cooper Bayswater', 'Cooper Camden', 'Cooper Classic', 'Cooper D', 'Cooper D Baker Street', 'Cooper D Bayswater', 'Cooper D Camden', 'Cooper D Graphite', 'Cooper D London 2012', 'Cooper D Mayfair', 'Cooper D Pimlico', 'Cooper D Seven', 'Cooper Exclusive', 'Cooper Graphite', 'Cooper London 2012', 'Cooper Mayfair', 'Cooper Park Lane', 'Cooper Pimlico', 'Cooper Resolute Edition', 'Cooper S', 'Cooper S 60 Years Edition', 'Cooper S Bayswater', 'Cooper S Camden', 'Cooper S Checkmate', 'Cooper S Classic', 'Cooper S Exclusive', 'Cooper S Inspired by Goodwood', 'Cooper S Mayfair', 'Cooper S Paddy Hopkirk Edition', 'Cooper S Resolute Edition', 'Cooper S Seven', 'Cooper S Shadow Edition', 'Cooper S Sport', 'Cooper S Works', 'Cooper SD', 'Cooper SD Bayswater', 'Cooper SD London 2012', 'Cooper SD Seven', 'Cooper Seven', 'Cooper Shadow Edition', 'Cooper Soho', 'Cooper Sport', 'First', 'John Cooper Works', 'John Cooper Works Anniversary Edition', 'John Cooper Works Challenge', 'John Cooper Works GP', 'John Cooper Works World Championship', 'One', 'One 1499 GT', 'One Baker Street', 'One Classic', 'One D', 'One D Baker Street', 'One Graphite', 'One Minimalist', 'One Pimlico', 'One Seven', 'Sport']}}, {'MINI': {'Moke': ''}}, {'MINI': {'Paceman': ['Cooper', 'Cooper D', 'Cooper S', 'Cooper SD', 'John Cooper Works']}}, {'MINI': {'Roadster': ['Cooper', 'Cooper S', 'Cooper SD', 'John Cooper Works']}}, {'MINI': {'Unspecified': ''}}, {'MK': {'Indy': ''}}, {'Moke': {'56': ''}}, {'Maserati': {'3200': ['GT', 'V8']}}, {'Maserati': {'Biturbo': ''}}, {'Maserati': {'Coupe': ['Cambiocorsa', 'GT']}}, {'Maserati': {'Ghibli': ['D V6', 'GT', 'GranLusso', 'GranSport', 'MC Edition', 'MHEV', 'Modena', 'Ribelle', 'S', 'S GranSport Nerissimo', 'Sportivo', 'Trofeo', 'V6', 'V8']}}, {'Maserati': {'Grancabrio': ['MC', 'MC Nerissimo', 'Sport', 'Sport Nerissimo', 'V8']}}, {'Maserati': {'Gransport': ''}}, {'Maserati': {'Granturismo': ['MC', 'MC Nerissimo', 'MC Stradale', 'S', 'Sport', 'Sport Nerissimo', 'V8']}}, {'Maserati': {'Grecale': ['GT', 'GT PrimaSerie', 'MHEV', 'Modena', 'Trofeo', 'Trofeo PrimaSerie', 'V6']}}, {'Maserati': {'Levante': ['D V6', 'GT', 'GranLusso', 'GranSport', 'GranSport Nerissimo', 'MC Edition', 'MHEV', 'Modena', 'Modena S', 'S', 'S GranSport', 'Trofeo', 'V6', 'V8']}}, {'Maserati': {'Mc20': 'V6'}}, {'Maserati': {'Quattroporte': ['D V6', 'Executive GT', 'GT', 'GTS', 'GranLusso', 'GranSport', 'S', 'Sport GT', 'Sport GT S', 'V6', 'V8']}}, {'Maserati': {'Spyder': ['Cambiocorsa', 'GT']}}, {'Maybach': {'57': ''}}, {'Maybach': {'62': 'S'}}, {'Mazda': {'323': ['GSi', 'SE Exec']}}, {'Mazda': {'626': 'GSi SE'}}, {'Mazda': {'B2500': ['4-Action', 'TD']}}, {'Mazda': {'BT-50': ['Intrepid', 'TD', 'TS', 'TS2']}}, {'Mazda': {'Bongo': ''}}, {'Mazda': {'CX-3': ['GT Sport', 'GT Sport Nav+', 'SE', 'SE Nav', 'SE Nav+', 'SE-L', 'SE-L Nav', 'SE-L Nav+', 'SKYACTIV-D', 'SKYACTIV-G', 'Sport Black+', 'Sport Nav', 'Sport Nav+']}}, {'Mazda': {'CX-30': ['100th Anniversary Edition', 'GT Sport', 'GT Sport Edition', 'GT Sport Tech', 'GT Sport Tech Edition', 'MHEV', 'SE-L', 'SE-L Lux', 'SKYACTIV-G', 'SKYACTIV-X', 'Sport Lux', 'e-SKYACTIV G', 'e-SKYACTIV X']}}, {'Mazda': {'CX-5': ['Centre-Line', 'Exclusive-Line', 'GT Sport', 'GT Sport Nav+', 'Homura', 'Kuro Edition', 'Newground', 'SE-L', 'SE-L Lux', 'SE-L Lux Nav', 'SE-L Nav', 'SE-L Nav+', 'SKYACTIV-D', 'SKYACTIV-G', 'Sport', 'Sport Black', 'Sport Black Edition', 'Sport Edition', 'Sport Nav', 'Sport Nav+', 'Takumi', 'e-SKYACTIV G MHEV']}}, {'Mazda': {'CX-60': ['Exclusive-Line', 'Homura', 'MHEV', 'Takumi', 'e-SKYACTIV', 'e-SKYACTIV-D']}}, {'Mazda': {'CX-7': ['Sport Tech', 'T', 'd']}}, {'Mazda': {'Demio': ['Atlantis', 'GSi']}}, {'Mazda': {'Eunos': ''}}, {'Mazda': {'MX-30': ['Exclusive-Line', 'First Edition', 'GT Sport Tech', 'Makoto', 'Prime-Line', 'SE-L Lux', 'Sport Lux']}}, {'Mazda': {'MX-5': ['20th Anniversary', '25th Anniversary', '30th Anniversary Edition', 'Angels', 'Arctic', 'Arizona', 'Berkeley', 'California', 'Euphonic', 'Exclusive-Line', 'GT Sport Nav+', 'GT Sport Tech', 'Homura', 'Icon', 'Isola', 'Jasper Conran', 'Kendo', 'Kuro', 'Limited Edition', 'Miyako', 'Montana', 'Nevada', 'Niseko', 'Phoenix SE', 'Prime-Line', 'R-Sport', 'S', 'SE', 'SE+', 'SE-L', 'SE-L Nav', 'SE-L Nav+', 'SKYACTIV-G', 'Sport', 'Sport Black', 'Sport Graphite', 'Sport Nav', 'Sport Nav+', 'Sport Recaro', 'Sport Tech', 'Sport Tech Nav', 'Sport Venture', 'Venture', 'Z-Sport', 'ZSport', 'i']}}, {'Mazda': {'MX-5 RF': ['30th Anniversary Edition', 'Exclusive-Line', 'GT Sport Nav+', 'GT Sport Tech', 'Homura', 'Launch Edition', 'Prime-Line', 'SE-L', 'SE-L Nav', 'SE-L Nav+', 'SKYACTIV-G', 'Sport', 'Sport Black', 'Sport Nav', 'Sport Nav+', 'Sport Tech']}}, {'Mazda': {'Mazda2': ['Agile', 'Antares', 'Black', 'Black Edition', 'Black+ Edition', 'Capella', 'Colour Edition', 'D', 'GT', 'GT Sport', 'GT Sport Nav', 'GT Sport Nav+', 'GT Sport Tech', 'MHEV', 'Pure', 'Red Edition', 'S', 'SE', 'SE+', 'SE-L', 'SE-L Nav', 'SE-L Nav+', 'SE-L+', 'SKYACTIV-D', 'SKYACTIV-G', 'Sakata', 'Select', 'Sport', 'Sport Black', 'Sport Black+', 'Sport Colour Edition', 'Sport Nav', 'Sport Nav+', 'Sport Venture', 'Sports Launch Edition', 'TD', 'TS', 'TS2', 'Takuya', 'Tamura', 'Tamura Nav', 'Tech Edition', 'Venture', 'White Edition', 'e-SKYACTIV-G']}}, {'Mazda': {'Mazda3': ['D', 'GT Sport', 'GT Sport Edition', 'GT Sport Tech', 'GT Sport Tech Edition', 'Katano', 'Katano 11', 'MHEV', 'MPS', 'SE', 'SE Nav', 'SE-L', 'SE-L Lux', 'SE-L Nav', 'SKYACTIV-D', 'SKYACTIV-G', 'SKYACTIV-X', 'Sport', 'Sport Black', 'Sport Lux', 'Sport Nav', 'T', 'TS', 'TS2', 'Takara', 'Takuya', 'Tamura', 'Venture', 'd', 'e-SKYACTIV G', 'e-SKYACTIV X', 'e-SKYACTIV-G', 'e-SKYACTIV-X']}}, {'Mazda': {'Mazda5': ['D', 'Furano', 'Sport', 'Sport Nav', 'Sport Venture', 'TS', 'TS2', 'Takara', 'Venture', 'd']}}, {'Mazda': {'Mazda6': ['Business Line', 'GT Sport', 'GT Sport Nav+', 'Katano', 'Kuro Edition', 'MPS', 'S', 'SE', 'SE Nav', 'SE-L', 'SE-L Lux Nav+', 'SE-L Nav', 'SE-L Nav+', 'SKYACTIV-D', 'SKYACTIV-G', 'SL', 'Sakata', 'Sport', 'Sport Nav', 'Sport Nav+', 'TD', 'TS', 'TS2', 'Takuya', 'Tamura', 'd', 'i']}}, {'Mazda': {'Premacy': 'Sport'}}, {'Mazda': {'Rx-7': 'Turbo'}}, {'Mazda': {'RX-8': ['40th Anniversary Edition', 'Nemesis', 'PZ', 'R3']}}, {'Mazda': {'Tribute': 'V6'}}, {'McLaren': {'12c': 'T V8'}}, {'McLaren': {'540c': 'T V8'}}, {'McLaren': {'570gt': 'T V8'}}, {'McLaren': {'570s': 'T V8'}}, {'McLaren': {'600lt': 'T V8'}}, {'McLaren': {'620r': ''}}, {'McLaren': {'650s': 'T V8'}}, {'McLaren': {'675lt': 'T V8'}}, {'McLaren': {'720s': ['Luxury', 'Performance', 'T V8']}}, {'McLaren': {'765lt': 'T V8'}}, {'McLaren': {'Gt': 'T V8'}}, {'McLaren': {'P1': 'T V8'}}, {'McLaren': {'Senna': ''}}, {'Mercedes-Benz': {'190': ['D', 'E', 'E 16', 'LE']}}, {'Mercedes-Benz': {'200': ['E', 'TE']}}, {'Mercedes-Benz': {'220': ''}}, {'Mercedes-Benz': {'230': ['CE', 'E', 'TE']}}, {'Mercedes-Benz': {'230 Sl': ''}}, {'Mercedes-Benz': {'250': ''}}, {'Mercedes-Benz': {'250 Sl': ''}}, {'Mercedes-Benz': {'260': 'E'}}, {'Mercedes-Benz': {'280': ['CE', 'E', 'SE', 'SL', 'TE']}}, {'Mercedes-Benz': {'280 Sl': ''}}, {'Mercedes-Benz': {'300': ['CE', 'CE-24', 'E', 'SE', 'SEL', 'SL', 'TD', 'TE']}}, {'Mercedes-Benz': {'310': ''}}, {'Mercedes-Benz': {'320': ['CE', 'TE']}}, {'Mercedes-Benz': {'350': ''}}, {'Mercedes-Benz': {'380': 'SL'}}, {'Mercedes-Benz': {'420': 'SE'}}, {'Mercedes-Benz': {'450': ''}}, {'Mercedes-Benz': {'500': ['SEC', 'SEL', 'SL']}}, {'Mercedes-Benz': {'560': 'SEL'}}, {'Mercedes-Benz': {'600': 'SL'}}, {'Mercedes-Benz': {'A Class': ['A140', 'A150', 'A160', 'A160 CDI', 'A170', 'A180', 'A180 CDI', 'A180d', 'A190', 'A200', 'A200 CDI', 'A200d', 'A220', 'A220 CDI', 'A220d', 'A250', 'A250e', 'A35', 'A45', 'AMG', 'AMG Edition', 'AMG Line', 'AMG Line Edition', 'AMG Line Night Edition', 'AMG Night Edition', 'AMG PETRONAS 2015 World Champion Edition', 'AMG S', 'AMG S Plus', 'AMG Sport', 'AMG WhiteArt', 'AMG Yellow Night Edition', 'Avantgarde', 'Avantgarde SE', 'BlueEfficiency', 'Classic', 'Classic SE', 'ECO', 'Elegance', 'Elegance SE', 'Engineered by AMG', 'Exclusive Edition', 'Exclusive Edition Plus', 'Motorsport Edition', 'Piccadilly', 'SE', 'Special Edition', 'Sport', 'Sport Edition', 'Sport Edition Plus', 'WhiteArt']}}, {'Mercedes-Benz': {'AMG': ['AMG', 'C36', 'C43', 'CL55', 'CLK55', 'E36', 'S55', 'SL60']}}, {'Mercedes-Benz': {'AMG GT': ['Black Series', 'C', 'C Edition 50', 'Edition 476 Plus', 'Night Edition', 'R', 'R PRO', 'S', 'S Edition 1', 'V8 BiTurbo']}}, {'Mercedes-Benz': {'Amg Gt 63': ['63 V8 BiTurbo', 'S', 'S Edition 1']}}, {'Mercedes-Benz': {'B Class': ['AMG Line', 'AMG Line Edition', 'B150', 'B160', 'B170', 'B180', 'B180 CDI', 'B180d', 'B200', 'B200 CDI', 'B200d', 'B220', 'B220 CDI', 'B220d', 'B250', 'B250e', 'BlueEfficiency', 'ECO', 'Electric Art', 'Exclusive Edition', 'SE', 'Sport']}}, {'Mercedes-Benz': {'C Class': ['AMG', 'AMG Black Series', 'AMG Edition', 'AMG Edition 125', 'AMG Edition 507', 'AMG Line', 'AMG Line Edition', 'AMG Line Night Edition', 'AMG Night Edition', 'AMG S', 'AMG S Edition 1', 'AMG S Night Edition', 'AMG Sport', 'AMG Sport Edition', 'AMG Sport Edition 125', 'AMG Sport Plus', 'Avantgarde', 'Avantgarde SE', 'BlueEfficiency', 'BlueTEC', 'C160', 'C180', 'C180K', 'C200', 'C200 CDI', 'C200K', 'C200d', 'C200h', 'C220', 'C220 CDI', 'C220d', 'C220dh', 'C230', 'C230 V6', 'C240', 'C250', 'C250 CDI', 'C250d', 'C270 CDI', 'C280', 'C280 V6', 'C300', 'C300d', 'C300de', 'C300dh', 'C300e', 'C300h', 'C32', 'C320', 'C320 CDI', 'C320 CDI V6', 'C350', 'C350 CDI V6', 'C350 V6', 'C350e', 'C43 V6', 'C55', 'C63 V8', 'C63 V8 BiTurbo', 'CDI', 'Classic', 'Classic SE', 'EQ Boost', 'Elegance', 'Elegance Edition 125', 'Elegance SE', 'Esprit', 'Evolution Panorama AMG', 'Evolution Panorama SE', 'Executive SE', 'Kompressor', 'MHEV', 'MHEV EQ Boost', 'Nightfall Edition', 'S Final Edition', 'SE', 'SE Edition 125', 'SE Executive', 'SE Executive Edition', 'Sport', 'Sport Edition', 'Sport Edition 125']}}, {'Mercedes-Benz': {'Cl': ['AMG', 'BlueEfficiency', 'CL420', 'CL500', 'CL500 V8', 'CL55', 'CL600', 'CL63', 'CL63 V8 BiTurbo']}}, {'Mercedes-Benz': {'CLA Class': ['AMG', 'AMG Line', 'AMG Line Edition', 'AMG Line Night Edition', 'AMG Night Edition', 'AMG S', 'AMG S Plus', 'AMG Sport', 'C220d', 'CLA180', 'CLA180d', 'CLA200', 'CLA200 CDI', 'CLA200d', 'CLA220', 'CLA220 CDI', 'CLA220d', 'CLA250', 'CLA250e', 'CLA35', 'CLA45', 'Engineered by AMG', 'OrangeArt', 'Sport', 'WhiteArt']}}, {'Mercedes-Benz': {'CLC Class': ['BlueEfficiency', 'CLC160', 'CLC180K', 'CLC200 CDI', 'CLC200K', 'CLC220 CDI', 'CLC230 V6', 'SE', 'Sport']}}, {'Mercedes-Benz': {'CLK': ['AMG', 'Avantgarde', 'CLK200', 'CLK220 CDI', 'CLK230', 'CLK240', 'CLK270 CDI', 'CLK280', 'CLK320', 'CLK320 CDI', 'CLK350', 'CLK430', 'CLK500', 'CLK55', 'CLK63', 'Elegance', 'Sport']}}, {'Mercedes-Benz': {'CLS': ['AMG', 'AMG Edition 1', 'AMG Line', 'AMG Line Night Edition', 'AMG S', 'AMG Sport', 'BlueEfficiency', 'BlueTEC', 'CLS220', 'CLS220d', 'CLS250 CDI', 'CLS300d', 'CLS300dh', 'CLS320 CDI', 'CLS350', 'CLS350 CDI', 'CLS350 CDI V6', 'CLS350 V6', 'CLS350d', 'CLS350d V6', 'CLS400 V6', 'CLS400d', 'CLS450', 'CLS500', 'CLS500 V8', 'CLS53', 'CLS55', 'CLS63', 'CLS63 V8', 'CLS63 V8 BiTurbo', 'EQ Boost', 'Grand Edition', 'MHEV', 'MHEV EQ Boost', 'Sport']}}, {'Mercedes-Benz': {'Citan': ['108 CDI', '109 CDI', '111 CDI', 'BlueEfficiency']}}, {'Mercedes-Benz': {'E Class': ['AMG', 'AMG Line', 'AMG Line Edition', 'AMG Line Night Edition', 'AMG Night Edition', 'AMG S', 'AMG S Edition 1', 'AMG S Night Edition', 'AMG Sport', 'AMG Sport Plus', 'All-Terrain Edition', 'Avantgarde', 'Avantgarde Edition 125', 'BlueEfficiency', 'BlueTEC', 'Classic', 'E200', 'E200 CDI', 'E200 CGI', 'E200d', 'E200h', 'E220', 'E220 CDI', 'E220d', 'E220dh', 'E230', 'E240', 'E250', 'E250 CDI', 'E250 CGI', 'E270 CDI', 'E280', 'E280 CDI', 'E300', 'E300d', 'E300de', 'E300dh', 'E300e', 'E300h', 'E320', 'E320 CDI', 'E350', 'E350 CDI V6', 'E350 CGI', 'E350 CGI V6', 'E350 V6', 'E350d', 'E350d V6', 'E350e', 'E400', 'E400 V6', 'E400d', 'E43 V6', 'E430', 'E450 V6', 'E450h', 'E500', 'E500 V8', 'E53', 'E53 BiTurbo', 'E53h BiTurbo', 'E55', 'E63', 'E63 V8', 'E63 V8 BiTurbo', 'Elegance', 'Executive', 'Executive SE', 'MHEV', 'MHEV EQ Boost', 'SE', 'SE Edition 125', 'Sport', 'Sport Edition 125']}}, {'Mercedes-Benz': {'EQA': ['AMG Line', 'EQA 250', 'EQA 250+', 'EQA 300', 'EQA 350', 'Sport']}}, {'Mercedes-Benz': {'Eqb': ['AMG Line', 'EQB 300', 'EQB 350']}}, {'Mercedes-Benz': {'EQC': ['AMG Line', 'EQC 400', 'Edition 1', 'Edition 1886', 'Sport']}}, {'Mercedes-Benz': {'Eqe': ['AMG Line', 'AMG Touring', 'EQE 300', 'EQE 350+', 'EQE 53', 'Exclusive Luxury']}}, {'Mercedes-Benz': {'Eqs': ['AMG Line', 'Exclusive Luxury', 'Luxury']}}, {'Mercedes-Benz': {'EQV': 'Sport'}}, {'Mercedes-Benz': {'G Class': ['AMG', 'AMG Carbon Edition', 'AMG Colour Edition', 'AMG Edition 1', 'AMG Edition 463', 'AMG Line', 'AMG Magno Edition', 'BlueTEC', 'G350 CDI V6', 'G350 V6', 'G350d', 'G400d', 'G500', 'G55 V8', 'G63 V8 BiTurbo', 'GD', 'GDL', 'GE300', 'Night Edition']}}, {'Mercedes-Benz': {'Gl Class': ['AMG', 'AMG Sport', 'BlueEfficiency', 'BlueTEC', 'GL320 CDI V6', 'GL350 V6', 'GL350 CDI V6', 'GL420 CDI V8', 'GL450 CDI V8', 'GL500 V8', 'GL63 V8', 'GL63 V8 BiTurbo']}}, {'Mercedes-Benz': {'GLA Class': ['AMG', 'AMG Line', 'AMG Line Edition', 'AMG S', 'AMG Yellow Night Edition', 'Exclusive Edition', 'GLA180', 'GLA200', 'GLA200 CDI', 'GLA200d', 'GLA220 CDI', 'GLA220d', 'GLA250', 'GLA250e', 'GLA35', 'GLA45', 'SE', 'Sport', 'Urban Edition', 'WhiteArt']}}, {'Mercedes-Benz': {'GLB Class': ['AMG', 'AMG Line', 'AMG Line Night Edition', 'GLB200', 'GLB200d', 'GLB220d', 'GLB35', 'Sport']}}, {'Mercedes-Benz': {'GLC Class': ['AMG', 'AMG Line', 'AMG Night Edition', 'AMG S', 'AMG S Edition 1', 'AMG S Night Edition', 'GLC220d', 'GLC250', 'GLC250d', 'GLC300', 'GLC300d', 'GLC300de', 'GLC300e', 'GLC300h', 'GLC350d V6', 'GLC400d', 'GLC43 V6', 'GLC63 V8 BiTurbo', 'MHEV', 'SE', 'Sport', 'Urban Edition']}}, {'Mercedes-Benz': {'GLE Class': ['AMG', 'AMG Line', 'AMG Night Edition', 'AMG S', 'AMG S Night Edition', 'AMG designo Line', 'GLE250d', 'GLE300d', 'GLE350d', 'GLE350d V6', 'GLE350de', 'GLE400d', 'GLE43 V6', 'GLE450 V6', 'GLE450h', 'GLE500e V6', 'GLE53', 'GLE53h BiTurbo', 'GLE63 V8', 'GLE63h V8 BiTurbo', 'MHEV', 'Sport', 'designo Line']}}, {'Mercedes-Benz': {'GLS Class': ['AMG', 'AMG Line', 'AMG Line Night Edition', 'AMG Night Edition', 'GLS350d V6', 'GLS400d', 'GLS63 V8', 'GLS63h V8 BiTurbo', 'Grand Edition', 'MHEV', 'Night Edition', 'designo Line']}}, {'Mercedes-Benz': {'M Class': ['AMG', 'AMG Line', 'AMG Sport', 'BlueEfficiency', 'BlueTEC', 'Edition 10', 'Grand Edition', 'Inspiration Edition', 'ML250', 'ML270 CDI', 'ML280 CDI', 'ML300 CDI V6', 'ML320', 'ML320 CDI', 'ML320 CDI V6', 'ML350', 'ML350 CDI V6', 'ML350 V6', 'ML420 CDI', 'ML420 CDI V8', 'ML430', 'ML500', 'ML500 V8', 'ML63', 'ML63 V8', 'S', 'SE', 'Special Edition', 'Sport']}}, {'Mercedes-Benz': {'Maybach Gls Class': ['First Class', 'GLS600h V8', 'MHEV']}}, {'Mercedes-Benz': {'Maybach S Class': ['MHEV', 'Maybach', 'S580h V8', 'S650 V12']}}, {'Mercedes-Benz': {'R Class': ['BlueEfficiency', 'Edition S', 'Grand Edition', 'R280 CDI', 'R300 CDI V6', 'R320 CDI', 'R350', 'R350 CDI', 'R350L CDI', 'SE', 'Sport']}}, {'Mercedes-Benz': {'S Class': ['350Ld V6', 'AMG', 'AMG Final Edition', 'AMG Line', 'AMG S', 'AMG Sport Edition', 'BlueEfficiency', 'BlueTEC', 'EQ Boost MHEV', 'EQ Power', 'Grand Edition', 'MHEV', 'Night Edition', 'S280', 'S300Ldh', 'S320', 'S320 CDI', 'S320 CDI V6', 'S320L CDI V6', 'S350', 'S350 CDI V6', 'S350 V6', 'S350L CDI V6', 'S350L V6', 'S350L d', 'S350Ld V6', 'S350d', 'S350d V6', 'S400L d', 'S400Lh V6', 'S420', 'S430', 'S450L', 'S500', 'S500 V8', 'S500L', 'S500L V8', 'S500Le V6', 'S500Lh', 'S55', 'S560 V8 BiTurbo', 'S560Le V6', 'S580Le V6', 'S600', 'S600L BiTurbo', 'S63 V8', 'S63 V8 BiTurbo', 'S63L V8', 'S63L V8 BiTurbo', 'S65 V12', 'S65L BiTurbo', 'S65L V12', 'SE', 'SE Line']}}, {'Mercedes-Benz': {'Se Class': ''}}, {'Mercedes-Benz': {'Sec Series': ''}}, {'Mercedes-Benz': {'SL Class': ['2LOOK Edition', 'AMG', 'AMG Line', 'AMG Sport', 'AMG Touring Plus', 'Alanite', 'BlueEfficiency', 'Edition', 'Grand Edition', 'Limited Edition', 'MHEV', 'Night Edition', 'SL280', 'SL300', 'SL320', 'SL350', 'SL350 V6', 'SL400', 'SL400 V6', 'SL43', 'SL500', 'SL500 V8', 'SL55', 'SL55 V8 BiTurbo', 'SL600', 'SL63', 'SL63 V8', 'SL65', 'Sport Edition']}}, {'Mercedes-Benz': {'SLC': ['AMG', 'AMG Final Edition', 'AMG Line', 'Final Edition', 'SLC180', 'SLC200', 'SLC250d', 'SLC300', 'SLC43 V6', 'Sport']}}, {'Mercedes-Benz': {'Slk': ['2LOOK', 'AMG', 'AMG Sport', 'AMG Sport Edition 125', 'BlueEfficiency', 'Edition 125', 'Grand Edition', 'Kompressor', 'SLK200', 'SLK200K', 'SLK230', 'SLK250', 'SLK250 CDI', 'SLK250d', 'SLK280', 'SLK280 V6', 'SLK300 V6', 'SLK32', 'SLK320', 'SLK350', 'SLK350 V6', 'SLK55', 'SLK55 V8', 'Sport']}}, {'Mercedes-Benz': {'Slr Mclaren': 'V8'}}, {'Mercedes-Benz': {'Sls': ['63 V8', 'AMG', 'AMG GT']}}, {'Mercedes-Benz': {'Sprinter': ['314 CDI', '413 CDI', '416 CDI', '517 CDI', '519 CDI V6', 'BlueEfficiency', 'CDI', 'Premium', 'Progressive', 'TL7']}}, {'Mercedes-Benz': {'V Class': ['AMG Line', 'BlueTEC', 'SE', 'Sport', 'V220d', 'V250d', 'V300d']}}, {'Mercedes-Benz': {'Vaneo': ['Ambiente', 'CDI']}}, {'Mercedes-Benz': {'Viano': ['Ambiente', 'Avantgarde', 'CDI', 'Trend']}}, {'Mercedes-Benz': {'Vito': ['109 CDI', '109CDI', '111', '111 CDI', '111CDI', '113 CDi', '114', '114 CDI', '115CDI', '116', '116 CDI', '116 CDi', '119', '119 CDI', '122 CDi', 'BlueEFFICIENCY', 'BlueTEC', 'CDI', 'Dualiner', 'Dualiner Sport', 'Dualiner Sport X', 'PRO', 'Premium', 'Progressive', 'SELECT', 'Shuttle', 'Sport', 'Traveliner']}}, {'Mercedes-Benz': {'X Class': ['CDI', 'CDI V6', 'Power', 'Progressive', 'Pure']}}, {'Mercedes-Benz': {'Evito': ['PREMIUM', 'PRO', 'SELECT']}}, {'Mercury': {'Cougar': ''}}, {'Microcar': {'M.go': ''}}, {'Mitsubishi': {'3000gt': ''}}, {'Mitsubishi': {'Asx': ['2', '3', '3 4 Work', '3 SE', '4', '4 Black', '5', 'Attivo', 'Black', 'ClearTec', 'D', 'DI-D', 'Dynamic', 'Exceed', 'Juro', 'MIVEC', 'ZC-H', 'ZC-M']}}, {'Mitsubishi': {'Carisma': ['GDi', 'Mirage']}}, {'Mitsubishi': {'Colt': ['Black Hawk', 'CZ1', 'CZ2', 'CZ3', 'CZT', 'DI-D', 'Elegance', 'Equippe', 'GTi', 'Juro', 'Ralliart', 'Sport']}}, {'Mitsubishi': {'Colt Cabriolet': ['CZC', 'CZC2', 'CZC3']}}, {'Mitsubishi': {'Colt Space Star': 'Classic'}}, {'Mitsubishi': {'Delica': ''}}, {'Mitsubishi': {'Eclipse': ''}}, {'Mitsubishi': {'Eclipse Cross': ['2', '3', '4', 'Black', 'Design SE', 'Dynamic', 'Exceed', 'First Edition', 'T', 'Verve']}}, {'Mitsubishi': {'Fto': ''}}, {'Mitsubishi': {'Gto': ''}}, {'Mitsubishi': {'Galant': ''}}, {'Mitsubishi': {'Grandis': ['Classic', 'Elegance', 'Equippe', 'Mirage', 'Warrior']}}, {'Mitsubishi': {'L200': ['4Life', '4Work', 'Animal', 'Barbarian', 'Barbarian Black', 'CC 4Life', 'Challenger', 'DC 4Life', 'DC Barbarian', 'DC Barbarian +', 'DC Barbarian Black', 'DC Barbarian SVP', 'DC Barbarian SVP II', 'DC Barbarian X', 'DC Challenger', 'DC Titan', 'DC Trojan', 'DC Warrior', 'DI-D', 'Diamond', 'Elegance', 'GLS', 'Raging Bull', 'SC 4Life', 'TD', 'Trojan', 'Trojan Black', 'Walkinshaw Performance', 'Warrior']}}, {'Mitsubishi': {'Lancer': ['DI-D', 'EVO IX MR FQ-300', 'EVO IX MR FQ-320', 'EVO IX MR FQ-340', 'EVO IX MR FQ-360', 'EVO IX MR FQ-360 HKS', 'EVO VI', 'EVO VI GSR', 'EVO VII RSII', 'EVO VIII 260', 'EVO VIII 280', 'EVO VIII FQ-300', 'EVO VIII MR FQ-320', 'EVO VIII MR FQ-340', 'EVO X FQ-440 MR', 'EVO X GSR FQ-300', 'EVO X GSR FQ-330', 'EVO X GSR FQ-360', 'EVO X GSR FQ-400', 'Elegance', 'Equippe', 'GLX', 'GS Ralliart', 'GS2', 'GS3', 'GS4', 'GSR Ralliart', 'Juro', 'MPi', 'SE', 'T']}}, {'Mitsubishi': {'Mirage': ['1', '2', '3', '4', 'Design', 'Design Pro', 'First Edition', 'Juro', 'Verve']}}, {'Mitsubishi': {'Outlander': ['2', '3', '3h', '4', '4h', '4hs', '5h', '5hs', 'DI-D', 'DI-DC', 'Design', 'Diamond', 'Dynamic', 'Dynamic Safety', 'Elegance', 'Equippe', 'Exceed', 'Exceed Safety', 'GSE', 'GX1 4work', 'GX2', 'GX3', 'GX3h', 'GX3h 4Work', 'GX3h+', 'GX4', 'GX4h', 'GX4hs', 'GX4s', 'GX5', 'GX5h', 'GX5hs', 'Juro', 'Keiko', 'Kotu', 'MIVEC', 'Reflex Plus', 'Verve', 'Warrior', 'h', 'h TwinMotor']}}, {'Mitsubishi': {'Pajero': ''}}, {'Mitsubishi': {'Shogun': ['Barbarian', 'DI-D', 'DI-DC', 'Diamond', 'Elegance', 'Equippe', 'Field', 'GLS', 'SG2', 'SG2 4work', 'SG3', 'SG4', 'SG4 Black', 'SG5', 'TD', 'TD DI-D', 'Warrior', 'Warrior 4work']}}, {'Mitsubishi': {'Shogun Pinin': ['Elegance', 'GDi']}}, {'Mitsubishi': {'Shogun Sport': ['3', '4', 'Classic', 'D', 'Elegance', 'Equippe', 'TD', 'Trojan', 'Warrior']}}, {'Mitsubishi': {'Space Star': 'Equippe'}}, {'Mitsubishi': {'Space Wagon': ['GDi', 'GLS', 'GLX']}}, {'Mitsubishi': {'Starion': 'Ex'}}, {'Mitsubishi': {'I': ''}}, {'Mitsubishi': {'I-miev': ''}}, {'Morgan': {'3 Wheeler': ''}}, {'Morgan': {'4/4': 'Runabout'}}, {'Morgan': {'Aero': 'SuperSport'}}, {'Morgan': {'Plus Eight': ''}}, {'Morgan': {'Plus Four': 'i'}}, {'Morgan': {'Plus Six': 'i'}}, {'Morgan': {'Roadster': ''}}, {'Morris': {'1000': ''}}, {'Morris': {'Isis': ''}}, {'Morris': {'Mini': ''}}, {'Morris': {'Minor': ''}}, {'Nosmoke': {'Original': ''}}, {'Nissan': {'200 Sx': ''}}, {'Nissan': {'300 Zx': 'Turbo'}}, {'Nissan': {'350 Z': ['GT', 'Grand Turismo 4', 'V6']}}, {'Nissan': {'370 Z': ['Black Edition', 'GT', 'GT 50th Anniversary', 'GT Edition', 'Nismo', 'V6']}}, {'Nissan': {'Almera': ['Activ', 'Premium', 'Pulse', 'S', 'SE', 'SVE', 'SX']}}, {'Nissan': {'Almera Tino': ['S', 'SE', 'Twister']}}, {'Nissan': {'Ariya': ['Advance', 'Evolve', 'Performance']}}, {'Nissan': {'Cherry': 'GS'}}, {'Nissan': {'Cube': ['Kaizen', 'LDN']}}, {'Nissan': {'Cubic': ''}}, {'Nissan': {'Elgrand': ''}}, {'Nissan': {'Figaro': ''}}, {'Nissan': {'GT-R': ['Black Edition', 'Premium Black', 'Premium Edition', 'Prestige', 'Recaro', 'Track Edition', 'V6']}}, {'Nissan': {'Juke': ['Acenta', 'Acenta Premium', 'Acenta Sport', 'Bose Personal Edition', 'DIG-T', 'Enigma', 'Envy', 'Kiiro', 'Kuro', 'Ministry of Sound', 'N-Connecta', 'N-Connecta Style', 'N-Vision', 'Nismo', 'Nismo RS', 'Premiere Edition', 'Shiro', 'Tekna', 'Tekna Pulse', 'Tekna+', 'Visia', 'dCi', 'n-tec']}}, {'Nissan': {'Kubistar': 'SE'}}, {'Nissan': {'Leaf': ['10', '2.Zero', '3.Zero', 'Acenta', 'Acenta+', 'Black Edition', 'N-Connecta', 'Solar Panel Spoiler', 'Tekna', 'Visia', 'Visia+', 'e+ 3.Zero', 'e+ N-Connecta', 'e+ Tekna', 'e+ n-tec']}}, {'Nissan': {'Maxima': 'S'}}, {'Nissan': {'Micra': ['160SR', '25th Anniversary', '30', 'Acenta', 'Acenta Limited Edition', 'Acenta+', 'Activ', 'Bose Personal Edition', 'Celebration', 'Chic', 'DIG-S', 'DIG-T', 'E', 'Elle', 'Equation', 'GX', 'IG', 'IG-T', 'Initia', 'Kiiro', 'Kuro', 'LS', 'N-Connecta', 'N-Sport', 'S', 'SE', 'SVE', 'SX', 'Shiro', 'Spirita', 'Sport', 'Sport+', 'Tekna', 'Tempest', 'Twister', 'URBIS', 'Vibe', 'Visia', 'Visia +', 'Visia+', 'XS', 'dCi', 'n-tec']}}, {'Nissan': {'Micra C+c': ['Acenta', 'Essenza', 'Pink', 'Sport', 'URBIS']}}, {'Nissan': {'Murano': 'V6'}}, {'Nissan': {'Nv200': ['Acenta', 'dCi']}}, {'Nissan': {'Nv250': ''}}, {'Nissan': {'Nv300': ['Acenta', 'Tekna', 'dCi 1.0t']}}, {'Nissan': {'Nv400': ''}}, {'Nissan': {'Navara': ['Acenta', 'Acenta+', 'Aventura', 'Di', 'Long Way Down', 'Long Way Down Expedition', 'Mammoth Sports Adventure', 'N-Connecta', 'N-Guard', 'N-Guard AT32', 'Off-Roader AT32', 'Outlaw', 'Platinum', 'SE', 'Salomon', 'Sport', 'Tekna', 'Trek -1', 'Visia', 'dCi', 'dCi V6']}}, {'Nissan': {'Note': ['Acenta', 'Acenta Premium', 'Acenta R', 'Acenta S', 'Black Edition', 'DIG-S', 'S', 'SE', 'SVE', 'Tekna', 'Visia', 'Visia Limited Edition', 'dCi', 'n-tec', 'n-tec+']}}, {'Nissan': {'Pathfinder': ['Acenta', 'Aventura', 'California Sports Adventure', 'Mammoth Sports Adventure', 'Platinum', 'SVE', 'Sport', 'TD', 'Tekna', 'dCi']}}, {'Nissan': {'Patrol': ['Di', 'S', 'SVE']}}, {'Nissan': {'Patrol Gr': ['Di', 'S', 'SE', 'SE+', 'SVE', 'TD']}}, {'Nissan': {'Pickup': 'Di'}}, {'Nissan': {'Pixo': ['Acenta', 'Tekna', 'Visia', 'n-tec']}}, {'Nissan': {'Primastar': ['Acenta', 'SE', 'Tekna', 'dCi 28', 'dCi 2900', 'dCi 30']}}, {'Nissan': {'Primera': ['Flare', 'GT', 'Precision', 'S', 'SE', 'SVE', 'SX', 'T-Spec']}}, {'Nissan': {'Pulsar': ['Acenta', 'DIG-T', 'N-Connecta', 'N-Connecta Style', 'Tekna', 'Visia', 'dCi', 'n-tec']}}, {'Nissan': {'Qashqai': ['360', 'Acenta', 'Acenta Premium', 'Acenta+', 'Black Edition', 'DIG-T', 'MHEV', 'N-Connecta', 'N-Motion', 'N-Vision', 'Pilot One Edition', 'Premier Limited Edition', 'Premiere Edition', 'Sound & Style', 'Tekna', 'Tekna+', 'Visia', 'dCi', 'h e-POWER', 'n-tec', 'n-tec+']}}, {'Nissan': {'Qashqai+2': ['360', 'Acenta', 'Tekna', 'Visia', 'dCi', 'n-tec', 'n-tec+']}}, {'Nissan': {'Serena': ['Excursion', 'GX', 'SLX']}}, {'Nissan': {'Skyline': 'GT-R'}}, {'Nissan': {'Sunny': ['GS', 'GSX']}}, {'Nissan': {'Terrano': ['Di', 'SE', 'SVE', 'TD']}}, {'Nissan': {'Terrano Ii': ['SE+', 'TDi']}}, {'Nissan': {'Tiida': ''}}, {'Nissan': {'Titan': ''}}, {'Nissan': {'X-trail': ['Acenta', 'Acenta Premium', 'Arctix Expedition Sports Adventure', 'Aventura', 'Aventura Explorer', 'Aventura Explorer Extreme', 'Columbia', 'DIG-T', 'N-Connecta', 'N-Design', 'N-Vision', 'SE', 'SVE', 'Sport', 'Sport Expedition', 'T-Spec', 'Tekna', 'Tekna SE', 'Tekna+', 'Trek', 'Visia', 'dCi', 'h e-POWER', 'i', 'n-tec', 'n-tec+']}}, {'Nissan': {'E-nv200': ['Acenta', 'Tekna']}}, {'Nissan': {'Env200 Evalia': ['Evalia', 'Tekna']}}, {'Noble': {'M12': 'GTO-3'}}, {'Opel': {'Zafira': ''}}, {'Packard': {'Sedan 4s': ''}}, {'Panther': {'Kallista': ''}}, {'Panther': {'Lima': ''}}, {'Perodua': {'Kelisa': 'EX SE'}}, {'Perodua': {'Myvi': ['EZi', 'SXi', 'SXi Sport', 'Xtra']}}, {'Peugeot': {'1007': ['Dolce', 'HDi', 'SE', 'Sport']}}, {'Peugeot': {'106': ['GTi', 'Independence', 'Rallye', 'XR', 'Zest 2']}}, {'Peugeot': {'107': ['Access', 'Active', 'Allure', 'Envy', 'Kiss', 'Millesim', 'Sport', 'Sport XS', 'Sportium', 'Urban', 'Urban Lite', 'Urban Move', 'Verve']}}, {'Peugeot': {'108': ['Access', 'Active', 'Allure', 'Collection', 'Feline', 'Feline Nav', 'GT Line', 'PureTech', 'Roland Garros', 'VTi']}}, {'Peugeot': {'2008': ['Access', 'Access+', 'Active', 'Active Premium', 'Active Premium +', 'Allure', 'Allure Premium', 'Allure Premium +', 'BlueHDi', 'Crossway', 'Feline', 'GT', 'GT Line', 'GT Premium', 'HDi', 'PureTech', 'Signature', 'Urban Cross', 'VTi', 'e-HDi', 'e-VTi']}}, {'Peugeot': {'205': ['CJ', 'CTi', 'GR', 'GTi', 'Mardi Gras', 'Roland Garros', 'XE']}}, {'Peugeot': {'206': ['Allure', 'Entice', 'GLX', 'GTi', 'GTi 180', 'HDi', 'Independence', 'LX', 'Look', 'S', 'SE', 'Sport', 'Style', 'Urban', 'Verve', 'Zest', 'Zest 3']}}, {'Peugeot': {'206 Cc': ['Allure', 'HDi', 'S', 'SE', 'Sport']}}, {'Peugeot': {'206 Sw': ['HDi', 'Quiksilver', 'S', 'Verve', 'XT']}}, {'Peugeot': {'207': ['70 Professional', 'Access', 'Active', 'Allure', 'Cielo', 'Economique', 'Envy', 'GT', 'GTi', 'HDi', 'Millesim', 'S', 'SE', 'SE Premium', 'Sport', 'Sport XS', 'Sportium', 'THP', 'Urban', 'VTi', 'Verve', 'm:play']}}, {'Peugeot': {'207 Cc': ['Active', 'Allure', 'ELLE', 'GT', 'HDi', 'Roland Garros', 'Sport', 'THP', 'VTi']}}, {'Peugeot': {'207 Sw': ['Active', 'Allure', 'HDi', 'Outdoor', 'S', 'Sport', 'VTi']}}, {'Peugeot': {'208': ['Access', 'Access+', 'Active', 'Active Design', 'Active Premium', 'Active Premium +', 'Allure', 'Allure Premium', 'Allure Premium +', 'Black Edition', 'BlueHDi', 'Feline', 'GT', 'GT Line', 'GT Premium', 'GTi', 'GTi 30th', 'GTi Prestige', 'GTi by Peugeot Sport', 'HDi', 'Ice Velvet', 'Intuitive', 'PureTech', 'Roland Garros', 'Signature', 'Style', 'THP', 'Tech Edition', 'VTi', 'XS', 'XY', 'e-HDi', 'e-VTi']}}, {'Peugeot': {'3008': ['Access', 'Active', 'Active Premium', 'Active Premium +', 'Allure', 'Allure Premium', 'Allure Premium +', 'BlueHDi', 'Crossway', 'Envy', 'Exclusive', 'GT', 'GT Line', 'GT Line Premium', 'GT Premium', 'HDi', 'PureTech', 'Roland Garros', 'SR', 'Sport', 'Sportium', 'Style', 'THP', 'VTi', 'e-HDi', 'h e-HDi']}}, {'Peugeot': {'306': ['HDi', 'Meridian']}}, {'Peugeot': {'307': ['E', 'Feline', 'GLX', 'HDi', 'LX', 'Rapier', 'S', 'SE', 'Sport', 'X-Line', 'XSi', 'Zest']}}, {'Peugeot': {'307 Cc': ['Allure', 'HDi', 'S', 'Sport']}}, {'Peugeot': {'307 Sw': ['HDi', 'Quiksilver', 'S', 'SE']}}, {'Peugeot': {'308': ['Access', 'Active', 'Active Premium', 'Allure', 'Allure Premium', 'BlueHDi', 'Envy', 'Feline', 'GT', 'GT Line', 'GT Premium', 'GTi', 'GTi by Peugeot Sport', 'HDi', 'Millesim', 'PureTech', 'S', 'SE', 'SR', 'Sport', 'Sportium', 'THP', 'Tech Edition', 'VTi', 'Verve', 'e-HDi', 'e-THP']}}, {'Peugeot': {'308 Cc': ['Access', 'Active', 'Allure', 'GT', 'HDi', 'Roland Garros', 'SE', 'Sport', 'THP', 'VTi', 'e-HDi']}}, {'Peugeot': {'308 Sw': ['Access', 'Active', 'Active Premium', 'Allure', 'Allure Premium', 'BlueHDi', 'GT', 'GT Line', 'GT Premium', 'HDi', 'PureTech', 'S', 'SE', 'SR', 'Sport', 'Tech Edition', 'VTi', 'e-HDi', 'e-THP']}}, {'Peugeot': {'4007': ['GT', 'HDi', 'SE', 'Sport XS']}}, {'Peugeot': {'406': ['HDi', 'LX', 'Rapier', 'SE']}}, {'Peugeot': {'407': ['Bellagio', 'Executive', 'GT', 'HDi', 'S', 'SE', 'SR', 'Sport']}}, {'Peugeot': {'407 Sw': ['GT', 'HDi', 'S', 'SE', 'SR', 'Sport', 'Sport XS', 'Zenith']}}, {'Peugeot': {'408': ['Allure Premium', 'First Edition', 'GT', 'PureTech']}}, {'Peugeot': {'5008': ['Access', 'Active', 'Active Premium', 'Active Premium +', 'Allure', 'Allure Premium', 'Allure Premium +', 'BlueHDi', 'Envy', 'Exclusive', 'GT', 'GT Line', 'GT Line Premium', 'GT Premium', 'HDi', 'PureTech', 'Sport', 'Style', 'THP', 'VTi', 'e-HDi']}}, {'Peugeot': {'508': ['Access', 'Active', 'Active Premium', 'Allure', 'Allure Edition', 'Allure Premium', 'BlueHDi', 'First Edition', 'GT', 'GT Line', 'GT Premium', 'HDi', 'HYbrid4', 'PureTech', 'Sport Engineered', 'THP', 'e-HDi', 'h e-HDi']}}, {'Peugeot': {'508 Sw': ['Access', 'Active', 'Active Premium', 'Allure', 'Allure Premium', 'BlueHDi', 'First Edition', 'GT', 'GT Line', 'GT Line Edition', 'GT Premium', 'HDi', 'HYbrid4', 'PureTech', 'RXH', 'SR', 'Sport Engineered', 'THP', 'e-HDi', 'h e-HDi']}}, {'Peugeot': {'607': 'Executive'}}, {'Peugeot': {'807': ['Executive', 'HDi', 'S', 'SE']}}, {'Peugeot': {'Bipper Tepee': ['HDi', 'Outdoor', 'S', 'Style']}}, {'Peugeot': {'Boxer': ['BlueHDi 335', 'Built for Business Professional', 'Professional']}}, {'Peugeot': {'Expert': ['Asphalt', 'BlueHDi', 'BlueHDi 1200', 'BlueHDi 1400', 'HDi', 'L2 H1', 'Professional', 'Professional Plus', 'Professional Premium']}}, {'Peugeot': {'Expert Tepee': ['Comfort', 'HDi', 'L1 Tepee Comfort', 'Leisure']}}, {'Peugeot': {'Partner': ['BlueHDi 744', 'BlueHDi 850', 'Combi', 'Escapade', 'HDi', 'HDi 750', 'Professional', 'Professional Premium', 'Quiksilver', 'S', 'S L2', 'S L2 744', 'S L2 750', 'Totem']}}, {'Peugeot': {'Partner Tepee': ['Active', 'Allure', 'BlueHDi', 'HDi', 'Outdoor', 'PureTech', 'Tepee Outdoor', 'Tepee S', 'Tepee Style', 'Tepee Urban', 'Tepee Zenith', 'VTi']}}, {'Peugeot': {'Rcz': ['Asphalt', 'GT', 'GT Line', 'HDi', 'Magnetic', 'R', 'Red Carbon', 'Sport', 'THP']}}, {'Peugeot': {'Rifter': ['Active', 'Allure', 'Allure Premium', 'BlueHDi', 'GT', 'GT Line', 'PureTech']}}, {'Peugeot': {'Traveller': ['Active', 'Allure', 'BlueHDi', 'Business', 'Business VIP']}}, {'Peugeot': {'E-2008': ['Active', 'Active Premium', 'Active Premium +', 'Allure', 'Allure Premium', 'Allure Premium +', 'GT', 'GT Line', 'GT Premium']}}, {'Peugeot': {'E-208': ['Active', 'Active Premium', 'Active Premium +', 'Allure', 'Allure Premium', 'Allure Premium +', 'GT', 'GT Line', 'GT Premium']}}, {'Peugeot': {'E-expert': ['Professional Premium', 'e 1000']}}, {'Peugeot': {'E-partner': ['700', 'Professional Premium']}}, {'Peugeot': {'E-rifter': ['Allure', 'Allure Premium', 'GT']}}, {'Peugeot': {'E-traveller': ['Active', 'Allure']}}, {'Peugeot': {'Ion': ''}}, {'Pilgrim': {'Sumo': ''}}, {'Polaris': {'Rzr': ''}}, {'Polestar': {'Polestar 2': ['Long Range', 'Long Range Plus Pilot', 'Standard Range']}}, {'Pontiac': {'Fiero': ''}}, {'Pontiac': {'Firebird': ''}}, {'Pontiac': {'Grand Prix': ''}}, {'Pontiac': {'Tempest': ''}}, {'Pontiac': {'Trans Am': ''}}, {'Porsche': {'356': ''}}, {'Porsche': {'550 Spyder': ''}}, {'Porsche': {'718 Boxster': ['25 Years', 'GTS', 'S', 'T']}}, {'Porsche': {'718 Cayman': ['GT4', 'GT4 RS', 'GTS', 'S', 'T']}}, {'Porsche': {'718 Spyder': ''}}, {'Porsche': {'911': ['964 Carrera 2', '964 Carrera 4', '991', '991 4', '991 4 GTS', '991 4S', '991 50th Anniversary Edition', '991 Carrera', '991 Carrera 4', '991 Carrera 4 GTS', '991 Carrera 4S', '991 Carrera Black Edition', '991 Carrera GTS', '991 Carrera S', '991 Carrera T', '991 GT2 RS', '991 GT3', '991 GT3 RS', '991 Turbo', '991 Turbo S', '991 Turbo S Exclusive Series', '992 GT3', '992 4', '992 4 GTS', '992 4S', '992 4S Heritage Design Edition', '992 Carrera', '992 Carrera 4', '992 Carrera 4 GTS', '992 Carrera 4S', '992 Carrera GTS', '992 Carrera S', '992 GT3 Touring', '992 Sport Classic', '992 Turbo', '992 Turbo S', '993 Carrera', '993 Carrera 4', '993 Carrera RS', '993 Carrera S', '993 Turbo', '996', '996 Carrera 2', '996 Carrera 4', '996 Carrera 4S', '996 GT2', '996 GT3', '996 GT3 RS', '996 Turbo', '996 Turbo S', '997', '997 4', '997 4S', '997 Black Edition', '997 Carrera', '997 Carrera 4', '997 Carrera 4 GTS', '997 Carrera 4S', '997 Carrera GTS', '997 Carrera S', '997 GT2', '997 GT2 RS', '997 GT3', '997 GT3 RS', '997 Turbo', '997 Turbo S', 'Classic Carrera', 'Classic Carrera Club Sport', 'Classic Carrera Sport', 'Classic Carrera Super Sport', 'Classic SC', 'Classic Turbo', 'T']}}, {'Porsche': {'918': ''}}, {'Porsche': {'924': 'S'}}, {'Porsche': {'928': ['GT', 'GTS', 'S', 'S Series 2', 'S Series 4']}}, {'Porsche': {'944': ['Lux', 'S', 'S2', 'Turbo']}}, {'Porsche': {'968': 'Sport'}}, {'Porsche': {'Boxster': ['981', '981 Black Edition', '981 GTS', '981 S', '986', '986 S', '987', '987 RS 60', '987 S']}}, {'Porsche': {'Cayenne': ['GTS', 'PiH V6', 'Platinum Edition', 'S', 'S Platinum Edition', 'T V6', 'T V8', 'TD V6', 'TD V8', 'TDI V6', 'Turbo', 'Turbo GT', 'Turbo S', 'V6', 'V6 E-Hybrid', 'V8 E-Hybrid', 'h V6']}}, {'Porsche': {'Cayman': ['981', '981 Black Edition', '981 GT4', '981 GTS', '981 S', '987', '987 R', '987 S', '987 Sport']}}, {'Porsche': {'Macan': ['GTS', 'S', 'T', 'T V6', 'TD V6', 'Turbo', 'Turbo Performance', 'V6']}}, {'Porsche': {'Panamera': ['4', '4 10 Years Edition', '4 Executive', '4 Platinum Edition', '4S', '4S Executive', 'D V6', 'GTS', 'Platinum Edition', 'S', 'T V6', 'T V8', 'TD V8', 'Turbo', 'Turbo Executive', 'Turbo S', 'Turbo S Executive', 'V6', 'V6 E-Hybrid', 'V8', 'V8 E-Hybrid', 'h V6']}}, {'Porsche': {'Taycan': ['4', '4S', 'GTS', 'Performance', 'Performance Plus', 'Turbo', 'Turbo S']}}, {'Proton': {'Gen-2': 'GLS'}}, {'Proton': {'Proton': 'SL'}}, {'Proton': {'Sl': ''}}, {'Proton': {'Satria': 'GTi'}}, {'Proton': {'Savvy': 'Style'}}, {'Radical': {'Rapture': ''}}, {'Rage': {'Comet': ''}}, {'Reliant': {'Regal': ''}}, {'Reliant': {'Robin': 'SLX'}}, {'Reliant': {'Scimitar': 'GTE'}}, {'Renault': {'5': ['GT', 'GT Raider', 'Gordini', 'Turbo']}}, {'Renault': {'Arkana': ['E-TECH', 'E-Tech engineered', 'Iconic', 'MHEV', 'S Edition', 'TCe', 'evolution', 'r.s. line', 'techno']}}, {'Renault': {'Captur': ['Blue dCi', 'Bose Launch Edition', 'Dynamique MediaNav', 'Dynamique Nav', 'Dynamique S MediaNav', 'Dynamique S Nav', 'E-TECH', 'E-Tech engineered', 'Expression +', 'Expression + Convenience', 'GT Line', 'Iconic', 'Iconic Edition', 'Iconic II Nav', 'Iconic Nav', 'Launch Edition', 'Limited', 'MHEV', 'Play', 'RS Line', 'Rive Gauche SE', 'S Edition', 'SE Edition', 'SE Limited', 'Signature', 'Signature Nav', 'Signature S Nav', 'Signature X Nav', 'TCe', 'TCe ENERGY', 'dCi', 'dCi ENERGY', 'evolution', 'techno']}}, {'Renault': {'Clio': ['20th Anniversary', 'Alize', 'Authentique', 'Bizu', 'Blue dCi', 'Campus', 'Campus Sport', 'Campus Sport I-Music', 'Dynamique', 'Dynamique MediaNav', 'Dynamique Nav', 'Dynamique S', 'Dynamique S MediaNav', 'Dynamique S Nav', 'Dynamique SX', 'Dynamique TomTom', 'E-TECH', 'E-Tech engineered', 'ECO', 'Etoile', 'Evolution', 'Expression', 'Expression +', 'Extreme', 'Extreme 2', 'Extreme 3', 'Extreme 4', 'GT', 'GT Line', 'GT Line Nav', 'GT Line TomTom', 'Gordini', 'Grande', 'I-Music', 'Iconic', 'Iconic 25 Nav', 'Iconic Edition', 'Initiale', 'Initiale TomTom', 'Launch Edition', 'Lutecia', 'Play', 'Privilege', 'Privilege TomTom', 'Pzaz', 'R.S.18 Nav', 'RS Line', 'RS Line Bose Edition', 'RT Champs Elysees', 'Renaultsport', 'Renaultsport Cup', 'Renaultsport Lux', 'Renaultsport Nav', 'Renaultsport Nav Lux', 'Renaultsport Nav Trophy', 'Renaultsport Raider', 'Renaultsport Silverstone GP', 'Rip Curl', 'Rush', 'S', 'S Edition', 'SCe', 'SE Edition', 'Signature Nav', 'Sport', 'TCe', 'Techno', 'TomTom', 'Urban Nav', 'V6', 'VVT', 'dCi', 'eco2']}}, {'Renault': {'Espace': ['Authentique', 'Dynamique', 'Dynamique S', 'Dynamique TomTom', 'Expression Dynamique', 'dCi']}}, {'Renault': {'Grand Espace': ['Dynamique', 'Dynamique TomTom', 'Expression', 'Initiale', 'Privilege', 'T', 'V6', 'dCi']}}, {'Renault': {'Grand Modus': ['Dynamique', 'Expression', 'I-Music', 'TCe', 'VVT', 'dCi']}}, {'Renault': {'Grand Scenic': ['Blue dCi', 'Dynamique', 'Dynamique Nav', 'Dynamique S', 'Dynamique S Nav', 'Dynamique TomTom', 'Expression', 'FAP', 'Iconic', 'Limited', 'Limited Nav', 'Oasis', 'Play', 'Privilege', 'Privilege TomTom', 'Signature', 'Signature Nav', 'TCe', 'VVT', 'dCi', 'dCi ENERGY', 'h dCi']}}, {'Renault': {'Kadjar': ['Blue dCi', 'Dynamique Nav', 'Dynamique S Nav', 'Dynamique SE Nav', 'Equilibre', 'Expression +', 'GT Line', 'Iconic', 'Play', 'S Edition', 'Signature Nav', 'Signature S Nav', 'TCe', 'Techno', 'dCi']}}, {'Renault': {'Kangoo': ['75 Authentique', 'Authentique', 'Expression', 'Extreme', 'Trekka', 'dCi']}}, {'Renault': {'Kangoo Maxi': ['Business', 'LL21 dCi', 'LL21 90', 'LL21 dCi', 'Z.E.', 'ZE', 'ZE LL21', 'dCi ENERGY LL21']}}, {'Renault': {'Koleos': ['Blue dCi', 'Dynamique', 'Dynamique S Nav', 'GT Line', 'Iconic', 'Privilege', 'Signature Nav', 'dCi']}}, {'Renault': {'Laguna': ['Dynamique', 'Dynamique S', 'Dynamique TomTom', 'Expression', 'Extreme', 'GT', 'GT Line TomTom', 'Initiale', 'Initiale TomTom', 'Monaco GP', 'Privilege', 'Privilege Nav', 'T', 'TomTom Edition', 'dCi', 'eco2']}}, {'Renault': {'Master': ['Business', 'Business+', 'dCi 35', 'dCi ENERGY 35']}}, {'Renault': {'Megane': ['Bizu', 'Blue dCi', 'Dynamique', 'Dynamique Nav', 'Dynamique S', 'Dynamique S Nav', 'Dynamique TomTom', 'E-TECH', 'Expression', 'Expression +', 'Extreme', 'FAP', 'Floride', 'GT', 'GT Line', 'GT Line Nav', 'GT Line TomTom', 'GT Nav', 'Generation', 'I-Music', 'Iconic', 'Knight Edition', 'Limited', 'Limited Nav', 'Oasis', 'Play', 'Privilege', 'Privilege TomTom', 'Pzaz', 'R.S.280', 'R.S.300', 'R.S.300 Trophy', 'R.S.300 Trophy-R', 'RS Line', 'Renaultsport', 'Renaultsport CUP-S', 'Renaultsport Cup', 'Renaultsport Nav', 'Renaultsport R26.R', 'Renaultsport Trophy', 'Renaultsport Trophy-R', 'Signature Nav', 'T', 'TCe', 'TCe ENERGY', 'TD', 'Team', 'Tech Run', 'VVT', 'dCi', 'dCi ENERGY']}}, {'Renault': {'Megane E-tech': ['EV60', 'equilibre', 'launch edition', 'techno']}}, {'Renault': {'Modus': ['Dynamique', 'Dynamique S', 'Expression', 'FAP', 'Initiale', 'Maxim', 'Oasis', 'Privilege', 'TCe', 'VVT', 'dCi']}}, {'Renault': {'Scenic': ['Authentique', 'Bizu', 'Blue dCi', 'Dynamique', 'Dynamique Nav', 'Dynamique S', 'Dynamique S Nav', 'Dynamique TomTom', 'Expression', 'Expression +', 'Extreme', 'FAP', 'Fidji', 'I-Music', 'Iconic', 'Limited', 'Limited Nav', 'Maxim', 'Oasis', 'Play', 'Privilege', 'Privilege TomTom', 'Signature', 'Signature Nav', 'Sport Alize', 'TCe', 'TCe ENERGY', 'TomTom Edition', 'VVT', 'dCi', 'dCi ENERGY', 'h dCi']}}, {'Renault': {'Scenic Xmod': ['Dynamique Nav', 'Dynamique TomTom', 'Expression +', 'TCe ENERGY', 'VVT', 'dCi', 'dCi ENERGY']}}, {'Renault': {'Trafic': ['Black Edition', 'Business', 'Business+', 'LL29', 'LL29 Navigation', 'LL29 Phase 3', 'SL27', 'SL27 Phase 3', 'SL27 Sport', 'SL29 Navigation', 'Sport', 'Sport Nav', 'Sport+', 'TD dCi', 'dCi 27', 'dCi 28', 'dCi 29', 'dCi 30', 'dCi Blue 30', 'dCi ENERGY 27', 'dCi ENERGY 28', 'dCi ENERGY 29', 'dCi ENERGY 30', 'dCi LL29', 'dCi SL27']}}, {'Renault': {'Twingo': ['Bizu', 'Dynamique', 'Dynamique S', 'Expression', 'Extreme', 'Freeway', 'GT', 'Gordini', 'I-Music', 'Iconic', 'Play', 'Pzaz', 'Renaultsport', 'Renaultsport Silverstone GP', 'SCe', 'TCe', 'TCe ENERGY', 'The Color Run', 'VVT']}}, {'Renault': {'Twizy': ['Colour', 'Dynamique', 'Technic']}}, {'Renault': {'Wind': ['Dynamique', 'Dynamique S', 'GT Line', 'TCe', 'VVT']}}, {'Renault': {'Zoe': ['Dynamique Intens', 'Dynamique Nav', 'Dynamique Zen', 'E R110', 'E R135', 'EV50', 'Expression', 'Expression Nav', 'GT', 'GT Edition', 'GT Line', 'GT Line +', 'Iconic', 'Play', 'Q90', 'R110', 'R135', 'R90', 'Riviera', 'S Edition', 'Signature Nav', 'Techno', 'i Venture Edition']}}, {'Reva': {'G-wiz': ''}}, {'Riley': {'Elf': ''}}, {'Robin Hood': {'Lighweight': ''}}, {'Robin Hood': {'Superspec': ''}}, {'Rolls-Royce': {'20/25': ''}}, {'Rolls-Royce': {'Corniche': ''}}, {'Rolls-Royce': {'Cullinan': ['Black Badge', 'V12']}}, {'Rolls-Royce': {'Dawn': ['Black Badge', 'Inspired By Fashion', 'V12']}}, {'Rolls-Royce': {'Ghost': ['Black Badge', 'V12']}}, {'Rolls-Royce': {'Limousine': ''}}, {'Rolls-Royce': {'Phantom': 'V12'}}, {'Rolls-Royce': {'Silver Cloud': ''}}, {'Rolls-Royce': {'Silver Dawn': ''}}, {'Rolls-Royce': {'Silver Seraph': ''}}, {'Rolls-Royce': {'Silver Shadow': 'II'}}, {'Rolls-Royce': {'Silver Spirit': ['II', 'III']}}, {'Rolls-Royce': {'Silver Spur': ['II', 'III', 'Muliner Park Ward']}}, {'Rolls-Royce': {'Silver Wraith': 'II'}}, {'Rolls-Royce': {'Wraith': ['Black Badge', 'V12']}}, {'Rover': {'100': ['111', '114', 'Ascot SE', 'Knightsbridge', 'Knightsbridge SE', 'SLi', 'Si']}}, {'Rover': {'200': ['214', 'SEi']}}, {'Rover': {'25': 'iL'}}, {'Rover': {'45': ['Club', 'Connoisseur', 'GSi', 'Impression', 'Impression S', 'TD', 'iXL']}}, {'Rover': {'600': ['618', '620', 'SLi', 'iL']}}, {'Rover': {'75': ['CDT', 'CDTi', 'Classic', 'Classic SE', 'Club SE', 'Connoisseur', 'Connoisseur SE', 'Contemporary', 'Contemporary SE', 'T', 'V6']}}, {'Rover': {'75 Tourer': ['Connoisseur SE', 'V6']}}, {'Rover': {'800': ['827', 'Si']}}, {'Rover': {'Cabriolet': 'SE'}}, {'Rover': {'Metro': ['GTa', 'Rio Grande']}}, {'Rover': {'Mini': ['30', '35', 'British Open Classic', 'Cooper', 'Cooper S Sport 5', 'Cooper S Works', 'Cooper Sport', 'Italian Job', 'John Cooper', 'Mayfair', 'Sidewalk', 'Sprite', 'Studio 2', 'Thirty', 'i']}}, {'Rover': {'P2': ''}}, {'Rover': {'P5': ''}}, {'Rover': {'P6': ['2000', 'SC']}}, {'Rover': {'Streetwise': 'SE'}}, {'Royale': {'Sabre': ''}}, {'SEAT': {'Alhambra': ['Ecomotive', 'FR Line', 'I TECH', 'Reference', 'S', 'SE', 'SE L', 'SE Lux', 'Stylance', 'Style Advanced', 'TDI', 'TSI', 'XCELLENCE']}}, {'SEAT': {'Altea': ['Ecomotive', 'Freetrack 4', 'I TECH', 'Reference', 'Reference Sport', 'S', 'S Emocion', 'SE', 'SE Copa', 'Sport', 'Stylance', 'TDI', 'TSI']}}, {'SEAT': {'Altea Xl': ['Ecomotive', 'FSI', 'I TECH', 'Reference', 'S', 'SE', 'SE Copa', 'Sport', 'Stylance', 'TDI']}}, {'SEAT': {'Arona': ['FR', 'FR Edition', 'FR Red Edition', 'FR Sport', 'SE', 'SE Edition', 'SE Technology', 'SE Technology Lux', 'TDI', 'TSI', 'TSI EVO', 'XCELLENCE', 'XCELLENCE Lux', 'XPERIENCE', 'XPERIENCE Lux']}}, {'SEAT': {'Arosa': 'S'}}, {'SEAT': {'Ateca': ['EcoTSI', 'Ecomotive', 'FR', 'FR Edition', 'FR Sport', 'First Edition', 'S', 'SE', 'SE L', 'SE Technology', 'TDI', 'TSI', 'TSI EVO', 'XCELLENCE', 'XCELLENCE Lux', 'XPERIENCE', 'XPERIENCE Edition', 'XPERIENCE Lux']}}, {'SEAT': {'Cupra Ateca': ['Limited Edition', 'TSI']}}, {'SEAT': {'Cupra Leon': ''}}, {'SEAT': {'Cordoba Vario': '.cool/S'}}, {'SEAT': {'Exeo': ['Ecomotive', 'SE', 'SE Lux', 'SE Tech', 'Sport', 'Sport Tech', 'TDI', 'TSI']}}, {'SEAT': {'Ibiza': ['30 Years', 'ACT', 'Bocanegra', 'Chill', 'Connect', 'Cupra', 'Cupra Black', 'E', 'EcoTSI', 'Ecomotive', 'FR', 'FR Black', 'FR Edition', 'FR Red Edition', 'FR Sport', 'FR Technology', 'Formula Sport', 'Good Stuff', 'I TECH', 'MPI', 'Reference', 'S', 'S Copa', 'SE', 'SE Copa', 'SE Design', 'SE Edition', 'SE Technology', 'SOL', 'SX', 'Sport', 'Sport Black', 'Sportrider', 'Stylance', 'TDI', 'TSI', 'TSI EVO', 'Toca', 'Turbo', 'Vista', 'XCELLENCE', 'XCELLENCE Lux']}}, {'SEAT': {'Leon': ['ACT', 'Cupra', 'Cupra 280', 'Cupra 290', 'Cupra 290 Black', 'Cupra 290 Lux', 'Cupra 300', 'Cupra 300 Lux', 'Cupra K1', 'Cupra R', 'EcoTSI', 'Ecomotive', 'Ecomotive S', 'Emocion', 'FR', 'FR Black Edition', 'FR Black Technology', 'FR First Edition', 'FR Sport', 'FR Technology', 'FR Titanium', 'FR Titanium Technology', 'FR+', 'FR+ Supercopa', 'FR550', 'FSI', 'MHEV', 'Reference', 'Reference Sport', 'S', 'S Copa', 'S Emocion', 'SE', 'SE Copa', 'SE Dynamic', 'SE Dynamic Technology', 'SE Special Edition', 'SE Technology', 'SE Technology Business', 'Sport', 'Stylance', 'TDI', 'TFSI', 'TSI', 'TSI EVO', 'X-PERIENCE SE Technology', 'XCELLENCE', 'XCELLENCE Lux', 'XCELLENCE Technology', 'eTSI']}}, {'SEAT': {'Mii': ['Design Mii', 'ECO', 'Ecomotive', 'FR Line', 'I TECH', 'Mii by Cosmopolitan', 'S', 'S AC', 'SE', 'Sport', 'Toca', 'by MANGO']}}, {'SEAT': {'Tarraco': ['EcoTSI EVO', 'FR', 'FR Sport', 'SE', 'SE First Edition', 'SE Technology', 'TDI', 'TSI', 'TSI EVO', 'XCELLENCE', 'XCELLENCE First Edition', 'XCELLENCE First Edition Plus', 'XCELLENCE Lux']}}, {'SEAT': {'Terra Vista': 'Vista'}}, {'SEAT': {'Toledo': ['Ecomotive', 'I-TECH', 'Reference', 'S', 'SE', 'Stylance', 'TDI', 'TSI', 'V5', 'XCELLENCE']}}, {'SKODA': {'Citigo': ['Black Edition', 'Colour Edition', 'Elegance', 'GreenTech', 'MPI', 'Monte Carlo', 'S', 'SE', 'SE L', 'Sport', 'iV']}}, {'SKODA': {'Enyaq': ['60', '60 Nav Lodge', '60 Nav Loft', '60 Nav Suite', '60 Nav ecoSuite', '80', '80 Founders Edition Suite', '80 Loft', '80 Lounge', '80 SportLine', '80 SportLine Plus', '80 Suite', '80 ecoSuite', '80x SportLine', '80x vRS']}}, {'SKODA': {'Fabia': ['1', '2', '3', 'Ambiente', 'Black Edition', 'Bohemia', 'Classic', 'Colour Edition', 'Comfort', 'Elegance', 'GreenLine', 'GreenLine II', 'HTP', 'MPI', 'Monte Carlo', 'Monte Carlo TECH', 'Reaction', 'RedLine', 'S', 'S Plus', 'SDI', 'SE', 'SE Comfort', 'SE Drive', 'SE L', 'SE Plus', 'Scout', 'Sport', 'TDI', 'TSI', 'vRS', 'vRS S2000']}}, {'SKODA': {'Felicia': 'GLi'}}, {'SKODA': {'Kamiq': ['ACT', 'Monte Carlo', 'S', 'SE', 'SE Drive', 'SE L', 'SE L Executive', 'TDI', 'TSI']}}, {'SKODA': {'Karoq': ['ACT', 'Edition', 'SE', 'SE Drive', 'SE L', 'SE Nav Plus', 'SE Technology', 'Scout', 'SportLine', 'TDI', 'TSI']}}, {'SKODA': {'Kodiaq': ['ACT', 'BiTDI', 'Edition', 'Laurin & Klement', 'S', 'SE', 'SE Drive', 'SE L', 'SE L Executive', 'SE Technology', 'Scout', 'SportLine', 'TDI', 'TSI', 'vRS']}}, {'SKODA': {'Octavia': ['ACT', 'Ambiente', 'Black Edition', 'Classic', 'Elegance', 'FSI', 'GreenLine II', 'GreenLine III', 'Laurin & Klement', 'MHEV', 'MPI', 'S', 'SCR', 'SE', 'SE Business', 'SE Drive', 'SE First Edition', 'SE L', 'SE L First Edition', 'SE Plus', 'SE Sport', 'SE Technology', 'Scout', 'Silverline', 'SportLine', 'TDI', 'TFSI', 'TSI', 'TSI e-TEC', 'TSI iV', 'Tour de France', 'vRS', 'vRS 230', 'vRS Blackline', 'vRS Challenge', 'vRS Limited Edition']}}, {'SKODA': {'Rapid': ['Elegance', 'GreenTech', 'S', 'SE', 'SE Connect', 'SE L', 'Sport', 'TDI', 'TSI']}}, {'SKODA': {'Rapid Spaceback': ['Black Edition', 'Elegance', 'GreenTech', 'S', 'SE', 'SE Sport', 'SE Tech', 'TDI', 'TSI']}}, {'SKODA': {'Roomster': ['1', '2', '3', 'Black Edition', 'GreenLine II', 'S', 'SE', 'SE Action', 'SE Plus', 'Scout', 'TDI', 'TSI']}}, {'SKODA': {'Scala': ['Monte Carlo', 'SE', 'SE L', 'SE Technology', 'TDI', 'TSI']}}, {'SKODA': {'Superb': ['ACT', 'Comfort', 'Elegance', 'Elegance GreenLine II', 'Elegance GreenLine III', 'GreenLine', 'Laurin & Klement', 'Outdoor', 'Outdoor Plus', 'S', 'S GreenLine II', 'S GreenLine III', 'SE', 'SE Business', 'SE L', 'SE L Executive', 'SE Plus', 'SE Technology', 'SportLine', 'SportLine Plus', 'TDI', 'TDI V6', 'TFSI', 'TSI', 'TSI iV', 'Tour de France', 'V6']}}, {'SKODA': {'Yeti': ['Adventure', 'Black Edition', 'Elegance', 'GreenLine II', 'Laurin & Klement', 'Monte Carlo', 'S', 'SE', 'SE Business', 'SE Drive', 'SE L', 'SE L Drive', 'SE Plus', 'TDI', 'TSI', 'Tour de France', 'Urban']}}, {'Saab': {'9-3': ['Aero', 'Aero Anniversary', 'Aero Carlsson', 'Aero Special Edition', 'Airflow', 'Arc', 'Cerulean Vector', 'Linear', 'Linear Anniversary', 'Linear SE', 'Linear Sport', 'Linear Sport Anniversary', 'S', 'SE', 'T', 'TTiD', 'TiD', 'Turbo', 'Turbo Edition', 'Turbo X', 'V6', 'Vector', 'Vector Anniversary', 'Vector Sport', 'Vector Sport Anniversary', 'Viggen', 'i']}}, {'Saab': {'9-3x': 'T'}}, {'Saab': {'9-5': ['Aero', 'Arc', 'Griffin', 'Linear', 'Linear SE', 'Linear Sport', 'SE', 'T', 'T V6', 'TTiD', 'TiD', 'Turbo Edition', 'Vector', 'Vector SE', 'Vector Sport', 'Vector Sport Anniversary']}}, {'Saab': {'900': ['LPT', 'S', 'S Aero', 'SE', 'i']}}, {'Saab': {'9000': ['CSE', 'EcoPower', 'Turbo']}}, {'Shelby': {'Cobra': ''}}, {'Smart': {'Roadster': ['Brabus', 'Speedsilver']}}, {'Smart': {'Forfour': ['BRABUS', 'BRABUS Sport', 'BRABUS Sport Night Sky', 'BRABUS Xclusive', 'Coolstyle', 'Disturbing London Edition', 'EQ', 'Edition 1', 'Edition Black', 'Edition White', 'Exclusive', 'Passion', 'Passion Advanced', 'Premium', 'Prime', 'Prime Exclusive', 'Prime Night Sky', 'Prime Sport', 'Prime Sport Night Sky', 'Proxy', 'Pulse', 'Pulse Premium', 'Purestyle', 'T', 'Urbanshadow']}}, {'Smart': {'Fortwo': ['BRABUS', 'BRABUS Line', 'BRABUS Sport', 'BRABUS Xclusive', 'CDI', 'City BRABUS', 'City Crossblade', 'City Grandstyle', 'City Passion', 'City Pulse', 'City Pure', 'City Spring', 'Cityflame', 'Edition 1', 'Edition Black', 'Edition Blue', 'Edition Bluedawn', 'Edition Nightsky', 'Edition White', 'Edition21', 'Electric Drive', 'Exclusive', 'Grandstyle', 'Iceshine', 'Limited Three', 'MHD', 'Passion', 'Passion Advanced', 'Premium', 'Prime', 'Prime Exclusive', 'Prime Sport', 'Proxy', 'Pulse', 'Pulse Premium', 'Pure', 'T', 'Urbanshadow']}}, {'Spyker': {'C8 Laviolette': ''}}, {'Ssangyong': {'Korando': ['60th Anniversary Edition', 'D', 'ELX', 'ELX4', 'ES', 'EX', 'GLS', 'LE', 'Pioneer', 'SE', 'SE4', 'Ultimate', 'Ventura']}}, {'Ssangyong': {'Korando Sports': ['D', 'EX', 'EXT', 'SX']}}, {'Ssangyong': {'Kyron': ['S', 'SX', 'TD']}}, {'Ssangyong': {'Musso': ['D', 'EX', 'Rebel', 'Rhino', 'Saracen']}}, {'Ssangyong': {'Rexton': ['60th Anniversary Edition', 'D', 'ELX', 'EX', 'Ice', 'S', 'SE', 'SPR', 'SX', 'TD RX 270', 'Ultimate', 'Ultimate Plus', 'Ventura', 'e-XDi']}}, {'Ssangyong': {'Rodius': ['D', 'ES', 'EX', 'S']}}, {'Ssangyong': {'Tivoli': ['D', 'ELX', 'EX', 'LE', 'P', 'SE', 'Ultimate', 'Ultimate Nav', 'Ventura', 'e-XDi', 'e-XGi']}}, {'Ssangyong': {'Tivoli Xlv': ['D', 'ELX', 'EX', 'P', 'Ultimate', 'e-XDi', 'e-XGi']}}, {'Ssangyong': {'Turismo': ['D', 'ELX', 'ES', 'EX', 'S', 'SE', 'e-XDi']}}, {'Subaru': {'Brz': ['SE', 'SE Lux', 'SE Lux Ltd', 'i']}}, {'Subaru': {'Exiga': ''}}, {'Subaru': {'Forester': ['D', 'Sport', 'X', 'XC', 'XC Premium', 'XE', 'XE Premium', 'XEn', 'XLn', 'XS', 'XSn', 'XT', 'XTE', 'XTEn', 'e-Boxer', 'i']}}, {'Subaru': {'Impreza': ['2000', 'Catalunya', 'D', 'P1', 'R', 'R Sport', 'RB5', 'RC', 'RX', 'SE', 'Series McRae', 'Sport', 'T', 'Terzo', 'Turbo', 'WRX', 'WRX S', 'WRX ST', 'WRX STI', 'WRX STI 330S', 'WRX STI WR1', 'WRX STi Type UK', 'i']}}, {'Subaru': {'Justy': 'R'}}, {'Subaru': {'Legacy': ['D', 'ES', 'GX', 'R', 'R spec.B', 'RE', 'REn', 'S', 'SE', 'i']}}, {'Subaru': {'Levorg': ['GT', 'i']}}, {'Subaru': {'Outback': ['D', 'Field', 'Limited', 'R', 'Rn', 'S', 'SE', 'SE Premium', 'SX', 'Touring', 'i']}}, {'Subaru': {'Solterra': ['Limited', 'Touring']}}, {'Subaru': {'Tribeca': ['S5', 'SE7']}}, {'Subaru': {'Wrx Sti': ['Final Edition', 'T', 'Type UK']}}, {'Subaru': {'Xv': ['D', 'S', 'SE', 'SE Lux Premium', 'SE Premium', 'e-Boxer', 'i']}}, {'Sunbeam': {'Alpine': ''}}, {'Suzuki': {'Across': ''}}, {'Suzuki': {'Alto': ['Cruz', 'GL', 'GLA', 'SZ', 'SZ2', 'SZ3', 'SZ4']}}, {'Suzuki': {'Baleno': ['Boosterjet', 'Dualjet', 'Dualjet MHEV', 'GLX', 'SZ-T', 'SZ3', 'SZ5']}}, {'Suzuki': {'Cappuccino': ''}}, {'Suzuki': {'Celerio': ['City', 'Dualjet', 'SZ2', 'SZ3', 'SZ4']}}, {'Suzuki': {'Grand Vitara': ['+', 'Attitude', 'DDiS', 'GV1600 Sport', 'SE', 'SZ3', 'SZ4', 'SZ5', 'TD', 'V6', 'VVT', 'XEC', 'XL-7']}}, {'Suzuki': {'Ignis': ['Adventure', 'Dualjet', 'Dualjet MHEV', 'GL', 'SZ-T', 'SZ3', 'SZ5', 'VVT']}}, {'Suzuki': {'Jimny': ['Adventure', 'JLX', 'JLX+', 'Mode', 'O2', 'SZ3', 'SZ4', 'SZ5', 'Special', 'VVT']}}, {'Suzuki': {'Kizashi': 'Sport'}}, {'Suzuki': {'Liana': ['GL', 'GLX', 'SE']}}, {'Suzuki': {'Sc': ''}}, {'Suzuki': {'Sx4': ['Aerio', 'DDiS', 'GL', 'GLX', 'SZ-L', 'SZ3', 'SZ4', 'SZ5', 'X-EC']}}, {'Suzuki': {'Sx4 S-cross': ['Boosterjet', 'Boosterjet MHEV', 'DDiS', 'Motion', 'SZ-T', 'SZ3', 'SZ4', 'SZ5', 'Ultra']}}, {'Suzuki': {'Splash': ['DDiS', 'GLS', 'GLS +', 'SZ2', 'SZ3', 'SZ4']}}, {'Suzuki': {'Swace': ['SZ-T', 'SZ5', 'h']}}, {'Suzuki': {'Swift': ['Attitude', 'Boosterjet', 'Boosterjet MHEV', 'DDiS', 'Dualjet', 'Dualjet MHEV', 'GL', 'GLX', 'SZ-L', 'SZ-T', 'SZ2', 'SZ3', 'SZ4', 'SZ5', 'Sport', 'Sport SZ-R', 'TD', 'VVT']}}, {'Suzuki': {'Vitara': ['Boosterjet', 'Boosterjet MHEV', 'DDiS', 'JX 4U', 'Kuro', 'S', 'SZ-T', 'SZ4', 'SZ5', 'TD']}}, {'Suzuki': {'Wagon R': ['GL', 'GL (R+)', 'GL S-Limited (R+)', 'Special (R+)']}}, {'TVR': {'3000m': ''}}, {'TVR': {'350i': ''}}, {'TVR': {'Cerbera': 'Speed Six'}}, {'TVR': {'Chimaera': 'HC'}}, {'TVR': {'Griffith': '500'}}, {'TVR': {'S3': ''}}, {'TVR': {'Sagaris': ''}}, {'TVR': {'T350': ''}}, {'TVR': {'Tamora': ''}}, {'TVR': {'Tuscan': 'S'}}, {'Tesla': {'Model 3': ['Long Range', 'Performance', 'Standard Range Plus']}}, {'Tesla': {'Model S': ['100D', '60D', '70', '70D', '75', '75D', '85', '85D', '90D', 'Executive Edition', 'Long Range', 'P100DL', 'P85', 'P85D', 'P90D', 'Performance', 'Performance Ludicrous', 'Standard Range']}}, {'Tesla': {'Model X': ['100D', '75D', '90D', 'Executive Edition', 'Long Range', 'Long Range Plus', 'P100DL', 'P90D', 'Performance', 'Signature', 'Standard Range']}}, {'Tesla': {'Model Y': ['Long Range', 'Performance']}}, {'Tesla': {'Roadster': ''}}, {'Toyota': {'Aygo': ['+', 'Active Plus', 'Black', 'Blue', 'D', 'Fire', 'Go', 'Ice', 'JBL Edition', 'Mode', 'Move', 'Move with Style', 'Platinum', 'Sport', 'VVT-i', 'x', 'x-cite', 'x-cite 3', 'x-cite 4', 'x-claim', 'x-clusiv', 'x-clusiv3', 'x-play', 'x-plore', 'x-press', 'x-pression', 'x-pure', 'x-style', 'x-trend']}}, {'Toyota': {'Alphard': ''}}, {'Toyota': {'Aqua': ''}}, {'Toyota': {'Auris': ['Active', 'Business Edition', 'Colour Collection', 'D-4D', 'Design', 'Dual VVT-i', 'Edition', 'Excel', 'GB25', 'Icon', 'Icon Plus', 'Icon Tech', 'Icon+', 'SR', 'Sport', 'T Spirit', 'T2', 'T3', 'T4', 'TR', 'V-Matic', 'VVT-h', 'VVT-i']}}, {'Toyota': {'Avensis': ['Active', 'Business Edition', 'Business Edition Plus', 'Colour Collection', 'D-4D', 'D-CAT', 'Design', 'Excel', 'GLS', 'GS', 'Icon', 'Icon Business Edition', 'Icon Plus', 'SE', 'T Spirit', 'T180', 'T2', 'T3-S', 'T3-X', 'T4', 'TD', 'TR', 'V-Matic', 'VVT-i', 'Vermont']}}, {'Toyota': {'Avensis Verso': ['D-4D', 'GLS', 'T Spirit', 'T3', 'VVT-i']}}, {'Toyota': {'Aygo X': ['Air Edition', 'Edge', 'Exclusive', 'Limited Edition', 'Pure', 'VVT-i']}}, {'Toyota': {'Bb': ''}}, {'Toyota': {'C-hr': ['Design', 'Dynamic', 'Dynamic Limited Edition', 'Excel', 'GR SPORT', 'Icon', 'Lime Edition', 'Orange Edition', 'Red Edition', 'VVT-h', 'VVT-i']}}, {'Toyota': {'Camry': ['CDX', 'Design', 'Excel', 'GLi', 'VVT-h', 'VVT-i', 'i']}}, {'Toyota': {'Carina': 'GL'}}, {'Toyota': {'Celica': ['Celica Blue', 'Celica Red', 'GT', 'GT Four', 'ST', 'Supra', 'T Sport', 'VVT-i', 'VVTL-i']}}, {'Toyota': {'Celsior': ''}}, {'Toyota': {'Chaser': ''}}, {'Toyota': {'Corolla': ['Collection', 'Colour Collection', 'D-4D', 'Design', 'Excel', 'Executive', 'GL', 'GLi', 'GR SPORT', 'GS', 'GT', 'GTi', 'Icon', 'Icon Tech', 'S', 'SE', 'SR', 'Sportif SE', 'T Spirit', 'T Sport', 'T2', 'T3', 'Trek', 'Trek Special Edition', 'VVT-h', 'VVT-i', 'VVTL-i', 'i']}}, {'Toyota': {'Corolla Verso': ['D-4D', 'SR', 'T Spirit', 'T180', 'T2', 'T3', 'TR', 'VVT-i']}}, {'Toyota': {'Crown': 'Super'}}, {'Toyota': {'Estima': ''}}, {'Toyota': {'Fj Cruiser': ''}}, {'Toyota': {'Gr86': 'Boxer D-4S'}}, {'Toyota': {'Gt86': ['Aero', 'Blanco', 'Boxer D-4S', 'Club Series Blue Edition', 'Club Series Orange Edition', 'Primo', 'Pro', 'TRD']}}, {'Toyota': {'Granvia': ''}}, {'Toyota': {'Hiace': '300 GS-Xtra'}}, {'Toyota': {'Highlander': ['Excel', 'Excel Premium', 'VVT-h']}}, {'Toyota': {'Hilux': ['240 FX', '250 EX', '270 EX', '270 VX', '280 EX', '280 GX', '4X2', 'Active', 'Active Narrow Body', 'D-4D', 'EX', 'GR SPORT', 'GX', 'HL2', 'HL3', 'Icon', 'Invincible', 'Invincible 200', 'Invincible 50', 'Invincible X', 'Invincible X AT35', 'LE']}}, {'Toyota': {'Ipsum': ''}}, {'Toyota': {'Isis': ''}}, {'Toyota': {'Land Cruiser': ['60th Anniversary', 'Active', 'D', 'D-4D', 'D-4D V8', 'II', 'Icon', 'Invincible', 'Invincible X', 'LC3', 'LC4', 'LC5', 'TD', 'VX']}}, {'Toyota': {'Land Cruiser Amazon': ['GX', 'TD', 'V8', 'VX']}}, {'Toyota': {'Land Cruiser Colorado': ['D-4D', 'GX', 'TD', 'VX']}}, {'Toyota': {'Mr2': ['GT', 'GT T-Bar', 'Roadster', 'T Bar', 'VVT-i', 'i']}}, {'Toyota': {'Mirai': 'Fuel Cell'}}, {'Toyota': {'Noah': ''}}, {'Toyota': {'Proace Verso': ['D', 'Family', 'Shuttle']}}, {'Toyota': {'Passo': ''}}, {'Toyota': {'Picnic': 'GS'}}, {'Toyota': {'Prado': ''}}, {'Toyota': {'Previa': ['D-4D', 'GL', 'GLS', 'T Spirit', 'T2', 'T3', 'VVT-i']}}, {'Toyota': {'Prius': ['Active', 'Business Edition', 'Business Edition Plus', 'Excel', 'T Spirit', 'T Spirit 10th Anniversary', 'T3', 'T4', 'VVT-h']}}, {'Toyota': {'Prius+': ['Excel', 'Excel Plus', 'Icon', 'T Spirit', 'T4', 'VVT-h']}}, {'Toyota': {'Proace': ['1200 HDi', '1200 L2H1', 'Combi', 'D', 'Design', 'Icon']}}, {'Toyota': {'Rav4': ['Active', 'Adventure', 'Black Edition', 'Business Edition', 'Business Edition Plus', 'D-4D', 'D-CAT', 'Design', 'Dynamic', 'Excel', 'GX', 'GXS', 'Granite', 'Icon', 'Icon Tech', 'Invincible', 'NRG', 'NV', 'SR', 'T180', 'V-Matic', 'VVT-h', 'VVT-i', 'VX', 'XT-R', 'XT3', 'XT4', 'XT5']}}, {'Toyota': {'Raum': ''}}, {'Toyota': {'Sera': ''}}, {'Toyota': {'Sienta': ''}}, {'Toyota': {'Soarer': ''}}, {'Toyota': {'Starlet Glanza V': ''}}, {'Toyota': {'Supra': ['GR', 'GR Fuji Speedway Edition', 'GR Jarama Race Track Edition', 'GR Pro', 'GR Special Edition', 'T', 'Turbo']}}, {'Toyota': {'Surf': ''}}, {'Toyota': {'Tundra': ''}}, {'Toyota': {'Urbancruiser': ['D-4D', 'Dual VVT-i']}}, {'Toyota': {'Vellfire': ''}}, {'Toyota': {'Verso': ['Active', 'D-4D', 'D-CAT', 'Design', 'Excel', 'Icon', 'T Spirit', 'T2', 'TR', 'Trend', 'Trend Plus', 'V-Matic']}}, {'Toyota': {'Verso S': ['Dual VVT-i', 'T Spirit', 'TR']}}, {'Toyota': {'Voxy': ''}}, {'Toyota': {'Wish': ''}}, {'Toyota': {'Yaris': ['Active', 'Blue', 'CDX', 'Colour Collection', 'D-4D', 'Design', 'Dual VVT-i', 'Dynamic', 'Edition', 'Excel', 'GLS', 'GR', 'GR Circuit', 'GR Convenience', 'GR SPORT', 'GRMN', 'GS', 'Icon', 'Icon Plus', 'Icon Tech', 'Ion', 'Launch Edition', 'Orange Edition', 'S', 'SR', 'Sport', 'T', 'T Spirit', 'T Sport', 'T2', 'T3', 'T4', 'TR', 'Trend', 'VVT-h', 'VVT-i', 'Y20', 'Y20 Bi-tone', 'Zinc']}}, {'Toyota': {'Yaris Cross': ['Design', 'Dynamic', 'Excel', 'GR SPORT', 'Icon', 'Premiere Edition', 'VVT-h']}}, {'Toyota': {'Yaris Verso': ['D-4D', 'T Spirit', 'VVT-i', 'Verso', 'Verso GLS', 'Verso GS']}}, {'Toyota': {'Bz4x': ['Motion', 'Premiere Edition', 'Pure', 'Vision']}}, {'Toyota': {'Iq': ['2', '3', 'Dual VVT-i', 'VVT-i']}}, {'Triumph': {'Gt6': ''}}, {'Triumph': {'Herald': ''}}, {'Triumph': {'Spitfire': ''}}, {'Triumph': {'Stag': ''}}, {'Triumph': {'Tr2': ''}}, {'Triumph': {'Tr4': ''}}, {'Triumph': {'Tr6': ''}}, {'Triumph': {'Tr7': ''}}, {'Ultima': {'Gtr': ''}}, {'Vauxhall': {'Adam': ['Black Edition', 'ENERGISED', 'ENERGISED Black Jack', 'GLAM', 'GRAND SLAM', 'Griffin', 'JAM', 'ROCKS', 'ROCKS AIR', 'ROCKS S', 'S', 'SLAM', 'Unlimited', 'White Edition', 'ecoFLEX', 'ecoTEC', 'i', 'i Turbo']}}, {'Vauxhall': {'Agila': ['CDTi', 'Club', 'Design', 'Enjoy', 'Expression', 'S', 'SE', 'VVT', 'ecoFLEX', 'i']}}, {'Vauxhall': {'Ampera': ['Electron', 'Positiv']}}, {'Vauxhall': {'Antara': ['CDTi', 'Diamond', 'E', 'Exclusiv', 'S', 'SE']}}, {'Vauxhall': {'Astra': ['Active', 'Active Limited Edition', 'Active Plus', 'Air', 'Atlas', 'BlueInjection', 'Breeze', 'Business Edition Nav', 'CDTi', 'CDTi BiTurbo', 'Club', 'DTi', 'Design', 'ES', 'Elegance', 'Elite', 'Elite Nav', 'Elite Nav Premium', 'Energy', 'Enjoy', 'Envoy', 'Excite', 'Exclusiv', 'GS', 'GS Line', 'GSi', 'GTE', 'Griffin', 'Griffin Edition', 'L', 'LPG', 'LS', 'Life', 'Limited Edition', 'Premier', 'S', 'SE', 'SRi', 'SRi Nav', 'SRi VX Line', 'SRi VX Line Nav', 'SXi', 'Sport', 'T', 'TD', 'Tech Line', 'Tech Line GT', 'Tech Line Nav', 'Turbo', 'Turbo D', 'TwinAir', 'Twinport', 'Ultimate', 'Ultimate Nav', 'VVT', 'VXR', 'ecoFLEX', 'ecoTEC', 'ecoTEC BlueInjection', 'i', 'i Turbo']}}, {'Vauxhall': {'Astra Gtc': ['CDTi', 'CDTi BiTurbo', 'Limited Edition', 'SRi', 'Sport', 'T', 'VXR', 'ecoFLEX', 'ecoTEC', 'i Turbo']}}, {'Vauxhall': {'Astra Van': ['CDTi', 'Club', 'DTi', 'Envoy', 'LS', 'Sportive', 'Sportive SE', 'ecoFLEX']}}, {'Vauxhall': {'Carlton': ['GSi', 'i']}}, {'Vauxhall': {'Cascada': ['CDTi', 'Elite', 'SE', 'T', 'i Turbo']}}, {'Vauxhall': {'Cavalier': ['GL', 'i']}}, {'Vauxhall': {'Combo': ['2000 CDTi', 'CDTi 2000', 'Dynamic', 'Edition', 'Sportive', 'Turbo D 2300']}}, {'Vauxhall': {'Combo Life': ['BlueInjection', 'Design', 'Edition', 'Elite', 'Energy', 'SE', 'Turbo', 'Turbo D']}}, {'Vauxhall': {'Combo Tour': ''}}, {'Vauxhall': {'Combo-e': ['2300', 'Dynamic']}}, {'Vauxhall': {'Combo-e Life': 'SE'}}, {'Vauxhall': {'Corsa': ['Active', 'Active Plus', 'Black Edition', 'Breeze', 'Breeze Plus', 'CDTi', 'Club', 'Comfort', 'Design', 'Diamond', 'ECO', 'Elegance', 'Elite', 'Elite Edition', 'Elite Nav', 'Elite Nav Premium', 'Energy', 'Excite', 'Exclusiv', 'Expression', 'GLS', 'GS Line', 'GSi', 'Griffin', 'Griffin Edition', 'LPG', 'LS', 'Life', 'Limited Edition', 'Premier', 'Red Edition', 'S', 'SE', 'SE Edition', 'SE Nav', 'SE Nav Premium', 'SE Premium', 'SRi', 'SRi Edition', 'SRi Nav', 'SRi Nav Premium', 'SRi Premium', 'SRi VX Line', 'SRi VX Line Nav Black', 'SXi', 'SXi+', 'Sport', 'Sting', 'Sting R', 'T', 'Trip', 'Turbo', 'Turbo D', 'Ultimate', 'Ultimate Nav', 'VXR', 'VXR Clubsport', 'VXR Nurburgring', 'White Edition', 'ecoFLEX', 'ecoTEC', 'i', 'i Turbo']}}, {'Vauxhall': {'Corsa Van': ['CDTi', 'L1H1', 'Limited Edition Nav', 'Sportive', 'Start/Stop ecoFLEX', 'Twinport', 'ecoFLEX']}}, {'Vauxhall': {'Corsa-e': ['Anniversary Edition', 'Elite Nav', 'Elite Nav Premium', 'Elite Premium', 'GS Line', 'Griffin', 'SE Nav', 'SE Nav Premium', 'SE Premium', 'SRi Nav Premium', 'SRi Premium', 'Ultimate']}}, {'Vauxhall': {'Crossland': ['Design', 'Elite', 'Elite Edition', 'Elite Nav', 'GS Line', 'Griffin', 'SE', 'SE Edition', 'SE Nav Premium', 'SRi Edition', 'SRi Nav', 'Turbo', 'Turbo D', 'Ultimate', 'Ultimate Nav']}}, {'Vauxhall': {'Crossland X': ['Business Edition Nav', 'Elite', 'Elite Nav', 'Griffin', 'SE', 'SE Nav', 'SRi Nav', 'Sport', 'Sport Nav Premium', 'Tech Line Nav', 'Turbo', 'Turbo D', 'Ultimate', 'ecoTEC']}}, {'Vauxhall': {'Frontera': ['Limited Edition', 'i']}}, {'Vauxhall': {'Grandland': ['Design', 'Elite', 'GS Line', 'SRi', 'Turbo', 'Turbo D', 'Ultimate']}}, {'Vauxhall': {'Grandland X': ['BlueInjection', 'Business Edition Nav', 'Business Edition Nav Premium', 'Design Line', 'Elite Nav', 'Elite Nav Premium', 'Griffin', 'Griffin Edition', 'SE', 'SE Premium', 'SRi Nav', 'Sport Nav', 'Tech Line Nav', 'Turbo', 'Turbo D', 'Ultimate', 'Ultimate Nav']}}, {'Vauxhall': {'Insignia': ['BlueInjection', 'CDTi', 'CDTi BiTurbo', 'Design', 'Design Nav', 'ES', 'Elite', 'Elite Nav', 'Energy', 'Exclusiv', 'Exclusiv Nav', 'GS Line', 'GSi', 'Limited Edition', 'Nav', 'S', 'SE', 'SE Nav', 'SRi', 'SRi Nav', 'SRi VX Line', 'SRi VX Line Nav', 'SRi VX line Red', 'SRi VX line Red Nav', 'T', 'T V6', 'Tech Line', 'Tech Line Nav', 'Turbo D', 'Ultimate Nav', 'VXR', 'VXR Nav', 'VXR SuperSport', 'VXR SuperSport Nav', 'ecoFLEX', 'ecoTEC', 'ecoTEC BlueInjection', 'i', 'i Turbo', 'i Turbo V6']}}, {'Vauxhall': {'Lotus Carlton': 'i'}}, {'Vauxhall': {'Meriva': ['Active', 'Active Limited Edition', 'Breeze', 'CDTi', 'Club', 'Design', 'Energy', 'Excite', 'Exclusiv', 'Exclusiv Limited Edition', 'Life', 'S', 'SE', 'T', 'Tech Line', 'ecoFLEX', 'i', 'i Turbo']}}, {'Vauxhall': {'Mokka': ['CDTi', 'Design', 'Elite Edition', 'Elite Nav', 'Elite Nav Premium', 'Elite Premium', 'Exclusiv', 'GS Line', 'Launch Edition', 'Limited Edition', 'S', 'SE', 'SE Edition', 'SRi', 'SRi Nav Premium', 'SRi Premium', 'T', 'Tech Line', 'Turbo', 'Turbo D', 'Ultimate', 'Ultimate Nav', 'Ultimate Nav Premium', 'ecoFLEX', 'ecoTEC', 'i', 'i Turbo']}}, {'Vauxhall': {'Mokka X': ['Active', 'CDTi', 'Design Line', 'Design Nav', 'Elite', 'Elite Nav', 'Griffin', 'Griffin Plus', 'Ultimate', 'ecoFLEX', 'ecoTEC', 'ecoTEC D', 'i', 'i Turbo']}}, {'Vauxhall': {'Mokka-e': ['Elite Nav Premium', 'Elite Premium', 'GS Line', 'Launch Edition', 'SE Nav Premium', 'SE Premium', 'SRi Nav Premium', 'SRi Premium', 'Ultimate', 'Ultimate Edition']}}, {'Vauxhall': {'Monaro': ['VXR', 'i V8']}}, {'Vauxhall': {'Movano': ['BiTurbo', 'CDTi', 'CDTi 3300', 'CDTi 3500', 'Edition', 'L2H2 F3500']}}, {'Vauxhall': {'Nova': ''}}, {'Vauxhall': {'Omega': ['CD', 'CDX', 'i', 'i V6']}}, {'Vauxhall': {'Signum': ['CDTi', 'CDTi V6', 'Elegance', 'Elite', 'Exclusiv', 'i']}}, {'Vauxhall': {'Tigra': ['Bermuda Ltd Edn', 'Blue', 'CDTi', 'Exclusiv', 'Sport', 'i']}}, {'Vauxhall': {'Vx220': ['Lightning Yellow', 'i', 'i Turbo']}}, {'Vauxhall': {'Vxr8': 'V8'}}, {'Vauxhall': {'Vectra': ['Active', 'Breeze', 'CD', 'CDTi', 'CDTi V6', 'Design', 'Elite', 'Exclusiv', 'LS', 'Life', 'SRi', 'SXi', 'VVT', 'i', 'i Turbo V6']}}, {'Vauxhall': {'Viva': ['ROCKS', 'SE', 'SE Nav', 'SL', 'ecoFLEX', 'i']}}, {'Vauxhall': {'Vivaro': ['2900 CDTi', 'BiTurbo', 'BiTurbo ecoFLEX', 'BiTurbo ecoTEC', 'CDTi', 'CDTi 2900', 'Combi+', 'Dynamic', 'Edition', 'Elite', 'Limited Edition Nav', 'Sportive', 'Sportive 2900', 'Turbo D 3100', 'ecoFLEX']}}, {'Vauxhall': {'Vivaro Life': ['BlueInjection', 'Edition L', 'Edition M', 'Elite L', 'Elite M', 'Turbo D']}}, {'Vauxhall': {'Vivaro-e': ['3100', 'Elite']}}, {'Vauxhall': {'Vivaro-e Life': ['Combi L', 'Combi M', 'Edition L', 'Elite L', 'Elite M']}}, {'Vauxhall': {'Zafira': ['Active', 'Active Plus', 'Breeze', 'Breeze Plus', 'CDTi', 'Club', 'Comfort', 'DTi', 'Design', 'Elegance', 'Elite', 'Energy', 'Excite', 'Exclusiv', 'Expression', 'Life', 'SRi', 'T', 'VXR', 'ecoFLEX', 'i']}}, {'Vauxhall': {'Zafira Tourer': ['CDTi', 'Design', 'Elite', 'Elite Nav', 'Energy', 'Exclusiv', 'SE', 'SRi', 'SRi Nav', 'T', 'Tech Line', 'Tech Line Nav', 'ecoFLEX', 'ecoTEC', 'i Turbo']}}, {'Volkswagen': {'Amarok': ['Atacama', 'Aventura', 'Aventura Black Edition', 'BiTDI', 'Black Edition', 'BlueMotion Tech', 'Canyon', 'Dark Label', 'Highline', 'Startline', 'TDI', 'TDI V6', 'Trendline', 'Ultimate']}}, {'Volkswagen': {'Arteon': ['BiTDI', 'Elegance', 'R', 'R-Line', 'R-Line Edition', 'SE', 'TDI', 'TSI', 'TSI EVO']}}, {'Volkswagen': {'Beetle': ['50s', '60s', '70s', 'BlueMotion Tech', 'Design', 'Dune', 'Fender', 'GSR', 'LHD', 'Luna', 'R-Line', 'S', 'Sola', 'Sport', 'T', 'TDI', 'TSI', 'Turbo Black', 'V5']}}, {'Volkswagen': {'Bora': ['Highline', 'SE', 'Sport', 'T', 'TDI']}}, {'Volkswagen': {'Cc': ['BlueMotion Tech', 'FSI V6', 'GT', 'GT Black Edition', 'Moda', 'R-Line', 'R-Line Black Edition', 'TDI', 'TSI']}}, {'Volkswagen': {'Caddy': ['D', 'Life', 'SD', 'TD', 'TDI', 'TSI']}}, {'Volkswagen': {'Caddy California': ['California', 'TDI']}}, {'Volkswagen': {'Caddy California Maxi': ['California Maxi', 'TDI', 'TSI']}}, {'Volkswagen': {'Caddy Camper': ''}}, {'Volkswagen': {'Caddy Life': ['BlueMotion Tech', 'TDI', 'TSI']}}, {'Volkswagen': {'Caddy Maxi': ['BlueMotion Tech', 'C20 Maxi', 'C20 Maxi Life', 'Life', 'Maxi', 'TDI', 'TDI C20', 'TDi', 'TSI']}}, {'Volkswagen': {'Caddy Maxi Life': ['BlueMotion Tech', 'Pure Drive', 'TDI', 'TSI']}}, {'Volkswagen': {'California': ['Beach', 'Beach Camper', 'Beach Edition', 'Beach Tour', 'BiTDI', 'BlueMotion Tech', 'Coast', 'Ocean', 'Ocean Edition', 'TDI', 'TSI']}}, {'Volkswagen': {'Campervan': ''}}, {'Volkswagen': {'Caravelle': ['BiTDI', 'BlueMotion Tech', 'CL', 'D', 'Edition 25', 'Executive', 'GL', 'Generation Six', 'Pure Drive', 'SE', 'TD', 'TDI', 'TSI']}}, {'Volkswagen': {'Corrado': ['G60', 'VR6']}}, {'Volkswagen': {'Eos': ['BlueMotion Tech', 'Exclusive', 'FSI', 'Individual', 'S', 'SE', 'Sport', 'TDI', 'TDI CR', 'TFSI', 'TSI']}}, {'Volkswagen': {'Fox': 'Urban Fox'}}, {'Volkswagen': {'Golf': ['Active', 'All White', 'Alltrack', 'Avantgarde', 'BlueMotion', 'BlueMotion Tech', 'Clipper', 'Colour Concept', 'Driver', 'E', 'FSI', 'GL', 'GT', 'GT Edition', 'GT Sport', 'GTD', 'GTD BlueLine', 'GTE', 'GTE Advance', 'GTE Nav', 'GTI', 'GTI Clubsport', 'GTI Clubsport 40', 'GTI Clubsport 45', 'GTI Clubsport S', 'GTI Edition 35', 'GTI Launch', 'GTI Performance', 'GTI TCR', 'Life', 'MHEV', 'Match', 'Match BlueMotion', 'Match Edition', 'Match Edition BlueMotion', 'R', 'R 20 Years', 'R-Line', 'R-Line Edition', 'R32', 'S', 'SDI', 'SE', 'SE Nav', 'Sport', 'Sportline', 'Style', 'Style Edition', 'T', 'TDI', 'TDI CR', 'TFSI', 'TSI', 'TSI EVO', 'Twist', 'V6', 'VR5', 'VR6', 'eTSI']}}, {'Volkswagen': {'Golf Plus': ['BlueMotion S', 'BlueMotion SE', 'Dune', 'FSI', 'GT', 'Luna', 'S', 'SE', 'TDI', 'TSI']}}, {'Volkswagen': {'Golf Sv': ['BlueMotion', 'BlueMotion Tech', 'GT', 'Match', 'Match Edition', 'S', 'SE', 'SE Nav', 'TDI', 'TSI', 'TSI EVO']}}, {'Volkswagen': {'Grand California': ['600', 'TDI']}}, {'Volkswagen': {'Id. Buzz': ['1ST Edition', 'Life', 'Pro']}}, {'Volkswagen': {'Id.3': ['1ST Edition', 'Business', 'City', 'Family', 'Life', 'Max', 'Pro', 'Pro Performance', 'Pro S', 'Pure Performance', 'Style', 'Tech', 'Tour']}}, {'Volkswagen': {'Id.4': ['1ST Edition', 'City', 'Family', 'GTX', 'GTX Max', 'Life', 'Max', 'Pro', 'Pro Performance', 'Pure', 'Pure Performance', 'Style', 'Style Edition']}}, {'Volkswagen': {'Id.5': ['GTX', 'Max', 'Pro', 'Pro Performance', 'Style', 'Tech']}}, {'Volkswagen': {'Jetta': ['BlueMotion Tech', 'FSI', 'GT', 'Limited Edition', 'S', 'SE', 'Sport', 'TDI', 'TSI', 'TX']}}, {'Volkswagen': {'Karmann': ''}}, {'Volkswagen': {'Lupo': ['E', 'GTI', 'S', 'SE']}}, {'Volkswagen': {'Multivan': ['Life', 'Style', 'TDI', 'TSI']}}, {'Volkswagen': {'Passat': ['Alltrack', 'BiTDI', 'BlueMotion', 'BlueMotion Tech', 'Executive', 'Executive Style', 'FSI', 'FSI V6', 'GT', 'GTE', 'GTE Advance', 'Highline', 'Highline Plus', 'R-Line', 'R-Line Edition', 'R36', 'S', 'SE', 'SE Business', 'SE Nav', 'SEL', 'Sport', 'TDI', 'TDI EVO', 'TSI', 'TSI EVO', 'Trendline', 'Turbo']}}, {'Volkswagen': {'Phaeton': ['TDI CR V6', 'TDI V10', 'TDI V6', 'V6']}}, {'Volkswagen': {'Polo': ['Active', 'BlueGT', 'BlueMotion', 'BlueMotion 1', 'BlueMotion 2', 'BlueMotion Tech', 'CL', 'Dune', 'E', 'EVO', 'FSI', 'Fox', 'GL', 'GT', 'GTI', 'GTI+', 'L', 'Life', 'Match', 'Match Edition', 'Moda', 'R-Line', 'R-Line Style', 'S', 'SDI', 'SE', 'SE Design', 'SE Tech Edition', 'SEL', 'Sport', 'Style', 'T', 'TDI', 'TSI', 'Twist', 'United', 'beats']}}, {'Volkswagen': {'Scirocco': ['BlueMotion Tech', 'GL', 'GT', 'GT Black Edition', 'GTS', 'R', 'R-Line', 'R-Line Black Edition', 'TDI', 'TSI']}}, {'Volkswagen': {'Sharan': ['BlueMotion Tech', 'Executive', 'S', 'SE', 'SE Nav', 'SEL', 'Sport', 'TDI', 'TSI']}}, {'Volkswagen': {'T-cross': ['Active', 'Black Edition', 'First Edition', 'R-Line', 'S', 'SE', 'SEL', 'TDI', 'TSI', 'TSI EVO', 'United']}}, {'Volkswagen': {'T-roc': ['Active', 'Black Edition', 'Design', 'Life', 'R', 'R-Line', 'S', 'SE', 'SEL', 'Style', 'TDI', 'TDI EVO', 'TSI', 'TSI EVO', 'United']}}, {'Volkswagen': {'Taigo': ['Life', 'R-Line', 'Style', 'TSI']}}, {'Volkswagen': {'Tiguan': ['Active', 'BiTDI', 'BlueMotion Tech', 'Elegance', 'Escape', 'Life', 'Match', 'Match Edition', 'R', 'R-Line', 'R-Line Edition', 'R-Line Tech', 'S', 'SE', 'SE Navigation', 'SEL', 'Sport', 'TDI', 'TSI', 'TSI EVO']}}, {'Volkswagen': {'Tiguan Allspace': ['BiTDI', 'Elegance', 'Life', 'Match', 'R-Line', 'R-Line Tech', 'SE Nav', 'SEL', 'TDI', 'TSI', 'TSI EVO']}}, {'Volkswagen': {'Touareg': ['Altitude', 'Black Edition', 'BlueMotion Tech', 'Escape', 'Hybrid', 'R', 'R-Line', 'R-Line Plus', 'R-Line Tech', 'R50', 'SE', 'SE Sport', 'SEL', 'SEL Tech', 'Sport', 'TDI', 'TDI V10', 'TDI V6', 'TDI V8', 'TSI V6', 'V6', 'h TSI V6']}}, {'Volkswagen': {'Touran': ['BlueMotion Tech', 'FSI', 'Match', 'R-Line', 'S', 'SE', 'SE Discover Nav', 'SE Family', 'SEL', 'Sport', 'TDI', 'TSI', 'TSI EVO']}}, {'Volkswagen': {'Transporter': ['1200', 'BiTDI', 'BiTDI T30', 'BiTDI T32', 'BlueMotion Tech', 'Edition', 'Highline', 'Sportline', 'Sportline Black Edition', 'Startline', 'Startline Business', 'T30', 'T30 Highline', 'T30 Startline', 'T30 Trendline', 'T32', 'T32 Highline', 'T32 Startline', 'TD', 'TDI', 'TDI T30', 'TDI T32', 'TSI T30', 'TSI T32', 'Trendline']}}, {'Volkswagen': {'Transporter Shuttle': ['BiTDI', 'BlueMotion Tech', 'S', 'SE', 'T30 S', 'T30 SE', 'T32 SE', 'TD', 'TDI']}}, {'Volkswagen': {'Transporter Sportline': ['BiTDI', 'BiTDI T32', 'BlueMotion Tech', 'Sportline', 'Sportline 60 T32', 'Sportline T30', 'Sportline T32', 'TDI']}}, {'Volkswagen': {'Vento': ''}}, {'Volkswagen': {'Xl1': ''}}, {'Volkswagen': {'E-golf': 'e-Golf'}}, {'Volkswagen': {'E-transporter': 'e 110'}}, {'Volkswagen': {'E-up!': 'e-up!'}}, {'Volkswagen': {'Up!': ['Black Edition', 'BlueMotion Tech', 'Club up!', 'GTI', 'Groove up!', 'High up!', 'Look up!', 'Move up!', 'Move up! Tech Edition', 'R-Line', 'Rock up!', 'Street up!', 'TSI', 'Take up!', 'White Edition', 'beats', 'up!', 'up! Black', 'up! GTI', 'up! White', 'up! beats']}}, {'Volvo': {'164': ''}}, {'Volvo': {'240': ['DL', 'GL', 'GLE']}}, {'Volvo': {'340': 'GLE'}}, {'Volvo': {'440': ['GL', 'SE']}}, {'Volvo': {'480': 'ES'}}, {'Volvo': {'740': 'GL'}}, {'Volvo': {'850': ['GLT', 'R', 'S', 'SE', 'T5-R']}}, {'Volvo': {'940': ['Celebration', 'Classic', 'GLE', 'LPT', 'SE', 'Torslanda']}}, {'Volvo': {'960': 'SE'}}, {'Volvo': {'C30': ['D', 'D2', 'D3', 'D4', 'D5', 'DRIVe', 'ES', 'R-Design', 'R-Design Lux', 'R-Design SE', 'R-Design SE Sport', 'R-Design Sport', 'S', 'SE', 'SE Lux', 'SE Sport', 'T5', 'i']}}, {'Volvo': {'C40': ['Core', 'Plus', 'Pro', 'Twin', 'Ultimate']}}, {'Volvo': {'C70': ['D', 'D3', 'D4', 'D5', 'ES', 'GT', 'Ice White', 'S', 'SE', 'SE Lux', 'SE Lux Solstice', 'Sport', 'T', 'T5', 'i']}}, {'Volvo': {'P1800': ''}}, {'Volvo': {'S40': ['D', 'D2', 'D3', 'D5', 'DRIVe', 'ES', 'R-Design', 'R-Design Edition', 'R-Design SE', 'R-Design Sport', 'S', 'SE', 'SE Edition', 'SE Lux', 'SE Lux Edition', 'Sport', 'XS', 'i']}}, {'Volvo': {'S60': ['B5', 'Business Edition', 'Business Edition Lux', 'D', 'D2', 'D3', 'D4', 'D5', 'DRIVe', 'ES Nav', 'Inscription', 'Inscription Plus', 'MHEV', 'Plus', 'Polestar Engineered', 'R', 'R-Design', 'R-Design Edition', 'R-Design Lux', 'R-Design Lux Nav', 'R-Design Nav', 'R-Design Plus', 'S', 'SE', 'SE Lux', 'SE Lux Nav', 'SE Lux Premium', 'SE Nav', 'Sport', 'T', 'T3', 'T4', 'T5', 'T6', 'Twin Engine', 'Ultimate', 'h T8']}}, {'Volvo': {'S80': ['D', 'D2', 'D3', 'D4', 'D5', 'DRIVe', 'Executive', 'S', 'SE', 'SE Lux', 'SE Nav', 'SE Sport', 'T', 'V8']}}, {'Volvo': {'S90': ['D4', 'D5', 'Executive', 'Inscription', 'Inscription Plus', 'Inscription Pro', 'Momentum', 'Momentum Plus', 'Momentum Pro', 'Plus Bright', 'Plus Dark', 'PowerPulse', 'R-Design', 'R-Design Plus', 'R-Design Pro', 'T4', 'T5', 'Twin Engine', 'Ultimate', 'h T8']}}, {'Volvo': {'Suv (2022 - )': ['Electric Twin 111kWh', 'Ultra']}}, {'Volvo': {'V40': ['D2', 'D3', 'D4', 'ES', 'ES Nav', 'Inscription', 'Inscription Edition', 'Momentum', 'Momentum Edition', 'Momentum Nav Plus', 'R-Design', 'R-Design Edition', 'R-Design Lux', 'R-Design Lux Nav', 'R-Design Nav', 'R-Design Nav Plus', 'R-Design Pro', 'S', 'SE', 'SE Lux', 'SE Lux Nav', 'SE Nav', 'T2', 'T3', 'T4', 'T5']}}, {'Volvo': {'V40 Cross Country': ['D2', 'D3', 'D4', 'Edition', 'Lux', 'Lux Nav', 'Nav Plus', 'Pro', 'SE', 'SE Nav', 'T3', 'T4', 'T5']}}, {'Volvo': {'V50': ['D', 'D2', 'D3', 'D5', 'DRIVe', 'ES', 'R-Design', 'R-Design Edition', 'R-Design SE', 'R-Design Sport', 'S', 'SE', 'SE Edition', 'SE Lux', 'SE Lux Edition', 'SE Sport', 'T5']}}, {'Volvo': {'V60': ['B3', 'B4', 'B5', 'B6', 'Business Edition', 'Business Edition Lux', 'Core', 'D', 'D2', 'D3', 'D4', 'D5', 'D6', 'DRIVe', 'ES', 'ES Nav', 'Inscription', 'Inscription Plus', 'Inscription Pro', 'MHEV', 'Momentum', 'Momentum Plus', 'Momentum Pro', 'Plus', 'Plus Bright', 'Polestar', 'Polestar Engineered', 'R-Design', 'R-Design Lux', 'R-Design Lux Nav', 'R-Design Nav', 'R-Design Plus', 'R-Design Premium', 'R-Design Pro', 'SE', 'SE Lux', 'SE Lux Nav', 'SE Lux Premium', 'SE Nav', 'SE Premium', 'T3', 'T4', 'T5', 'T6', 'Twin Engine', 'Ultimate', 'h T6', 'h T8']}}, {'Volvo': {'V60 Cross Country': ['B4', 'B5', 'D3', 'D4', 'Lux Nav', 'MHEV', 'Plus', 'SE Nav', 'T5']}}, {'Volvo': {'V70': ['Business Edition', 'D', 'D2', 'D3', 'D4', 'D5', 'DRIVe', 'ES', 'R', 'R-Design', 'R-Design SE', 'S', 'SE', 'SE Lux', 'SE Lux RTI Nav', 'SE Nav', 'SE Premium', 'SE RTI Nav', 'SE Sport', 'Sport', 'T', 'T4', 'T5', 'T6', 'TDi', 'XLT', 'XT']}}, {'Volvo': {'V70 Cross Country': 'T'}}, {'Volvo': {'V90': ['B4', 'B5', 'B6', 'Core', 'D4', 'D5', 'Inscription', 'Inscription Plus', 'Inscription Pro', 'MHEV', 'Momentum', 'Momentum Plus', 'Momentum Pro', 'Plus', 'Plus Bright', 'Plus Dark', 'PowerPulse', 'R-Design', 'R-Design Plus', 'R-Design Pro', 'T4', 'T5', 'T6', 'T8', 'Twin Engine', 'Ultimate', 'h T6', 'h T8']}}, {'Volvo': {'V90 Cross Country': ['B4', 'B5', 'B6', 'D4', 'D5', 'MHEV', 'Plus', 'PowerPulse', 'Pro', 'T5', 'Ultimate', 'Volvo Ocean Race']}}, {'Volvo': {'XC40': ['B3', 'B4', 'B5', 'Core', 'D3', 'D4', 'First Edition', 'Inscription', 'Inscription Expression', 'Inscription Pro', 'MHEV', 'Momentum', 'Momentum Core', 'Momentum Pro', 'P8', 'Plus', 'Pro', 'R-Design', 'R-Design Pro', 'Start', 'T2', 'T3', 'T4', 'T5', 'Twin Engine', 'Ultimate', 'Ultimate Bright', 'Ultimate Dark', 'h T4', 'h T5']}}, {'Volvo': {'XC60': ['B4', 'B5', 'B6', 'Core', 'D', 'D3', 'D4', 'D5', 'DRIVe', 'ES', 'Edition', 'Inscription', 'Inscription Expression', 'Inscription Pro', 'MHEV', 'Momentum', 'Momentum Pro', 'Plus', 'Polestar Engineered', 'PowerPulse', 'R-Design', 'R-Design Lux', 'R-Design Lux Nav', 'R-Design Nav', 'R-Design Premium', 'R-Design Pro', 'R-Design SE', 'R-Design SE Premium', 'S', 'SE', 'SE Lux', 'SE Lux Nav', 'SE Lux Premium', 'SE Nav', 'SE Premium', 'T4', 'T5', 'T6', 'T8', 'Twin Engine', 'Ultimate', 'Ultimate Bright', 'Ultimate Dark', 'h T6', 'h T8']}}, {'Volvo': {'XC70': ['D', 'D3', 'D4', 'D5', 'DRIVe', 'Ocean Race', 'SE', 'SE Lux', 'SE Lux Premium', 'SE Nav', 'SE Sport', 'T', 'T6']}}, {'Volvo': {'XC90': ['Active', 'Active Premium', 'B5', 'B6', 'Core', 'D', 'D5', 'ES', 'ES Premium', 'Executive', 'Inscription', 'Inscription Expression', 'Inscription Pro', 'MHEV', 'Momentum', 'Momentum Pro', 'Plus', 'PowerPulse', 'R-Design', 'R-Design Nav', 'R-Design Pro', 'R-Design SE', 'SE', 'SE Lux', 'SE Lux Premium', 'SE Nav', 'SE Premium', 'SE Sport', 'SE Volvo Ocean Race', 'T', 'T5', 'T6', 'Twin Engine', 'Ultimate', 'Ultimate Bright', 'Ultimate Dark', 'V8', 'h T8']}}, {'Westfield': {'1800': 'Sport'}}, {'Westfield': {'Mega': ''}}, {'Westfield': {'Seight': ''}}, {'Westfield': {'Se': ''}}, {'Westfield': {'Sei': ''}}, {'Westfield': {'Sport': ''}}, {'Westfield': {'Xi': ''}}, {'Zimmer': {'Golden Spirit': ''}}]


In [29]:

make_model_variant_AutoTrader_lower = []
for make_model in make_model_variant_AutoTrader:
    dict_lower = eval(repr(make_model).lower())
    make_model_variant_AutoTrader_lower.append(dict_lower)
#print(make_model_variant_AutoTrader_lower)
for make_model in make_model_variant_AutoTrader_lower:
    for key in make_model.keys():
        print(key)

ac
ak
abarth
abarth
abarth
abarth
abarth
abarth
abarth
abarth
abarth
alfa romeo
alfa romeo
alfa romeo
alfa romeo
alfa romeo
alfa romeo
alfa romeo
alfa romeo
alfa romeo
alfa romeo
alfa romeo
alfa romeo
alfa romeo
alfa romeo
alfa romeo
alfa romeo
alfa romeo
alfa romeo
alfa romeo
alfa romeo
alpine
ariel
ariel
ariel
aston martin
aston martin
aston martin
aston martin
aston martin
aston martin
aston martin
aston martin
aston martin
aston martin
aston martin
aston martin
aston martin
aston martin
aston martin
aston martin
aston martin
aston martin
aston martin
aston martin
aston martin
audi
audi
audi
audi
audi
audi
audi
audi
audi
audi
audi
audi
audi
audi
audi
audi
audi
audi
audi
audi
audi
audi
audi
audi
audi
audi
audi
audi
audi
audi
audi
audi
audi
audi
audi
audi
audi
audi
audi
audi
audi
audi
audi
audi
audi
audi
audi
audi
audi
audi
audi
audi
audi
audi
audi
audi
audi
audi
audi
audi
audi
austin
austin
austin
austin
austin
austin
austin
austin
austin
austin
bmw
bmw
bmw
bmw
bmw
bmw
bmw
bmw
bmw
bm

In [16]:
for make_model in make_model_variant_AutoTrader:
    #print(make_model)
    if 'BMW' in make_model:
        print(make_model['BMW'])
    

{'1 Series': ['114d', '114i', '116d', '116d ED', '116i', '118d', '118i', '120d', '120i', '123d', '125d', '125i', '128ti', '130i', '135i', 'BluePerformance', 'ED Plus', 'ES', 'Edition ES', 'EfficientDynamics', 'EfficientDynamics Business', 'Exclusive Edition', 'M', 'M Sport', 'M Sport LE', 'M Sport Shadow Edition', 'M135i', 'M140i', 'Performance Edition', 'SE', 'SE Business', 'Shadow Edition', 'Sport', 'Sport Plus Edition', 'Urban']}
{'1502': ''}
{'2 Series': ['218d', '218i', '220d', '220i', '225d', '228i', '230i', 'Luxury', 'M Sport', 'M235i', 'M240i', 'MHT', 'Modern', 'SE', 'Sport']}
{'2 Series Active Tourer': ['216d', '218d', '218i', '220d', '220i', '223i', '225i', '225xe', '230xe', 'Luxury', 'M Sport', 'MHT', 'SE', 'Sport']}
{'2 Series Gran Coupe': ['218d', '218i', '220d', '220i', 'M Sport', 'M235i', 'Sport']}
{'2 Series Gran Tourer': ['216d', '218d', '218i', '220d', '220i', 'Luxury', 'M Sport', 'SE', 'Sport']}
{'2002': ''}
{'3 Series': ['316', '316d', '316i', '316ti', '318', '318 E

In [ ]:

overall_model_variant_df = pd.DataFrame(columns=['Manufacturer', 'Model', 'Variant(s)'])
for index in range(len(makes_on_autoTrader)):
    #print(makes_on_autoTrader[index])
    for i in make_model_variant_AutoTrader:
        manufacturer = makes_on_autoTrader[index]
        model_variant = i.get(makes_on_autoTrader[index])
        model = []
        if model_variant is not None:
            for key in model_variant.keys():
                model.append(key)
            model_val = model[0]
            
            make_model_variant = {"Manufacturer":manufacturer, "Model":model_val, "Variant(s)":model_variant[model_val]}
            #print(make_model_variant)
            
            #model_variant_df = pd.DataFrame.from_dict(make_model_variant, orient = 'index')
            #print(model_variant_df)
            overall_model_variant_df = pd.concat([overall_model_variant_df, pd.DataFrame.from_dict([make_model_variant])], axis = 0)
        else:
            pass

with pd.ExcelWriter(file_path,mode="a",engine="openpyxl",if_sheet_exists="overlay") as writer:
    overall_model_variant_df.to_excel(writer, sheet_name="Model_Variant",header=True, startrow=writer.sheets["Model_Variant"].max_row,index=False)

In [ ]:
dict_test = {'Cobra': ['MK IV', 'MK IV CRS']}
list(dict_test.keys())[0]

In [ ]:
make_model_variant_df = pd.DataFrame.from_dict(make_model_variant_AutoTrader, orient = 'index')

In [111]:
print(makes_on_autoTrader)

['AC', 'AK', 'Abarth', 'Alfa Romeo', 'Alpine', 'Ariel', 'Aston Martin', 'Audi', 'Austin', 'BMW', 'Beauford', 'Bentley', 'Bowler', 'Bristol', 'Bugatti', 'Cfmoto', 'CUPRA', 'Cadillac', 'Caterham', 'Chesil', 'Chevrolet', 'Chrysler', 'Citroen', 'Corvette', 'Crendon', 'Custom Vehicle', 'DFSK', 'DS Automobiles', 'Dacia', 'Daewoo', 'Daihatsu', 'Daimler', 'Datsun', 'DAX', 'De Tomaso', 'Dodge', 'Ferrari', 'Fiat', 'Ford', 'GMC', 'Gwm Ora', 'Gardener', 'Great Wall', 'Hillman', 'Holden', 'Honda', 'Hummer', 'Hyundai', 'Infiniti', 'Isuzu', 'Iveco', 'JBA', 'Jaguar', 'Jeep', 'Jensen', 'KIA', 'Koenigsegg', 'LEVC', 'Lada', 'Lamborghini', 'Lancia', 'Land Rover', 'Lexus', 'Leyland', 'Lincoln', 'London Taxis International', 'Lotus', 'Maxus', 'MEV', 'MG', 'MINI', 'MK', 'Moke', 'Maserati', 'Maybach', 'Mazda', 'McLaren', 'Mercedes-Benz', 'Mercury', 'Microcar', 'Mitsubishi', 'Morgan', 'Morris', 'Nosmoke', 'Nissan', 'Noble', 'Opel', 'Packard', 'Panther', 'Perodua', 'Peugeot', 'Pilgrim', 'Polaris', 'Polestar', '

In [88]:
len(make_model_variant_AutoTrader)

1379

In [13]:
len(makes_on_autoTrader)

127

In [93]:
manufacturer_df = pd.DataFrame(makes_on_autoTrader)